In [ ]:
!pip install -U vnstock
!pip install vnstock ipywidgets pandas matplotlib
!pip install ta
!pip install vnstock pandas numpy ta tensorflow scikit-learn matplotlib
!pip install requests beautifulsoup4 pandas lxml
!pip install selenium
!pip install plotly
!pip install vnstock pandas matplotlib ipywidgets ta sklearn xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=5dd02c3118b0440865a5221c7658ad908ed1eb32e758c62ad76654610a14bc7a
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 27.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-gener

# Phân tích kỹ thuật: Quét toàn sàn. *Lấy dữ liệu từ https://ezsearch.fpts.com.vn/Services/EzData/Home.aspx*

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
import logging
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import requests

warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")
logging.getLogger("vnstock").setLevel(logging.CRITICAL)
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.CRITICAL)

log_file = "/content/stock_forecast.log"
with open(log_file, 'w') as f:
    f.write("Bắt đầu ghi log...\n")
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s', filename=log_file, filemode='a')

error_summary = Counter()

# 1. Lấy danh sách mã cổ phiếu từ API FPTS
def get_stock_symbols_from_cafef():
    # URL API lấy dữ liệu chứng khoán
    api_url = "https://ezir.fpts.com.vn/ThongTinDoanhNghiep/ThongTinCompanyViEn?culture=vi-VN"

    try:
        # Gửi request lấy dữ liệu JSON
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        # Chuyển đổi JSON thành DataFrame
        df = pd.DataFrame(data)

        # Lọc chỉ các mã chứng khoán (loại bỏ trái phiếu, OTC)
        df = df[df["aposT_TO"].isin(["HOSE", "HNX", "UPCOM"])]

        # Giữ lại các cột quan trọng và đổi tên
        df = df[["astock_CODE", "aposT_TO"]]
        df.rename(columns={"astock_CODE": "Mã CK", "aposT_TO": "Sàn"}, inplace=True)

        # Lấy danh sách mã chứng khoán
        symbols = df["Mã CK"].unique().tolist()
        print(f"Tổng số mã từ API FPTS: {len(symbols)}")

        # Lưu vào file CSV
        df.to_csv("filtered_stock_data.csv", index=False)
        print("Dữ liệu đã được lưu vào filtered_stock_data.csv")

        return symbols

    except requests.exceptions.RequestException as e:
        logging.error(f"Lỗi khi gọi API: {e}")
        return []
    except Exception as e:
        logging.error(f"Lỗi khi xử lý dữ liệu từ API: {e}")
        return []

# 2. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 14 or not all(col in df.columns for col in required_columns):
            error_summary["Dữ liệu không đủ 14 ngày"] += 1
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        error_summary["Lỗi API từ vnstock"] += 1
        return None

# 3. Chỉ báo kỹ thuật (cho phân tích toàn bộ danh sách)
def calculate_indicators(df, symbol="Unknown"):
    if df is None or df.empty or len(df) < 14:
        error_summary["Dữ liệu không đủ 14 ngày"] += 1
        return None
    try:
        required_columns = ["close", "high", "low", "volume"]
        if any(col not in df.columns or df[col].isna().any() for col in required_columns):
            error_summary["Thiếu cột hoặc chứa NaN"] += 1
            return None

        indicators = {
            "SMA_10": ta.trend.sma_indicator(df["close"], window=10),
            "EMA_10": ta.trend.ema_indicator(df["close"], window=10),
            "MACD": ta.trend.MACD(df["close"], window_fast=12, window_slow=26, window_sign=9).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=12, window_slow=26, window_sign=9).macd_signal(),
            "RSI_14": ta.momentum.rsi(df["close"], window=14),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=14, smooth_window=3),
            "CCI_10": ta.trend.cci(df["high"], df["low"], df["close"], window=10),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "CMF_10": ta.volume.chaikin_money_flow(df["high"], df["low"], df["close"], df["volume"], window=10),
            "Volume_MA_5": ta.trend.sma_indicator(df["volume"], window=5),
            "ATR_14": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=14),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=10, window_dev=2),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=10, window_dev=2),
            "Momentum_10": df["close"].diff(10),
            "ADX_14": ta.trend.adx(df["high"], df["low"], df["close"], window=14)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty or df.isna().all().any():
            error_summary["Dữ liệu trống sau tính toán"] += 1
            return None
        return df
    except Exception as e:
        error_summary["Lỗi khác"] += 1
        return None

# 4. Alligator (cho phân tích toàn bộ danh sách)
def add_alligator(df, symbol="Unknown"):
    if df is None or df.empty or len(df) < 14:
        error_summary["Dữ liệu không đủ 14 ngày"] += 1
        return None
    try:
        if df["close"].isna().any():
            error_summary["Thiếu cột hoặc chứa NaN"] += 1
            return None
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty or df.isna().all().any():
            error_summary["Dữ liệu trống sau tính toán"] += 1
            return None
        return df
    except Exception as e:
        error_summary["Lỗi khác"] += 1
        return None

# 5. Chuẩn bị dữ liệu phân loại (cho phân tích toàn bộ danh sách)
def prepare_classification_data(df, symbol="Unknown"):
    if df is None or df.empty or df.isna().all().any():
        error_summary["Thiếu cột hoặc chứa NaN"] += 1
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty or len(df) < 20:
            error_summary["Dữ liệu trống hoặc quá ít sau loại NaN"] += 1
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        error_summary["Chỉ có một lớp dữ liệu hoặc lỗi khác"] += 1
        return None, None, None

# 6. Dự báo và backtesting (cho phân tích toàn bộ danh sách)
def forecast_and_backtest(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        df = calculate_indicators(df, symbol)
        if df is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        df = add_alligator(df, symbol)
        if df is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        if 'RSI_14' not in df.columns:
            error_summary["Lỗi khác"] += 1
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df, symbol)
        if X_train_5 is None or X_train_10 is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        models = {
            "Logistic Regression": LogisticRegression(max_iter=1000, C=0.5, random_state=42),
            "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=5, random_state=42),
            "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=4, subsample=0.8, colsample_bytree=0.8, random_state=42)
        }

        predictions_5 = {}
        predictions_10 = {}
        accuracy_5 = {}
        accuracy_10 = {}

        for name, model in models.items():
            if len(np.unique(y_train_5)) < 2:
                predictions_5[name] = np.unique(y_train_5)[0]
                accuracy_5[name] = 0
            else:
                model.fit(X_train_5, y_train_5)
                y_pred_5 = model.predict(X_test_5)
                latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
                predictions_5[name] = model.predict(latest_data)[0]
                accuracy_5[name] = accuracy_score(y_test_5, y_pred_5)

            if len(np.unique(y_train_10)) < 2:
                predictions_10[name] = np.unique(y_train_10)[0]
                accuracy_10[name] = 0
            else:
                model.fit(X_train_10, y_train_10)
                y_pred_10 = model.predict(X_test_10)
                latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
                predictions_10[name] = model.predict(latest_data)[0]
                accuracy_10[name] = accuracy_score(y_test_10, y_pred_10)

        buy_votes_5 = sum(1 for name, pred in predictions_5.items() if pred == 1 and accuracy_5[name] >= 0.6)
        buy_votes_10 = sum(1 for name, pred in predictions_10.items() if pred == 1 and accuracy_10[name] >= 0.6)
        total_valid_models_5 = sum(1 for acc in accuracy_5.values() if acc >= 0.6)
        total_valid_models_10 = sum(1 for acc in accuracy_10.values() if acc >= 0.6)

        final_rec = "N/A"
        if total_valid_models_5 > 0 or total_valid_models_10 > 0:
            final_rec = "MUA" if (buy_votes_5 >= 2 or buy_votes_10 >= 2) else "BÁN"

        avg_accuracy_5 = np.mean(list(accuracy_5.values())) if accuracy_5 else 0
        avg_accuracy_10 = np.mean(list(accuracy_10.values())) if accuracy_10 else 0

        rf_model = models["Random Forest"]
        rf_model.fit(X_train_5, y_train_5)
        feature_importance = pd.DataFrame({"Feature": feature_columns, "Importance": rf_model.feature_importances_})
        top_features = feature_importance.sort_values("Importance", ascending=False).head(5).to_dict()

        return {
            "Symbol": symbol,
            "Final Recommendation": final_rec,
            "Buy Votes 5 Days": buy_votes_5,
            "Buy Votes 10 Days": buy_votes_10,
            "Accuracy_5": avg_accuracy_5,
            "Accuracy_10": avg_accuracy_10,
            "Top_Features": top_features
        }
    except Exception as e:
        error_summary["Lỗi khác"] += 1
        return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

# 7. Xử lý song song cho toàn bộ danh sách
def process_all_symbols(symbols, days_window=120):
    end_date = datetime.now().strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=days_window)).strftime("%Y-%m-%d")
    print(f"Đang xử lý dự báo xu hướng 5-10 ngày với 120 ngày lịch sử từ {start_date} đến {end_date}...")

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda s: forecast_and_backtest(s, start_date, end_date), symbols))

    recommendations = [r for r in results if r]
    return pd.DataFrame(recommendations)

# 8. Lấy top 10 mã "MUA" hoặc "BÁN"
def get_top_recommendations(df, recommendation_type="MUA"):
    df["Total Buy Votes"] = df["Buy Votes 5 Days"] + df["Buy Votes 10 Days"]
    df["Avg Accuracy"] = (df["Accuracy_5"] + df["Accuracy_10"]) / 2

    filtered_df = df[df["Final Recommendation"] == recommendation_type].copy()

    if filtered_df.empty:
        print(f"Không có mã nào được khuyến nghị '{recommendation_type}'.")
        return pd.DataFrame()

    top_df = filtered_df.sort_values(by=["Total Buy Votes", "Avg Accuracy"], ascending=[False, False])
    return top_df.head(10)[["Symbol", "Final Recommendation", "Buy Votes 5 Days", "Buy Votes 10 Days",
                            "Accuracy_5", "Accuracy_10", "Avg Accuracy", "Total Buy Votes"]]

# 9. Chỉ báo kỹ thuật (cho phân tích mã cụ thể)
def calculate_indicators_specific(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_3": ta.trend.ema_indicator(df["close"], window=3),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "Chaikin_Vol": (df["high"] - df["low"]).ewm(span=5, adjust=False).mean(),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "Daily_VWAP": (df["close"] * df["volume"]) / df["volume"],
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5),
            "StochRSI": ta.momentum.stochrsi(close=df["close"], window=5, smooth1=3, smooth2=3)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi tính chỉ báo: {e}")
        return None

# 10. Alligator (cho phân tích mã cụ thể)
def add_alligator_specific(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi thêm Alligator: {e}")
        return None

# 11. Chuẩn bị dữ liệu phân loại (cho phân tích mã cụ thể)
def prepare_classification_data_specific(df):
    if df is None or df.empty:
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty:
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        print(f"Lỗi khi chuẩn bị dữ liệu: {e}")
        return None, None, None

# 12. Dự báo và backtesting (cho phân tích mã cụ thể)
def forecast_and_backtest_specific(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators_specific(df)
    if df is None:
        return None

    df = add_alligator_specific(df)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data_specific(df)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for pred in predictions_5.values() if pred == 1)
    buy_votes_10 = sum(1 for pred in predictions_10.values() if pred == 1)
    final_rec = "MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN"

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df
    }

# 13. So sánh kết quả với các khoảng thời gian (cho mã cụ thể)
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}
    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest_specific(symbol, custom_start_date, end_date)

    start_180 = (end_dt - timedelta(days=180)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 180 ngày từ {start_180} đến {end_date}...")
    results["180 ngày"] = forecast_and_backtest_specific(symbol, start_180, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest_specific(symbol, start_120, end_date)

    start_90 = (end_dt - timedelta(days=90)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 90 ngày từ {start_90} đến {end_date}...")
    results["90 ngày"] = forecast_and_backtest_specific(symbol, start_90, end_date)

    start_60 = (end_dt - timedelta(days=60)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 60 ngày từ {start_60} đến {end_date}...")
    results["60 ngày"] = forecast_and_backtest_specific(symbol, start_60, end_date)

    periods = ["Toàn bộ từ ngày chọn", "180 ngày", "120 ngày", "90 ngày", "60 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else (180 if period == "180 ngày" else (120 if period == "120 ngày" else (90 if period == "90 ngày" else 60)))

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.applymap(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 14. Giao diện chính với widget
def run_program():
    end_date = datetime.now().strftime("%Y-%m-%d")
    output = widgets.Output()

    # Phân tích toàn bộ danh sách
    symbols_list = get_stock_symbols_from_cafef()
    if not symbols_list:
        print("Không thể lấy danh sách mã cổ phiếu.")
        return

    print(f"Tổng số mã đưa vào phân tích: {len(symbols_list)}")
    recommendations_df = process_all_symbols(symbols_list, days_window=120)

    if not recommendations_df.empty:
        recommendations_df.to_csv("recommendations_120_days_ml_optimized_new_indicators.csv", index=False)
        print(f"Kết quả dự báo xu hướng 5-10 ngày với 120 ngày lịch sử đã được lưu vào recommendations_120_days_ml_optimized_new_indicators.csv")
        print("Kết quả khuyến nghị:")
        print(recommendations_df[["Symbol", "Final Recommendation", "Buy Votes 5 Days", "Buy Votes 10 Days",
                                 "Accuracy_5", "Accuracy_10", "Top_Features"]])

        successful = len(recommendations_df[recommendations_df["Final Recommendation"].isin(["MUA", "BÁN"])])
        unsuccessful = len(recommendations_df[recommendations_df["Final Recommendation"] == "N/A"])
        buy_count = len(recommendations_df[recommendations_df["Final Recommendation"] == "MUA"])
        total_analyzed = len(recommendations_df)

        print(f"\nTóm tắt kết quả:")
        print(f"Số mã phân tích thành công: {successful}/{len(symbols_list)}")
        print(f"Số mã không thành công: {unsuccessful}")
        print(f"Tỷ lệ khuyến nghị MUA: {buy_count/total_analyzed*100:.2f}%")

        top_buy = get_top_recommendations(recommendations_df, "MUA")
        top_sell = get_top_recommendations(recommendations_df, "BÁN")

        print("\nTop 10 mã khuyến nghị 'MUA' cao nhất:")
        if not top_buy.empty:
            print(top_buy)
        else:
            print("Không có mã nào.")

        print("\nTop 10 mã khuyến nghị 'BÁN' cao nhất:")
        if not top_sell.empty:
            print(top_sell)
        else:
            print("Không có mã nào.")

    # Giao diện chọn mã cụ thể
    date_picker = widgets.DatePicker(description='Chọn ngày bắt đầu:', value=datetime(2023, 1, 1))
    symbol_input = widgets.Text(description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')
    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output:
            clear_output()
            custom_start_date = date_picker.value.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")
            compare_predictions(symbol, custom_start_date, end_date)
            display(widgets.HBox([button_continue_yes, button_continue_no]))

    def on_yes_clicked(b):
        with output:
            clear_output()
            display(date_picker, symbol_input, button_confirm, output)

    def on_no_clicked(b):
        with output:
            clear_output()
            print("Đã thoát chương trình.")

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    with output:
        print("\nPhân tích mã cụ thể sau khi hoàn tất phân tích toàn bộ danh sách:")
        display(date_picker, symbol_input, button_confirm, output)

    display(output)

# 15. Thực thi chương trình
if __name__ == "__main__":
    run_program()

Dự báo một mã và phân tích tổng quan

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datetime import datetime, timedelta
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")

# 1. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 5 or not all(col in df.columns for col in required_columns):
            print(f"Dữ liệu không đủ cho {symbol} (ít hơn 5 ngày hoặc thiếu cột).")
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")
        return None

# 2. Chỉ báo kỹ thuật
def calculate_indicators(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_3": ta.trend.ema_indicator(df["close"], window=3),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "Chaikin_Vol": (df["high"] - df["low"]).ewm(span=5, adjust=False).mean(),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "Daily_VWAP": (df["close"] * df["volume"]) / df["volume"],
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5),
            "StochRSI": ta.momentum.stochrsi(close=df["close"], window=5, smooth1=3, smooth2=3)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi tính chỉ báo: {e}")
        return None

# 3. Alligator
def add_alligator(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi thêm Alligator: {e}")
        return None

# 4. Chuẩn bị dữ liệu phân loại
def prepare_classification_data(df):
    if df is None or df.empty:
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty:
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        print(f"Lỗi khi chuẩn bị dữ liệu: {e}")
        return None, None, None

# 5. Dự báo và backtesting
def forecast_and_backtest(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators(df)
    if df is None:
        return None

    df = add_alligator(df)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 5 ngày. Bỏ qua {name}.")
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 10 ngày. Bỏ qua {name}.")
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for pred in predictions_5.values() if pred == 1)
    buy_votes_10 = sum(1 for pred in predictions_10.values() if pred == 1)
    final_rec = "MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN"

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df
    }

# 6. So sánh kết quả với "Toàn bộ từ ngày chọn" và "120 ngày"
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}

    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest(symbol, custom_start_date, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest(symbol, start_120, end_date)

    periods = ["Toàn bộ từ ngày chọn", "120 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else 120

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.map(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 7. Hàm phân tích tài chính
def get_financial_data(symbol, year=2024):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        income_statement = stock.finance.income_statement(period="year", lang="en")
        balance_sheet = stock.finance.balance_sheet(period="year", lang="en")

        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        company_info = company.overview()
        if company_info.empty or 'outstanding_share' not in company_info.columns:
            print(f"Không thể lấy thông tin cổ phiếu đang lưu hành cho {symbol}, dùng giá trị mặc định")
            outstanding_share = 1000000000
        else:
            outstanding_share = company_info['outstanding_share'].iloc[0] * 1_000_000

        # Lấy thông tin ngành
        industry = company_info['industry'].iloc[0] if not company_info.empty and 'industry' in company_info.columns else "Khác"

        revenue_col = "Revenue (Bn. VND)"
        net_profit_col = "Attributable to parent company"
        current_assets_col = "CURRENT ASSETS (Bn. VND)"
        current_liabilities_col = "CURRENT LIABILITIES (Bn. VND)"
        long_term_assets_col = "LONG-TERM ASSETS (Bn. VND)"
        short_term_debt_col = "Short-term borrowings (Bn. VND)"
        long_term_debt_col = "Long-term borrowings (Bn. VND)"
        equity_col = "Common shares (Bn. VND)"
        profit_before_tax_col = "Profit before tax"
        interest_expense_col = "Interest Expenses"

        if income_statement[income_statement["yearReport"] == year].empty or balance_sheet[balance_sheet["yearReport"] == year].empty:
            print(f"Không có dữ liệu tài chính cho năm {year} của {symbol}")
            return None

        required_income_cols = [revenue_col, net_profit_col, profit_before_tax_col, interest_expense_col]
        required_balance_cols = [current_assets_col, long_term_assets_col, short_term_debt_col, long_term_debt_col, equity_col]

        missing_income_cols = [col for col in required_income_cols if col not in income_statement.columns]
        missing_balance_cols = [col for col in required_balance_cols if col not in balance_sheet.columns]

        if missing_income_cols or missing_balance_cols:
            print(f"Thiếu cột trong income_statement: {missing_income_cols}, balance_sheet: {missing_balance_cols}")
            return None

        revenue = income_statement[income_statement["yearReport"] == year][revenue_col].iloc[-1]
        net_profit = income_statement[income_statement["yearReport"] == year][net_profit_col].iloc[-1]
        profit_before_tax = income_statement[income_statement["yearReport"] == year][profit_before_tax_col].iloc[-1]
        interest_expense = abs(income_statement[income_statement["yearReport"] == year][interest_expense_col].iloc[-1])

        ebit = profit_before_tax + interest_expense

        current_assets = balance_sheet[balance_sheet["yearReport"] == year][current_assets_col].iloc[-1]
        long_term_assets = balance_sheet[balance_sheet["yearReport"] == year][long_term_assets_col].iloc[-1]
        short_term_debt = balance_sheet[balance_sheet["yearReport"] == year][short_term_debt_col].iloc[-1]
        long_term_debt = balance_sheet[balance_sheet["yearReport"] == year][long_term_debt_col].iloc[-1]
        equity = balance_sheet[balance_sheet["yearReport"] == year][equity_col].iloc[-1]

        if current_liabilities_col in balance_sheet.columns:
            current_liabilities = balance_sheet[balance_sheet["yearReport"] == year][current_liabilities_col].iloc[-1]
            current_ratio = current_assets / current_liabilities if current_liabilities > 0 else float("inf")
        else:
            print(f"Cột {current_liabilities_col} không tồn tại, dùng Short-term borrowings thay thế")
            current_liabilities = short_term_debt
            current_ratio = current_assets / current_liabilities if current_liabilities > 0 else float("inf")

        total_assets = current_assets + long_term_assets
        total_debt = total_assets - equity if equity_col in balance_sheet.columns else short_term_debt + long_term_debt
        roe = (net_profit / equity) * 100 if equity > 0 else 0
        roa = (net_profit / total_assets) * 100 if total_assets > 0 else 0
        interest_coverage = ebit / interest_expense if interest_expense > 0 else float("inf")

        end_date = "2024-12-31"
        start_date = "2024-12-01"
        price_data = get_stock_data(symbol, start_date, end_date)
        if price_data is not None and not price_data.empty:
            price = price_data["close"].iloc[-1] * 1000
            eps = net_profit / outstanding_share if outstanding_share > 0 else 0
            pe_ratio = price / eps if eps != 0 else 0
        else:
            pe_ratio = 0

        return {
            "revenue": revenue,
            "net_profit": net_profit,
            "total_assets": total_assets,
            "total_debt": total_debt,
            "pe_ratio": pe_ratio,
            "current_ratio": current_ratio,
            "roe": roe,
            "roa": roa,
            "interest_coverage": interest_coverage,
            "equity": equity,
            "outstanding_share": outstanding_share,
            "industry": industry
        }
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu tài chính cho {symbol}: {e}")
        return None
# 8. Phân tích tài chính
def analyze_financials(financial_data, symbol):
    if not financial_data or any(v is None for v in financial_data.values()):
        return {
            "Assessment": "N/A",
            "Financial Score": 0,
            "Detailed Assessment": "Không có dữ liệu để phân tích.",
            "Comments": []
        }

    revenue = financial_data["revenue"]
    net_profit = financial_data["net_profit"]
    total_assets = financial_data["total_assets"]
    total_debt = financial_data["total_debt"]
    pe_ratio = financial_data["pe_ratio"]
    current_ratio = financial_data["current_ratio"]
    roe = financial_data["roe"]
    roa = financial_data["roa"]
    interest_coverage = financial_data["interest_coverage"]
    equity = financial_data["equity"]
    industry = financial_data["industry"]

    net_profit_margin = (net_profit / revenue) * 100 if revenue > 0 else 0
    debt_to_equity = total_debt / equity if equity > 0 else float("inf")

    # Trọng số theo yêu cầu
    weights = {
        "Net Profit Margin": 2.0,
        "Debt to Equity": 1.5,
        "P/E Ratio": 1.0,
        "Current Ratio": 1.5,
        "ROE": 2.0,
        "ROA": 1.5,
        "Interest Coverage": 0.5
    }

    # Ngưỡng ngành theo yêu cầu
    industry_thresholds = {
        "Bất động sản": {"npm": 10, "dte": 2, "pe": 20, "cr": 1, "roe": 8, "roa": 3, "ic": 2},
        "Công nghệ": {"npm": 15, "dte": 1, "pe": 25, "cr": 1.5, "roe": 15, "roa": 10, "ic": 5},
        "Ngân hàng": {"npm": 20, "dte": 10, "pe": 15, "cr": 1, "roe": 12, "roa": 1, "ic": 3},
        "Khác": {"npm": 10, "dte": 1, "pe": 15, "cr": 1, "roe": 10, "roa": 5, "ic": 3}
    }

    th = industry_thresholds.get(industry, industry_thresholds["Khác"])

    # Tính điểm theo yêu cầu
    financial_score = 0
    if net_profit_margin > th["npm"]:
        financial_score += weights["Net Profit Margin"]
    if debt_to_equity < th["dte"]:
        financial_score += weights["Debt to Equity"]
    if pe_ratio > 0 and pe_ratio < th["pe"]:
        financial_score += weights["P/E Ratio"]
    if current_ratio > th["cr"]:
        financial_score += weights["Current Ratio"]
    if roe > th["roe"]:
        financial_score += weights["ROE"]
    if roa > th["roa"]:
        financial_score += weights["ROA"]
    if interest_coverage > th["ic"]:
        financial_score += weights["Interest Coverage"]

    # Phân loại theo yêu cầu
    assessment = "Tích cực" if financial_score >= 7 else "Trung lập" if financial_score >= 4 else "Tiêu cực"

    # Nhận xét chi tiết (Detailed Assessment)
    detailed_assessment = f"Ngành: {industry}\n"
    detailed_assessment += f"- Biên lợi nhuận ròng: {net_profit_margin:.2f}% (Ngưỡng ngành: {th['npm']}%). "
    detailed_assessment += "Rất tốt" if net_profit_margin > th["npm"] else "Cần cải thiện" if net_profit_margin < th["npm"] * 0.5 else "Ổn."
    detailed_assessment += f"\n- Tỷ lệ nợ/vốn: {debt_to_equity:.2f} (Ngưỡng ngành: {th['dte']}). "
    detailed_assessment += "An toàn" if debt_to_equity < th["dte"] else "Rủi ro cao" if debt_to_equity > th["dte"] * 1.5 else "Chấp nhận được."
    detailed_assessment += f"\n- P/E: {pe_ratio:.2f} (Ngưỡng ngành: {th['pe']}). "
    detailed_assessment += "Hấp dẫn" if pe_ratio < th["pe"] and pe_ratio > 0 else "Cao" if pe_ratio > th["pe"] * 1.5 else "Bình thường."
    detailed_assessment += f"\n- Thanh khoản ngắn hạn: {current_ratio:.2f} (Ngưỡng ngành: {th['cr']}). "
    detailed_assessment += "Tốt" if current_ratio > th["cr"] else "Yếu" if current_ratio < 1 else "Ổn."
    detailed_assessment += f"\n- ROE: {roe:.2f}% (Ngưỡng ngành: {th['roe']}%). "
    detailed_assessment += "Hiệu quả cao" if roe > th["roe"] else "Thấp" if roe < 0 else "Bình thường."
    detailed_assessment += f"\n- ROA: {roa:.2f}% (Ngưỡng ngành: {th['roa']}%). "
    detailed_assessment += "Hiệu quả" if roa > th["roa"] else "Thấp" if roa < 0 else "Ổn."
    detailed_assessment += f"\n- Khả năng trả lãi: {interest_coverage:.2f} (Ngưỡng ngành: {th['ic']}). "
    detailed_assessment += "Mạnh" if interest_coverage > th["ic"] else "Yếu" if interest_coverage < 1 else "Ổn."

    # Thêm phần Comments dưới dạng danh sách
    comments = [
        f"**Sức khỏe tài chính**: {symbol} đạt điểm {financial_score:.2f}/10.00, đánh giá '{assessment}' trong ngành {industry}.",
        f"**Net Profit Margin ({net_profit_margin:.2f}%)**: {'Cao hơn' if net_profit_margin > th['npm'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['npm']}%), cho thấy khả năng sinh lời {'vượt trội' if net_profit_margin > th['npm'] else 'cần cải thiện' if net_profit_margin < th['npm'] * 0.5 else 'ổn định'}.",
        f"**Debt to Equity ({debt_to_equity:.2f})**: {'Dưới' if debt_to_equity < th['dte'] else 'Trên hoặc bằng'} ngưỡng ngành ({th['dte']}), {'ít phụ thuộc vào nợ, rủi ro tài chính thấp' if debt_to_equity < th['dte'] else 'rủi ro tài chính cao' if debt_to_equity > th['dte'] * 1.5 else 'mức nợ chấp nhận được'}.",
        f"**P/E Ratio ({pe_ratio:.2f})**: {'Trong khoảng hợp lý' if 0 < pe_ratio < th['pe'] else 'Không hợp lý hoặc không xác định'} (<{th['pe']}), định giá cổ phiếu {'hấp dẫn' if 0 < pe_ratio < th['pe'] else 'cao hoặc không xác định'}.",
        f"**Current Ratio ({current_ratio:.2f})**: {'Vượt' if current_ratio > th['cr'] else 'Dưới hoặc bằng'} ngưỡng ngành ({th['cr']}), khả năng thanh toán ngắn hạn {'tốt' if current_ratio > th['cr'] else 'yếu' if current_ratio < 1 else 'ổn định'}.",
        f"**ROE ({roe:.2f}%)**: {'Cao hơn' if roe > th['roe'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['roe']}%), hiệu quả sử dụng vốn chủ sở hữu {'vượt trội' if roe > th['roe'] else 'thấp' if roe < 0 else 'bình thường'}.",
        f"**ROA ({roa:.2f}%)**: {'Vượt' if roa > th['roa'] else 'Dưới hoặc bằng'} ngưỡng ngành ({th['roa']}%), sử dụng tài sản {'hiệu quả' if roa > th['roa'] else 'thấp' if roa < 0 else 'ổn định'}.",
        f"**Interest Coverage ({interest_coverage:.2f})**: {'Cao hơn' if interest_coverage > th['ic'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['ic']}), khả năng trả lãi vay {'tốt' if interest_coverage > th['ic'] else 'yếu' if interest_coverage < 1 else 'ổn định'}."
    ]

    return {
        "Net Profit Margin": round(net_profit_margin, 2),
        "Debt to Equity": round(debt_to_equity, 2) if debt_to_equity != float("inf") else "N/A",
        "P/E Ratio": round(pe_ratio, 2) if pe_ratio != float("inf") else "N/A",
        "Current Ratio": round(current_ratio, 2) if current_ratio != float("inf") else "N/A",
        "ROE": round(roe, 2),
        "ROA": round(roa, 2),
        "Interest Coverage": round(interest_coverage, 2) if interest_coverage != float("inf") else "N/A",
        "Financial Score": financial_score,
        "Assessment": assessment,
        "Detailed Assessment": detailed_assessment,
        "Comments": comments
    }
# 9. Phân tích cổ đông lớn
def analyze_major_shareholders(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        shareholders = company.shareholders()
        if shareholders.empty or 'share_own_percent' not in shareholders.columns:
            print(f"Dữ liệu cổ đông lớn cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_major_share = shareholders['share_own_percent'].sum() * 100  # Chuyển đổi từ tỷ lệ sang phần trăm
        top_shareholder = shareholders['share_own_percent'].iloc[0] * 100

        if total_major_share > 50:
            assessment = "Cao - Rủi ro quản trị cao, nhưng có thể tăng giá nếu cổ đông lớn mua thêm."
        elif total_major_share > 30:
            assessment = "Trung bình - Ảnh hưởng đáng kể từ cổ đông lớn."
        else:
            assessment = "Thấp - Quyền kiểm soát phân tán, ít rủi ro quản trị."

        return {
            "Total Major Share (%)": round(total_major_share, 2),
            "Top Shareholder (%)": round(top_shareholder, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích cổ đông lớn cho {symbol}: {e}")
        return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 10. Phân tích mức độ gắn bó của lãnh đạo
def analyze_management_commitment(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        officers = company.officers()
        if officers.empty or 'ownPercent' not in officers.columns:
            print(f"Dữ liệu lãnh đạo cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_management_share = officers['ownPercent'].sum()

        if total_management_share > 5:
            assessment = "Cao - Lãnh đạo có cam kết mạnh mẽ với công ty."
        elif total_management_share > 1:
            assessment = "Trung bình - Cam kết ở mức khá."
        else:
            assessment = "Thấp - Lãnh đạo ít gắn bó về mặt sở hữu."

        return {
            "Management Ownership (%)": round(total_management_share, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích mức độ gắn bó của lãnh đạo cho {symbol}: {e}")
        return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 11. Phân tích công ty con
def analyze_subsidiaries(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        subsidiaries = company.subsidiaries()
        if subsidiaries.empty or 'sub_own_percent' not in subsidiaries.columns:
            print(f"Dữ liệu công ty con cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        num_subsidiaries = len(subsidiaries)
        avg_ownership = subsidiaries['sub_own_percent'].mean() * 100  # Chuyển đổi từ tỷ lệ sang phần trăm

        if num_subsidiaries > 5 and avg_ownership > 50:
            assessment = "Đa dạng hóa tốt - Tiềm năng lợi nhuận từ công ty con cao."
        elif num_subsidiaries > 2:
            assessment = "Đa dạng hóa trung bình - Tiềm năng lợi nhuận khá."
        else:
            assessment = "Đa dạng hóa thấp - Ít phụ thuộc vào công ty con."

        return {
            "Number of Subsidiaries": num_subsidiaries,
            "Average Ownership (%)": round(avg_ownership, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích công ty con cho {symbol}: {e}")
        return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 12. Phát hiện tín hiệu nội bộ
def detect_insider_signals(symbol, days=30):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        insider_deals = company.insider_deals()
        if insider_deals.empty or 'deal_quantity' not in insider_deals.columns:
            print(f"Dữ liệu giao dịch nội bộ cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

        insider_deals['deal_announce_date'] = pd.to_datetime(insider_deals['deal_announce_date'])
        recent_deals = insider_deals[insider_deals['deal_announce_date'] > datetime.now() - timedelta(days=days)]

        if recent_deals.empty:
            return {"Net Insider Volume": 0, "Assessment": "Trung lập - Không có giao dịch gần đây."}

        net_volume = recent_deals['deal_quantity'].sum()  # Tổng khối lượng mua/bán (dương là mua, âm là bán)

        if net_volume > 0:
            assessment = "Tích cực - Lãnh đạo mua ròng, dấu hiệu tốt cho triển vọng."
        elif net_volume < 0:
            assessment = "Tiêu cực - Lãnh đạo bán ròng, cần thận trọng."
        else:
            assessment = "Trung lập - Không có tín hiệu rõ ràng."

        return {
            "Net Insider Volume": int(net_volume),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phát hiện tín hiệu nội bộ cho {symbol}: {e}")
        return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

# 13. Dự đoán biến động giá quanh sự kiện
def predict_event_impact(symbol, days_before=5, days_after=5):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        events = company.events()
        if events.empty or 'exer_date' not in events.columns:
            print(f"Dữ liệu sự kiện cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

        events['exer_date'] = pd.to_datetime(events['exer_date'])
        recent_event = events[events['exer_date'] <= datetime.now()].iloc[-1]
        event_date = recent_event['exer_date']
        event_desc = recent_event['event_desc']

        start_date = (event_date - timedelta(days=days_before)).strftime("%Y-%m-%d")
        end_date = (event_date + timedelta(days=days_after)).strftime("%Y-%m-%d")
        price_data = get_stock_data(symbol, start_date, end_date)

        if price_data is None or len(price_data) < days_before + days_after:
            print(f"Không đủ dữ liệu giá quanh sự kiện cho {symbol}")
            return {"Event Date": event_date.strftime("%Y-%m-%d"), "Event Description": event_desc, "Price Change (%)": "N/A", "Assessment": "Không đủ dữ liệu giá"}

        pre_event_price = price_data['close'].iloc[days_before - 1]
        post_event_price = price_data['close'].iloc[-1]
        price_change = ((post_event_price - pre_event_price) / pre_event_price) * 100

        assessment = "Tăng giá" if price_change > 0 else "Giảm giá"

        return {
            "Event Date": event_date.strftime("%Y-%m-%d"),
            "Event Description": event_desc,
            "Price Change (%)": round(price_change, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi dự đoán biến động giá quanh sự kiện cho {symbol}: {e}")
        return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 14. Cập nhật tin tức nóng
def get_latest_news(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        news = company.news()
        if news.empty or 'publish_date' not in news.columns:
            print(f"Dữ liệu tin tức cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

        news['publish_date'] = pd.to_datetime(news['publish_date'])
        latest_news = news.iloc[0]

        return {
            "Date": latest_news['publish_date'].strftime("%Y-%m-%d"),
            "Title": latest_news['title'],
            "Source": latest_news['source']
        }
    except Exception as e:
        print(f"Lỗi khi lấy tin tức cho {symbol}: {e}")
        return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

# 15. Phân tích dòng tiền và cảnh báo bất thường
def analyze_cash_flow(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

        avg_volume = df['volume'].mean()
        latest_volume = df['volume'].iloc[-1]
        volume_spike = latest_volume > avg_volume * 2
        volume_warning = latest_volume > avg_volume * 1.5

        if volume_spike:
            assessment = "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền."
        elif volume_warning:
            assessment = "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình."
        else:
            assessment = "Không - Dòng tiền ổn định."

        return {
            "Data Range": f"{start_date} đến {end_date}",
            "Average Volume": round(avg_volume, 0),
            "Latest Volume": round(latest_volume, 0),
            "Volume Spike": volume_spike,
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích dòng tiền cho {symbol}: {e}")
        return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}
# CSS để định dạng bảng
TABLE_STYLE = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-size: 14px;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""
# 16. Chạy trên Colab với widget và tích hợp phân tích tài chính (cập nhật giao diện bảng)
def run_prediction():
    end_date = datetime.now().strftime("%Y-%m-%d")
    output = widgets.Output()

    date_picker = widgets.DatePicker(
        description='Chọn ngày bắt đầu:',
        value=datetime(2023, 1, 1),
        disabled=False
    )

    symbol_input = widgets.Text(
        description='Mã chứng khoán:',
        placeholder='Nhập mã (hoặc "exit" để thoát)',
    )

    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output:
            clear_output()
            custom_start_date = date_picker.value
            if custom_start_date is None:
                print("Vui lòng chọn ngày bắt đầu!")
                return
            custom_start_date = custom_start_date.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")
            compare_predictions(symbol, custom_start_date, end_date)

            # Phân tích tài chính
            print(f"\nPHÂN TÍCH TÀI CHÍNH CHO MÃ {symbol} NĂM 2024:")
            financial_data = get_financial_data(symbol, year=2024)
            if financial_data is None:
                print(f"Không thể lấy dữ liệu tài chính cho {symbol}")
            else:
                # Dữ liệu tài chính thô
                financial_df = pd.DataFrame([{
                    "Doanh thu (Bn. VND)": f"{financial_data['revenue'] / 1e9:,.2f}",
                    "Lợi nhuận ròng (Bn. VND)": f"{financial_data['net_profit'] / 1e9:,.2f}",
                    "Tổng tài sản (Bn. VND)": f"{financial_data['total_assets'] / 1e9:,.2f}",
                    "Tổng nợ (Bn. VND)": f"{financial_data['total_debt'] / 1e9:,.2f}",
                    "Vốn chủ sở hữu (Bn. VND)": f"{financial_data['equity'] / 1e9:,.2f}",
                    "Số cổ phiếu lưu hành": f"{financial_data['outstanding_share']:,.0f}"
                }])
                print("Dữ liệu tài chính thô:")
                display(HTML(TABLE_STYLE + financial_df.to_html(index=False)))

                # Kết quả phân tích tài chính
                financial_result = analyze_financials(financial_data, symbol)
                financial_result_df = pd.DataFrame([{
                    key: financial_result[key] for key in financial_result if key not in ["Detailed Assessment", "Comments"]
                }])
                print("\nKết quả phân tích tài chính:")
                styled_financial_df = financial_result_df.style.map(
                    lambda val: 'color: green' if val == "Tích cực" else 'color: orange' if val == "Trung lập" else 'color: red' if val == "Tiêu cực" else 'color: black',
                    subset=["Assessment"]
                )
                display(HTML(TABLE_STYLE + styled_financial_df.to_html(index=False)))

                # Nhận xét chi tiết dạng bảng
                comments_df = pd.DataFrame(financial_result["Comments"], columns=["Nhận xét chi tiết"])
                print("\nNhận xét chi tiết:")
                display(HTML(TABLE_STYLE + comments_df.to_html(index=False, escape=False)))

            # Phân tích cổ đông lớn
            print(f"\nPHÂN TÍCH CỔ ĐÔNG LỚN CHO MÃ {symbol}:")
            shareholders_result = analyze_major_shareholders(symbol)
            shareholders_df = pd.DataFrame([shareholders_result])
            display(HTML(TABLE_STYLE + shareholders_df.to_html(index=False)))

            # Phân tích mức độ gắn bó của lãnh đạo
            print(f"\nPHÂN TÍCH MỨC ĐỘ GẮN BÓ CỦA LÃNH ĐẠO CHO MÃ {symbol}:")
            management_result = analyze_management_commitment(symbol)
            management_df = pd.DataFrame([management_result])
            display(HTML(TABLE_STYLE + management_df.to_html(index=False)))

            # Phân tích công ty con
            print(f"\nPHÂN TÍCH CÔNG TY CON CHO MÃ {symbol}:")
            subsidiaries_result = analyze_subsidiaries(symbol)
            subsidiaries_df = pd.DataFrame([subsidiaries_result])
            display(HTML(TABLE_STYLE + subsidiaries_df.to_html(index=False)))

            # Phân tích tín hiệu nội bộ
            print(f"\nPHÂN TÍCH TÍN HIỆU NỘI BỘ (30 NGÀY GẦN NHẤT) CHO MÃ {symbol}:")
            insider_result = detect_insider_signals(symbol)
            insider_df = pd.DataFrame([insider_result])
            display(HTML(TABLE_STYLE + insider_df.to_html(index=False)))

            # Dự đoán biến động giá quanh sự kiện
            print(f"\nDỰ ĐOÁN BIẾN ĐỘNG GIÁ QUANH SỰ KIỆN GẦN NHẤT CHO MÃ {symbol}:")
            event_result = predict_event_impact(symbol)
            event_df = pd.DataFrame([event_result])
            display(HTML(TABLE_STYLE + event_df.to_html(index=False)))

            # Tin tức nóng
            print(f"\nTIN TỨC NÓNG GẦN NHẤT CHO MÃ {symbol}:")
            news_result = get_latest_news(symbol)
            news_df = pd.DataFrame([news_result])
            display(HTML(TABLE_STYLE + news_df.to_html(index=False)))

            # Phân tích dòng tiền
            print(f"\nPHÂN TÍCH DÒNG TIỀN VÀ CẢNH BÁO BẤT THƯỜNG CHO MÃ {symbol}:")
            cash_flow_result = analyze_cash_flow(symbol, custom_start_date, end_date)
            cash_flow_df = pd.DataFrame([cash_flow_result])
            styled_cash_flow_df = cash_flow_df.style.map(
                lambda val: 'color: red' if val == "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền." else 'color: orange' if val == "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình." else 'color: black',
                subset=["Assessment"]
            )
            display(HTML(TABLE_STYLE + styled_cash_flow_df.to_html(index=False)))

            display(widgets.HBox([button_continue_yes, button_continue_no]))

    def on_yes_clicked(b):
        with output:
            clear_output()
            display(date_picker, symbol_input, button_confirm, output)

    def on_no_clicked(b):
        with output:
            clear_output()
            print("Đã thoát chương trình.")

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    display(date_picker, symbol_input, button_confirm, output)

if __name__ == "__main__":
    run_prediction()

Nhiều năm

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datetime import datetime, timedelta
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import logging
logging.getLogger("vnstock").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.ERROR)
pio.renderers.default = "colab"
# 1. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 5 or not all(col in df.columns for col in required_columns):
            print(f"Dữ liệu không đủ cho {symbol} (ít hơn 5 ngày hoặc thiếu cột).")
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")
        return None

# 2. Chỉ báo kỹ thuật
def calculate_indicators(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_3": ta.trend.ema_indicator(df["close"], window=3),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "Chaikin_Vol": (df["high"] - df["low"]).ewm(span=5, adjust=False).mean(),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "Daily_VWAP": (df["close"] * df["volume"]) / df["volume"],
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5),
            "StochRSI": ta.momentum.stochrsi(close=df["close"], window=5, smooth1=3, smooth2=3)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi tính chỉ báo: {e}")
        return None

# 3. Alligator
def add_alligator(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi thêm Alligator: {e}")
        return None

# 4. Chuẩn bị dữ liệu phân loại
def prepare_classification_data(df):
    if df is None or df.empty:
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty:
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        print(f"Lỗi khi chuẩn bị dữ liệu: {e}")
        return None, None, None

# 5. Dự báo và backtesting
def forecast_and_backtest(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators(df)
    if df is None:
        return None

    df = add_alligator(df)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 5 ngày. Bỏ qua {name}.")
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 10 ngày. Bỏ qua {name}.")
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for pred in predictions_5.values() if pred == 1)
    buy_votes_10 = sum(1 for pred in predictions_10.values() if pred == 1)
    final_rec = "MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN"

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df
    }

# 6. So sánh kết quả với "Toàn bộ từ ngày chọn" và "120 ngày"
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}

    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest(symbol, custom_start_date, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest(symbol, start_120, end_date)

    periods = ["Toàn bộ từ ngày chọn", "120 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else 120

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.map(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 7. Hàm phân tích tài chính (đã cập nhật để linh hoạt với năm và tìm Current liabilities)
def get_financial_data(symbol, year=2024):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        income_statement = stock.finance.income_statement(period="year", lang="en")
        balance_sheet = stock.finance.balance_sheet(period="year", lang="en")

        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        company_info = company.overview()
        if company_info.empty or 'outstanding_share' not in company_info.columns:
            print(f"Không thể lấy thông tin cổ phiếu đang lưu hành cho {symbol}, dùng giá trị mặc định")
            outstanding_share = 1000000000
        else:
            outstanding_share = company_info['outstanding_share'].iloc[0] * 1_000_000

        industry = company_info['industry'].iloc[0] if not company_info.empty and 'industry' in company_info.columns else "Khác"

        revenue_col = "Revenue (Bn. VND)"
        net_profit_col = "Attributable to parent company"
        current_assets_col = "CURRENT ASSETS (Bn. VND)"
        current_liabilities_col = "CURRENT LIABILITIES (Bn. VND)"
        long_term_assets_col = "LONG-TERM ASSETS (Bn. VND)"
        short_term_debt_col = "Short-term borrowings (Bn. VND)"
        long_term_debt_col = "Long-term borrowings (Bn. VND)"
        equity_col = "Common shares (Bn. VND)"
        profit_before_tax_col = "Profit before tax"
        interest_expense_col = "Interest Expenses"

        income_data = income_statement[income_statement["yearReport"] == year]
        balance_data = balance_sheet[balance_sheet["yearReport"] == year]

        if income_data.empty or balance_data.empty:
            print(f"Không có dữ liệu tài chính cho năm {year} của {symbol}")
            return None

        required_income_cols = [revenue_col, net_profit_col, profit_before_tax_col, interest_expense_col]
        required_balance_cols = [current_assets_col, long_term_assets_col, short_term_debt_col, long_term_debt_col, equity_col]

        missing_income_cols = [col for col in required_income_cols if col not in income_data.columns]
        missing_balance_cols = [col for col in required_balance_cols if col not in balance_data.columns]

        if missing_income_cols or missing_balance_cols:
            print(f"Thiếu cột trong income_statement: {missing_income_cols}, balance_sheet: {missing_balance_cols}")
            return None

        revenue = income_data[revenue_col].iloc[-1]
        net_profit = income_data[net_profit_col].iloc[-1]
        profit_before_tax = income_data[profit_before_tax_col].iloc[-1]
        interest_expense = abs(income_data[interest_expense_col].iloc[-1])

        ebit = profit_before_tax + interest_expense

        current_assets = balance_data[current_assets_col].iloc[-1]
        long_term_assets = balance_data[long_term_assets_col].iloc[-1]
        short_term_debt = balance_data[short_term_debt_col].iloc[-1]
        long_term_debt = balance_data[long_term_debt_col].iloc[-1]
        equity = balance_data[equity_col].iloc[-1]

        # Tìm Current liabilities không phân biệt hoa/thường, không in thông báo
        matching_col = [col for col in balance_data.columns if col.lower() == current_liabilities_col.lower()]
        if matching_col:
            current_liabilities = balance_data[matching_col[0]].iloc[-1]
            current_ratio = current_assets / current_liabilities if current_liabilities > 0 else float("inf")
        else:
            print(f"Cột '{current_liabilities_col}' không tồn tại, dùng Short-term borrowings thay thế")
            current_liabilities = short_term_debt
            current_ratio = current_assets / current_liabilities if current_liabilities > 0 else float("inf")

        total_assets = current_assets + long_term_assets
        total_debt = total_assets - equity if equity_col in balance_data.columns else short_term_debt + long_term_debt
        roe = (net_profit / equity) * 100 if equity > 0 else 0
        roa = (net_profit / total_assets) * 100 if total_assets > 0 else 0
        interest_coverage = ebit / interest_expense if interest_expense > 0 else float("inf")

        end_date = f"{year}-12-31"
        start_date = f"{year}-12-01"
        price_data = get_stock_data(symbol, start_date, end_date)
        if price_data is not None and not price_data.empty:
            price = price_data["close"].iloc[-1] * 1000
            eps = net_profit / outstanding_share if outstanding_share > 0 else 0
            pe_ratio = price / eps if eps != 0 else 0
        else:
            print(f"Không có dữ liệu giá cho {symbol} trong khoảng {start_date} đến {end_date}, P/E sẽ là 0")
            pe_ratio = 0

        return {
            "revenue": revenue,
            "net_profit": net_profit,
            "total_assets": total_assets,
            "total_debt": total_debt,
            "pe_ratio": pe_ratio,
            "current_ratio": current_ratio,
            "roe": roe,
            "roa": roa,
            "interest_coverage": interest_coverage,
            "equity": equity,
            "outstanding_share": outstanding_share,
            "industry": industry
        }
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu tài chính cho {symbol} năm {year}: {e}")
        return None
# 8. Phân tích tài chính
def analyze_financials(financial_data, symbol):
    if not financial_data or any(v is None for v in financial_data.values()):
        return {
            "Assessment": "N/A",
            "Financial Score": 0,
            "Detailed Assessment": "Không có dữ liệu để phân tích.",
            "Comments": []
        }

    revenue = financial_data["revenue"]
    net_profit = financial_data["net_profit"]
    total_assets = financial_data["total_assets"]
    total_debt = financial_data["total_debt"]
    pe_ratio = financial_data["pe_ratio"]
    current_ratio = financial_data["current_ratio"]
    roe = financial_data["roe"]
    roa = financial_data["roa"]
    interest_coverage = financial_data["interest_coverage"]
    equity = financial_data["equity"]
    industry = financial_data["industry"]

    net_profit_margin = (net_profit / revenue) * 100 if revenue > 0 else 0
    debt_to_equity = total_debt / equity if equity > 0 else float("inf")

    # Trọng số theo yêu cầu
    weights = {
        "Net Profit Margin": 2.0,
        "Debt to Equity": 1.5,
        "P/E Ratio": 1.0,
        "Current Ratio": 1.5,
        "ROE": 2.0,
        "ROA": 1.5,
        "Interest Coverage": 0.5
    }

    # Ngưỡng ngành theo yêu cầu
    industry_thresholds = {
        "Bất động sản": {"npm": 10, "dte": 2, "pe": 20, "cr": 1, "roe": 8, "roa": 3, "ic": 2},
        "Công nghệ": {"npm": 15, "dte": 1, "pe": 25, "cr": 1.5, "roe": 15, "roa": 10, "ic": 5},
        "Ngân hàng": {"npm": 20, "dte": 10, "pe": 15, "cr": 1, "roe": 12, "roa": 1, "ic": 3},
        "Khác": {"npm": 10, "dte": 1, "pe": 15, "cr": 1, "roe": 10, "roa": 5, "ic": 3}
    }

    th = industry_thresholds.get(industry, industry_thresholds["Khác"])

    # Tính điểm theo yêu cầu
    financial_score = 0
    if net_profit_margin > th["npm"]:
        financial_score += weights["Net Profit Margin"]
    if debt_to_equity < th["dte"]:
        financial_score += weights["Debt to Equity"]
    if pe_ratio > 0 and pe_ratio < th["pe"]:
        financial_score += weights["P/E Ratio"]
    if current_ratio > th["cr"]:
        financial_score += weights["Current Ratio"]
    if roe > th["roe"]:
        financial_score += weights["ROE"]
    if roa > th["roa"]:
        financial_score += weights["ROA"]
    if interest_coverage > th["ic"]:
        financial_score += weights["Interest Coverage"]

    # Phân loại theo yêu cầu
    assessment = "Tích cực" if financial_score >= 7 else "Trung lập" if financial_score >= 4 else "Tiêu cực"

    # Nhận xét chi tiết (Detailed Assessment)
    detailed_assessment = f"Ngành: {industry}\n"
    detailed_assessment += f"- Biên lợi nhuận ròng: {net_profit_margin:.2f}% (Ngưỡng ngành: {th['npm']}%). "
    detailed_assessment += "Rất tốt" if net_profit_margin > th["npm"] else "Cần cải thiện" if net_profit_margin < th["npm"] * 0.5 else "Ổn."
    detailed_assessment += f"\n- Tỷ lệ nợ/vốn: {debt_to_equity:.2f} (Ngưỡng ngành: {th['dte']}). "
    detailed_assessment += "An toàn" if debt_to_equity < th["dte"] else "Rủi ro cao" if debt_to_equity > th["dte"] * 1.5 else "Chấp nhận được."
    detailed_assessment += f"\n- P/E: {pe_ratio:.2f} (Ngưỡng ngành: {th['pe']}). "
    detailed_assessment += "Hấp dẫn" if pe_ratio < th["pe"] and pe_ratio > 0 else "Cao" if pe_ratio > th["pe"] * 1.5 else "Bình thường."
    detailed_assessment += f"\n- Thanh khoản ngắn hạn: {current_ratio:.2f} (Ngưỡng ngành: {th['cr']}). "
    detailed_assessment += "Tốt" if current_ratio > th["cr"] else "Yếu" if current_ratio < 1 else "Ổn."
    detailed_assessment += f"\n- ROE: {roe:.2f}% (Ngưỡng ngành: {th['roe']}%). "
    detailed_assessment += "Hiệu quả cao" if roe > th["roe"] else "Thấp" if roe < 0 else "Bình thường."
    detailed_assessment += f"\n- ROA: {roa:.2f}% (Ngưỡng ngành: {th['roa']}%). "
    detailed_assessment += "Hiệu quả" if roa > th["roa"] else "Thấp" if roa < 0 else "Ổn."
    detailed_assessment += f"\n- Khả năng trả lãi: {interest_coverage:.2f} (Ngưỡng ngành: {th['ic']}). "
    detailed_assessment += "Mạnh" if interest_coverage > th["ic"] else "Yếu" if interest_coverage < 1 else "Ổn."

    # Thêm phần Comments dưới dạng danh sách
    comments = [
        f"**Sức khỏe tài chính**: {symbol} đạt điểm {financial_score:.2f}/10.00, đánh giá '{assessment}' trong ngành {industry}.",
        f"**Net Profit Margin ({net_profit_margin:.2f}%)**: {'Cao hơn' if net_profit_margin > th['npm'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['npm']}%), cho thấy khả năng sinh lời {'vượt trội' if net_profit_margin > th['npm'] else 'cần cải thiện' if net_profit_margin < th['npm'] * 0.5 else 'ổn định'}.",
        f"**Debt to Equity ({debt_to_equity:.2f})**: {'Dưới' if debt_to_equity < th['dte'] else 'Trên hoặc bằng'} ngưỡng ngành ({th['dte']}), {'ít phụ thuộc vào nợ, rủi ro tài chính thấp' if debt_to_equity < th['dte'] else 'rủi ro tài chính cao' if debt_to_equity > th['dte'] * 1.5 else 'mức nợ chấp nhận được'}.",
        f"**P/E Ratio ({pe_ratio:.2f})**: {'Trong khoảng hợp lý' if 0 < pe_ratio < th['pe'] else 'Không hợp lý hoặc không xác định'} (<{th['pe']}), định giá cổ phiếu {'hấp dẫn' if 0 < pe_ratio < th['pe'] else 'cao hoặc không xác định'}.",
        f"**Current Ratio ({current_ratio:.2f})**: {'Vượt' if current_ratio > th['cr'] else 'Dưới hoặc bằng'} ngưỡng ngành ({th['cr']}), khả năng thanh toán ngắn hạn {'tốt' if current_ratio > th['cr'] else 'yếu' if current_ratio < 1 else 'ổn định'}.",
        f"**ROE ({roe:.2f}%)**: {'Cao hơn' if roe > th['roe'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['roe']}%), hiệu quả sử dụng vốn chủ sở hữu {'vượt trội' if roe > th['roe'] else 'thấp' if roe < 0 else 'bình thường'}.",
        f"**ROA ({roa:.2f}%)**: {'Vượt' if roa > th['roa'] else 'Dưới hoặc bằng'} ngưỡng ngành ({th['roa']}%), sử dụng tài sản {'hiệu quả' if roa > th['roa'] else 'thấp' if roa < 0 else 'ổn định'}.",
        f"**Interest Coverage ({interest_coverage:.2f})**: {'Cao hơn' if interest_coverage > th['ic'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['ic']}), khả năng trả lãi vay {'tốt' if interest_coverage > th['ic'] else 'yếu' if interest_coverage < 1 else 'ổn định'}."
    ]

    return {
        "Net Profit Margin": round(net_profit_margin, 2),
        "Debt to Equity": round(debt_to_equity, 2) if debt_to_equity != float("inf") else "N/A",
        "P/E Ratio": round(pe_ratio, 2) if pe_ratio != float("inf") else "N/A",
        "Current Ratio": round(current_ratio, 2) if current_ratio != float("inf") else "N/A",
        "ROE": round(roe, 2),
        "ROA": round(roa, 2),
        "Interest Coverage": round(interest_coverage, 2) if interest_coverage != float("inf") else "N/A",
        "Financial Score": financial_score,
        "Assessment": assessment,
        "Detailed Assessment": detailed_assessment,
        "Comments": comments
    }
# 9. Phân tích cổ đông lớn
def analyze_major_shareholders(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        shareholders = company.shareholders()
        if shareholders.empty or 'share_own_percent' not in shareholders.columns:
            print(f"Dữ liệu cổ đông lớn cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_major_share = shareholders['share_own_percent'].sum() * 100  # Chuyển đổi từ tỷ lệ sang phần trăm
        top_shareholder = shareholders['share_own_percent'].iloc[0] * 100

        if total_major_share > 50:
            assessment = "Cao - Rủi ro quản trị cao, nhưng có thể tăng giá nếu cổ đông lớn mua thêm."
        elif total_major_share > 30:
            assessment = "Trung bình - Ảnh hưởng đáng kể từ cổ đông lớn."
        else:
            assessment = "Thấp - Quyền kiểm soát phân tán, ít rủi ro quản trị."

        return {
            "Total Major Share (%)": round(total_major_share, 2),
            "Top Shareholder (%)": round(top_shareholder, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích cổ đông lớn cho {symbol}: {e}")
        return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 10. Phân tích mức độ gắn bó của lãnh đạo
def analyze_management_commitment(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        officers = company.officers()
        if officers.empty or 'ownPercent' not in officers.columns:
            print(f"Dữ liệu lãnh đạo cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_management_share = officers['ownPercent'].sum()

        if total_management_share > 5:
            assessment = "Cao - Lãnh đạo có cam kết mạnh mẽ với công ty."
        elif total_management_share > 1:
            assessment = "Trung bình - Cam kết ở mức khá."
        else:
            assessment = "Thấp - Lãnh đạo ít gắn bó về mặt sở hữu."

        return {
            "Management Ownership (%)": round(total_management_share, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích mức độ gắn bó của lãnh đạo cho {symbol}: {e}")
        return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 11. Phân tích công ty con
def analyze_subsidiaries(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        subsidiaries = company.subsidiaries()
        if subsidiaries.empty or 'sub_own_percent' not in subsidiaries.columns:
            print(f"Dữ liệu công ty con cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        num_subsidiaries = len(subsidiaries)
        avg_ownership = subsidiaries['sub_own_percent'].mean() * 100  # Chuyển đổi từ tỷ lệ sang phần trăm

        if num_subsidiaries > 5 and avg_ownership > 50:
            assessment = "Đa dạng hóa tốt - Tiềm năng lợi nhuận từ công ty con cao."
        elif num_subsidiaries > 2:
            assessment = "Đa dạng hóa trung bình - Tiềm năng lợi nhuận khá."
        else:
            assessment = "Đa dạng hóa thấp - Ít phụ thuộc vào công ty con."

        return {
            "Number of Subsidiaries": num_subsidiaries,
            "Average Ownership (%)": round(avg_ownership, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích công ty con cho {symbol}: {e}")
        return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 12. Phát hiện tín hiệu nội bộ
def detect_insider_signals(symbol, days=30):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        insider_deals = company.insider_deals()
        if insider_deals.empty or 'deal_quantity' not in insider_deals.columns:
            print(f"Dữ liệu giao dịch nội bộ cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

        insider_deals['deal_announce_date'] = pd.to_datetime(insider_deals['deal_announce_date'])
        recent_deals = insider_deals[insider_deals['deal_announce_date'] > datetime.now() - timedelta(days=days)]

        if recent_deals.empty:
            return {"Net Insider Volume": 0, "Assessment": "Trung lập - Không có giao dịch gần đây."}

        net_volume = recent_deals['deal_quantity'].sum()  # Tổng khối lượng mua/bán (dương là mua, âm là bán)

        if net_volume > 0:
            assessment = "Tích cực - Lãnh đạo mua ròng, dấu hiệu tốt cho triển vọng."
        elif net_volume < 0:
            assessment = "Tiêu cực - Lãnh đạo bán ròng, cần thận trọng."
        else:
            assessment = "Trung lập - Không có tín hiệu rõ ràng."

        return {
            "Net Insider Volume": int(net_volume),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phát hiện tín hiệu nội bộ cho {symbol}: {e}")
        return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

# 13. Dự đoán biến động giá quanh sự kiện
def predict_event_impact(symbol, days_before=5, days_after=5):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        events = company.events()
        if events.empty or 'exer_date' not in events.columns:
            print(f"Dữ liệu sự kiện cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

        events['exer_date'] = pd.to_datetime(events['exer_date'])
        recent_event = events[events['exer_date'] <= datetime.now()].iloc[-1]
        event_date = recent_event['exer_date']
        event_desc = recent_event['event_desc']

        start_date = (event_date - timedelta(days=days_before)).strftime("%Y-%m-%d")
        end_date = (event_date + timedelta(days=days_after)).strftime("%Y-%m-%d")
        price_data = get_stock_data(symbol, start_date, end_date)

        if price_data is None or len(price_data) < days_before + days_after:
            print(f"Không đủ dữ liệu giá quanh sự kiện cho {symbol}")
            return {"Event Date": event_date.strftime("%Y-%m-%d"), "Event Description": event_desc, "Price Change (%)": "N/A", "Assessment": "Không đủ dữ liệu giá"}

        pre_event_price = price_data['close'].iloc[days_before - 1]
        post_event_price = price_data['close'].iloc[-1]
        price_change = ((post_event_price - pre_event_price) / pre_event_price) * 100

        assessment = "Tăng giá" if price_change > 0 else "Giảm giá"

        return {
            "Event Date": event_date.strftime("%Y-%m-%d"),
            "Event Description": event_desc,
            "Price Change (%)": round(price_change, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi dự đoán biến động giá quanh sự kiện cho {symbol}: {e}")
        return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 14. Cập nhật tin tức nóng
def get_latest_news(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        news = company.news()
        if news.empty or 'publish_date' not in news.columns:
            print(f"Dữ liệu tin tức cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

        news['publish_date'] = pd.to_datetime(news['publish_date'])
        latest_news = news.iloc[0]

        return {
            "Date": latest_news['publish_date'].strftime("%Y-%m-%d"),
            "Title": latest_news['title'],
            "Source": latest_news['source']
        }
    except Exception as e:
        print(f"Lỗi khi lấy tin tức cho {symbol}: {e}")
        return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

# 15. Phân tích dòng tiền và cảnh báo bất thường
def analyze_cash_flow(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

        avg_volume = df['volume'].mean()
        latest_volume = df['volume'].iloc[-1]
        volume_spike = latest_volume > avg_volume * 2
        volume_warning = latest_volume > avg_volume * 1.5

        if volume_spike:
            assessment = "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền."
        elif volume_warning:
            assessment = "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình."
        else:
            assessment = "Không - Dòng tiền ổn định."

        return {
            "Data Range": f"{start_date} đến {end_date}",
            "Average Volume": round(avg_volume, 0),
            "Latest Volume": round(latest_volume, 0),
            "Volume Spike": volume_spike,
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích dòng tiền cho {symbol}: {e}")
        return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}
# CSS để định dạng bảng
TABLE_STYLE = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-size: 14px;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""

# Hàm tạo dashboard tài chính
def create_financial_dashboard(financial_results, symbol):
    if not financial_results:
        print("Không có dữ liệu để tạo dashboard.")
        return

    df = pd.DataFrame(financial_results)
    years = df["Year"].astype(str)

    # Tạo subplot với 4 dòng, 2 cột
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=(
            "Net Profit Margin (%)", "Debt to Equity",
            "P/E Ratio", "Current Ratio",
            "ROE (%)", "ROA (%)",
            "Interest Coverage", ""
        ),
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )

    # Biểu đồ Net Profit Margin
    fig.add_trace(
        go.Scatter(x=years, y=df["Net Profit Margin"], mode="lines+markers", name="Net Profit Margin"),
        row=1, col=1
    )

    # Biểu đồ Debt to Equity
    df["Debt to Equity"] = pd.to_numeric(df["Debt to Equity"], errors="coerce")
    fig.add_trace(
        go.Scatter(x=years, y=df["Debt to Equity"], mode="lines+markers", name="Debt to Equity"),
        row=1, col=2
    )

    # Biểu đồ P/E Ratio
    df["P/E Ratio"] = pd.to_numeric(df["P/E Ratio"], errors="coerce")
    fig.add_trace(
        go.Scatter(x=years, y=df["P/E Ratio"], mode="lines+markers", name="P/E Ratio"),
        row=2, col=1
    )

    # Biểu đồ Current Ratio
    df["Current Ratio"] = pd.to_numeric(df["Current Ratio"], errors="coerce")
    fig.add_trace(
        go.Scatter(x=years, y=df["Current Ratio"], mode="lines+markers", name="Current Ratio"),
        row=2, col=2
    )

    # Biểu đồ ROE
    fig.add_trace(
        go.Scatter(x=years, y=df["ROE"], mode="lines+markers", name="ROE"),
        row=3, col=1
    )

    # Biểu đồ ROA
    fig.add_trace(
        go.Scatter(x=years, y=df["ROA"], mode="lines+markers", name="ROA"),
        row=3, col=2
    )

    # Biểu đồ Interest Coverage
    df["Interest Coverage"] = pd.to_numeric(df["Interest Coverage"], errors="coerce")
    fig.add_trace(
        go.Scatter(x=years, y=df["Interest Coverage"], mode="lines+markers", name="Interest Coverage"),
        row=4, col=1
    )

    # Cập nhật layout
    fig.update_layout(
        height=1000,
        width=1000,
        title_text=f"Dashboard Tài Chính cho mã {symbol} qua các năm",
        showlegend=False,
        template="plotly_white"
    )

    # Cập nhật trục Y
    fig.update_yaxes(title_text="%", row=1, col=1)
    fig.update_yaxes(title_text="Tỷ lệ", row=1, col=2)
    fig.update_yaxes(title_text="Lần", row=2, col=1)
    fig.update_yaxes(title_text="Tỷ lệ", row=2, col=2)
    fig.update_yaxes(title_text="%", row=3, col=1)
    fig.update_yaxes(title_text="%", row=3, col=2)
    fig.update_yaxes(title_text="Lần", row=4, col=1)

    # Hiển thị dashboard
    fig.show()

# 16. Chạy trên Colab với widget (sửa để hiển thị dashboard)
def run_prediction():
    end_date = datetime.now().strftime("%Y-%m-%d")

    # Tạo ba widget Output
    output_prediction = widgets.Output()  # Output cho dự đoán giá
    output_financial = widgets.Output()   # Output cho phân tích tài chính và các phần khác
    output_dashboard = widgets.Output()   # Output cho dashboard

    date_picker = widgets.DatePicker(
        description='Chọn ngày bắt đầu:',
        value=datetime(2023, 1, 1),
        disabled=False
    )

    symbol_input = widgets.Text(
        description='Mã chứng khoán:',
        placeholder='Nhập mã (hoặc "exit" để thoát)',
    )

    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output_prediction:
            clear_output()  # Chỉ xóa output dự đoán khi chạy lại lần mới
            custom_start_date = date_picker.value
            if custom_start_date is None:
                print("Vui lòng chọn ngày bắt đầu!")
                return
            custom_start_date = custom_start_date.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")
            compare_predictions(symbol, custom_start_date, end_date)

            # Kiểm tra các năm có dữ liệu
            stock = Vnstock().stock(symbol=symbol, source="VCI")
            income_statement = stock.finance.income_statement(period="year", lang="en")
            balance_sheet = stock.finance.balance_sheet(period="year", lang="en")
            income_years = income_statement["yearReport"].unique().tolist() if not income_statement.empty else []
            balance_years = balance_sheet["yearReport"].unique().tolist() if not balance_sheet.empty else []
            all_years = sorted(set(income_years + balance_years))
            print(f"Danh sách các năm có dữ liệu tài chính cho {symbol}: {all_years}")

            # Giao diện chọn số năm
            num_years_picker = widgets.IntSlider(
                value=5,
                min=1,
                max=min(12, len(all_years)),
                step=1,
                description='Số năm phân tích:',
            )
            button_analyze_years = widgets.Button(description="Phân tích sức khỏe tài chính")

            def on_analyze_years_clicked(b):
                with output_financial:
                    clear_output()  # Chỉ xóa output tài chính khi phân tích lại
                    num_years = num_years_picker.value
                    latest_year = 2024
                    selected_years = [latest_year - i for i in range(num_years) if (latest_year - i) in all_years]
                    selected_years.reverse()
                    print(f"\nPHÂN TÍCH SỨC KHỎE TÀI CHÍNH CHO MÃ {symbol} TRONG {num_years} NĂM GẦN NHẤT: {selected_years}")

                    # Tính toán chỉ số tài chính cho từng năm
                    financial_results = []
                    for year in selected_years:
                        financial_data = get_financial_data(symbol, year=year)
                        if financial_data:
                            net_profit_margin = (financial_data["net_profit"] / financial_data["revenue"]) * 100 if financial_data["revenue"] > 0 else 0
                            debt_to_equity = financial_data["total_debt"] / financial_data["equity"] if financial_data["equity"] > 0 else float("inf")
                            financial_results.append({
                                "Year": year,
                                "Net Profit Margin": round(net_profit_margin, 2),
                                "Debt to Equity": round(debt_to_equity, 2) if debt_to_equity != float("inf") else "N/A",
                                "P/E Ratio": round(financial_data["pe_ratio"], 2) if financial_data["pe_ratio"] != float("inf") else "N/A",
                                "Current Ratio": round(financial_data["current_ratio"], 2) if financial_data["current_ratio"] != float("inf") else "N/A",
                                "ROE": round(financial_data["roe"], 2),
                                "ROA": round(financial_data["roa"], 2),
                                "Interest Coverage": round(financial_data["interest_coverage"], 2) if financial_data["interest_coverage"] != float("inf") else "N/A"
                            })

                    # Hiển thị bảng kết quả
                    if financial_results:
                        financial_df = pd.DataFrame(financial_results)
                        display(HTML(TABLE_STYLE + financial_df.to_html(index=False)))
                    else:
                        print("Không có dữ liệu tài chính để hiển thị.")

                # Hiển thị dashboard
                with output_dashboard:
                    clear_output()
                    if financial_results:
                        print("Hiển thị dashboard tài chính:")
                        create_financial_dashboard(financial_results, symbol)
                    else:
                        print("Không có dữ liệu tài chính để hiển thị dashboard.")

                with output_financial:
                    # Phân tích chi tiết cho năm 2024
                    if 2024 in selected_years:
                        print(f"\nPHÂN TÍCH TÀI CHÍNH CHI TIẾT CHO MÃ {symbol} NĂM 2024:")
                        financial_data_2024 = get_financial_data(symbol, year=2024)
                        if financial_data_2024:
                            financial_result_2024 = analyze_financials(financial_data_2024, symbol)
                            comments_df = pd.DataFrame(financial_result_2024["Comments"], columns=["Nhận xét chi tiết"])
                            display(HTML(TABLE_STYLE + comments_df.to_html(index=False, escape=False)))

                    # Các phần khác
                    print(f"\nPHÂN TÍCH CỔ ĐÔNG LỚN CHO MÃ {symbol}:")
                    shareholders_result = analyze_major_shareholders(symbol)
                    shareholders_df = pd.DataFrame([shareholders_result])
                    display(HTML(TABLE_STYLE + shareholders_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH MỨC ĐỘ GẮN BÓ CỦA LÃNH ĐẠO CHO MÃ {symbol}:")
                    management_result = analyze_management_commitment(symbol)
                    management_df = pd.DataFrame([management_result])
                    display(HTML(TABLE_STYLE + management_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH CÔNG TY CON CHO MÃ {symbol}:")
                    subsidiaries_result = analyze_subsidiaries(symbol)
                    subsidiaries_df = pd.DataFrame([subsidiaries_result])
                    display(HTML(TABLE_STYLE + subsidiaries_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH TÍN HIỆU NỘI BỘ (30 NGÀY GẦN NHẤT) CHO MÃ {symbol}:")
                    insider_result = detect_insider_signals(symbol)
                    insider_df = pd.DataFrame([insider_result])
                    display(HTML(TABLE_STYLE + insider_df.to_html(index=False)))

                    print(f"\nDỰ ĐOÁN BIẾN ĐỘNG GIÁ QUANH SỰ KIỆN GẦN NHẤT CHO MÃ {symbol}:")
                    event_result = predict_event_impact(symbol)
                    event_df = pd.DataFrame([event_result])
                    display(HTML(TABLE_STYLE + event_df.to_html(index=False)))

                    print(f"\nTIN TỨC NÓNG GẦN NHẤT CHO MÃ {symbol}:")
                    news_result = get_latest_news(symbol)
                    news_df = pd.DataFrame([news_result])
                    display(HTML(TABLE_STYLE + news_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH DÒNG TIỀN VÀ CẢNH BÁO BẤT THƯỜNG CHO MÃ {symbol}:")
                    cash_flow_result = analyze_cash_flow(symbol, custom_start_date, end_date)
                    cash_flow_df = pd.DataFrame([cash_flow_result])
                    styled_cash_flow_df = cash_flow_df.style.applymap(
                        lambda val: 'color: red' if val == "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền." else 'color: orange' if val == "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình." else 'color: black',
                        subset=["Assessment"]
                    )
                    display(HTML(TABLE_STYLE + styled_cash_flow_df.to_html(index=False)))

                    display(widgets.HBox([button_continue_yes, button_continue_no]))

            # Hiển thị giao diện chọn số năm ngay sau khi dự đoán
            with output_prediction:
                print("\nChọn số năm để phân tích sức khỏe tài chính:")
                display(widgets.VBox([num_years_picker, button_analyze_years]))
            button_analyze_years.on_click(on_analyze_years_clicked)

    def on_yes_clicked(b):
        with output_prediction:
            clear_output()
            display(date_picker, symbol_input, button_confirm, output_prediction)
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()

    def on_no_clicked(b):
        with output_prediction:
            clear_output()
            print("Đã thoát chương trình.")
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    # Hiển thị giao diện ban đầu
    display(date_picker, symbol_input, button_confirm, output_prediction, output_financial, output_dashboard)

if __name__ == "__main__":
    run_prediction()

DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Chọn ngày bắt đầu:')

Text(value='', description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')

Button(description='Xác nhận và dự đoán', style=ButtonStyle())

Output()

Output()

Output()

Nhiều năm + đồ thị

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datetime import datetime, timedelta
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import logging
import matplotlib.pyplot as plt

# Tắt cảnh báo từ vnstock
logging.getLogger("vnstock").setLevel(logging.ERROR)
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")

# Cấu hình renderer Plotly cho Colab
pio.renderers.default = "colab"

# 1. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 5 or not all(col in df.columns for col in required_columns):
            print(f"Dữ liệu không đủ cho {symbol} (ít hơn 5 ngày hoặc thiếu cột).")
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")
        return None

# 2. Chỉ báo kỹ thuật
def calculate_indicators(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_3": ta.trend.ema_indicator(df["close"], window=3),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "Chaikin_Vol": (df["high"] - df["low"]).ewm(span=5, adjust=False).mean(),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "Daily_VWAP": (df["close"] * df["volume"]) / df["volume"],
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5),
            "StochRSI": ta.momentum.stochrsi(close=df["close"], window=5, smooth1=3, smooth2=3)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi tính chỉ báo: {e}")
        return None

# 3. Alligator
def add_alligator(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi thêm Alligator: {e}")
        return None

# 4. Chuẩn bị dữ liệu phân loại
def prepare_classification_data(df):
    if df is None or df.empty:
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty:
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        print(f"Lỗi khi chuẩn bị dữ liệu: {e}")
        return None, None, None

# 5. Dự báo và backtesting
def forecast_and_backtest(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators(df)
    if df is None:
        return None

    df = add_alligator(df)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 5 ngày. Bỏ qua {name}.")
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 10 ngày. Bỏ qua {name}.")
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for pred in predictions_5.values() if pred == 1)
    buy_votes_10 = sum(1 for pred in predictions_10.values() if pred == 1)
    final_rec = "MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN"

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df
    }

# 6. So sánh kết quả với "Toàn bộ từ ngày chọn" và "120 ngày"
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}

    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest(symbol, custom_start_date, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest(symbol, start_120, end_date)

    periods = ["Toàn bộ từ ngày chọn", "120 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else 120

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.applymap(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 7. Hàm phân tích tài chính (đã cập nhật để linh hoạt với năm và tìm Current liabilities)
def get_financial_data(symbol, year=2024):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        income_statement = stock.finance.income_statement(period="year", lang="en")
        balance_sheet = stock.finance.balance_sheet(period="year", lang="en")

        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        company_info = company.overview()
        if company_info.empty or 'outstanding_share' not in company_info.columns:
            print(f"Không thể lấy thông tin cổ phiếu đang lưu hành cho {symbol}, dùng giá trị mặc định")
            outstanding_share = 1000000000
        else:
            outstanding_share = company_info['outstanding_share'].iloc[0] * 1_000_000

        industry = company_info['industry'].iloc[0] if not company_info.empty and 'industry' in company_info.columns else "Khác"

        revenue_col = "Revenue (Bn. VND)"
        net_profit_col = "Attributable to parent company"
        current_assets_col = "CURRENT ASSETS (Bn. VND)"
        current_liabilities_col = "CURRENT LIABILITIES (Bn. VND)"
        long_term_assets_col = "LONG-TERM ASSETS (Bn. VND)"
        short_term_debt_col = "Short-term borrowings (Bn. VND)"
        long_term_debt_col = "Long-term borrowings (Bn. VND)"
        equity_col = "Common shares (Bn. VND)"
        profit_before_tax_col = "Profit before tax"
        interest_expense_col = "Interest Expenses"

        income_data = income_statement[income_statement["yearReport"] == year]
        balance_data = balance_sheet[balance_sheet["yearReport"] == year]

        if income_data.empty or balance_data.empty:
            print(f"Không có dữ liệu tài chính cho năm {year} của {symbol}")
            return None

        required_income_cols = [revenue_col, net_profit_col, profit_before_tax_col, interest_expense_col]
        required_balance_cols = [current_assets_col, long_term_assets_col, short_term_debt_col, long_term_debt_col, equity_col]

        missing_income_cols = [col for col in required_income_cols if col not in income_data.columns]
        missing_balance_cols = [col for col in required_balance_cols if col not in balance_data.columns]

        if missing_income_cols or missing_balance_cols:
            print(f"Thiếu cột trong income_statement: {missing_income_cols}, balance_sheet: {missing_balance_cols}")
            return None

        revenue = income_data[revenue_col].iloc[-1]
        net_profit = income_data[net_profit_col].iloc[-1]
        profit_before_tax = income_data[profit_before_tax_col].iloc[-1]
        interest_expense = abs(income_data[interest_expense_col].iloc[-1])

        ebit = profit_before_tax + interest_expense

        current_assets = balance_data[current_assets_col].iloc[-1]
        long_term_assets = balance_data[long_term_assets_col].iloc[-1]
        short_term_debt = balance_data[short_term_debt_col].iloc[-1]
        long_term_debt = balance_data[long_term_debt_col].iloc[-1]
        equity = balance_data[equity_col].iloc[-1]

        # Tìm Current liabilities không phân biệt hoa/thường, không in thông báo
        matching_col = [col for col in balance_data.columns if col.lower() == current_liabilities_col.lower()]
        if matching_col:
            current_liabilities = balance_data[matching_col[0]].iloc[-1]
            current_ratio = current_assets / current_liabilities if current_liabilities > 0 else float("inf")
        else:
            print(f"Cột '{current_liabilities_col}' không tồn tại, dùng Short-term borrowings thay thế")
            current_liabilities = short_term_debt
            current_ratio = current_assets / current_liabilities if current_liabilities > 0 else float("inf")

        total_assets = current_assets + long_term_assets
        total_debt = total_assets - equity if equity_col in balance_data.columns else short_term_debt + long_term_debt
        roe = (net_profit / equity) * 100 if equity > 0 else 0
        roa = (net_profit / total_assets) * 100 if total_assets > 0 else 0
        interest_coverage = ebit / interest_expense if interest_expense > 0 else float("inf")

        end_date = f"{year}-12-31"
        start_date = f"{year}-12-01"
        price_data = get_stock_data(symbol, start_date, end_date)
        if price_data is not None and not price_data.empty:
            price = price_data["close"].iloc[-1] * 1000
            eps = net_profit / outstanding_share if outstanding_share > 0 else 0
            pe_ratio = price / eps if eps != 0 else 0
        else:
            print(f"Không có dữ liệu giá cho {symbol} trong khoảng {start_date} đến {end_date}, P/E sẽ là 0")
            pe_ratio = 0

        return {
            "revenue": revenue,
            "net_profit": net_profit,
            "total_assets": total_assets,
            "total_debt": total_debt,
            "pe_ratio": pe_ratio,
            "current_ratio": current_ratio,
            "roe": roe,
            "roa": roa,
            "interest_coverage": interest_coverage,
            "equity": equity,
            "outstanding_share": outstanding_share,
            "industry": industry
        }
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu tài chính cho {symbol} năm {year}: {e}")
        return None

# 8. Phân tích tài chính
def analyze_financials(financial_data, symbol):
    if not financial_data or any(v is None for v in financial_data.values()):
        return {
            "Assessment": "N/A",
            "Financial Score": 0,
            "Detailed Assessment": "Không có dữ liệu để phân tích.",
            "Comments": []
        }

    revenue = financial_data["revenue"]
    net_profit = financial_data["net_profit"]
    total_assets = financial_data["total_assets"]
    total_debt = financial_data["total_debt"]
    pe_ratio = financial_data["pe_ratio"]
    current_ratio = financial_data["current_ratio"]
    roe = financial_data["roe"]
    roa = financial_data["roa"]
    interest_coverage = financial_data["interest_coverage"]
    equity = financial_data["equity"]
    industry = financial_data["industry"]

    net_profit_margin = (net_profit / revenue) * 100 if revenue > 0 else 0
    debt_to_equity = total_debt / equity if equity > 0 else float("inf")

    weights = {
        "Net Profit Margin": 2.0,
        "Debt to Equity": 1.5,
        "P/E Ratio": 1.0,
        "Current Ratio": 1.5,
        "ROE": 2.0,
        "ROA": 1.5,
        "Interest Coverage": 0.5
    }

    industry_thresholds = {
        "Bất động sản": {"npm": 10, "dte": 2, "pe": 20, "cr": 1, "roe": 8, "roa": 3, "ic": 2},
        "Công nghệ": {"npm": 15, "dte": 1, "pe": 25, "cr": 1.5, "roe": 15, "roa": 10, "ic": 5},
        "Ngân hàng": {"npm": 20, "dte": 10, "pe": 15, "cr": 1, "roe": 12, "roa": 1, "ic": 3},
        "Khác": {"npm": 10, "dte": 1, "pe": 15, "cr": 1, "roe": 10, "roa": 5, "ic": 3}
    }

    th = industry_thresholds.get(industry, industry_thresholds["Khác"])

    financial_score = 0
    if net_profit_margin > th["npm"]:
        financial_score += weights["Net Profit Margin"]
    if debt_to_equity < th["dte"]:
        financial_score += weights["Debt to Equity"]
    if pe_ratio > 0 and pe_ratio < th["pe"]:
        financial_score += weights["P/E Ratio"]
    if current_ratio > th["cr"]:
        financial_score += weights["Current Ratio"]
    if roe > th["roe"]:
        financial_score += weights["ROE"]
    if roa > th["roa"]:
        financial_score += weights["ROA"]
    if interest_coverage > th["ic"]:
        financial_score += weights["Interest Coverage"]

    assessment = "Tích cực" if financial_score >= 7 else "Trung lập" if financial_score >= 4 else "Tiêu cực"

    detailed_assessment = f"Ngành: {industry}\n"
    detailed_assessment += f"- Biên lợi nhuận ròng: {net_profit_margin:.2f}% (Ngưỡng ngành: {th['npm']}%). "
    detailed_assessment += "Rất tốt" if net_profit_margin > th["npm"] else "Cần cải thiện" if net_profit_margin < th["npm"] * 0.5 else "Ổn."
    detailed_assessment += f"\n- Tỷ lệ nợ/vốn: {debt_to_equity:.2f} (Ngưỡng ngành: {th['dte']}). "
    detailed_assessment += "An toàn" if debt_to_equity < th["dte"] else "Rủi ro cao" if debt_to_equity > th["dte"] * 1.5 else "Chấp nhận được."
    detailed_assessment += f"\n- P/E: {pe_ratio:.2f} (Ngưỡng ngành: {th['pe']}). "
    detailed_assessment += "Hấp dẫn" if pe_ratio < th["pe"] and pe_ratio > 0 else "Cao" if pe_ratio > th["pe"] * 1.5 else "Bình thường."
    detailed_assessment += f"\n- Thanh khoản ngắn hạn: {current_ratio:.2f} (Ngưỡng ngành: {th['cr']}). "
    detailed_assessment += "Tốt" if current_ratio > th["cr"] else "Yếu" if current_ratio < 1 else "Ổn."
    detailed_assessment += f"\n- ROE: {roe:.2f}% (Ngưỡng ngành: {th['roe']}%). "
    detailed_assessment += "Hiệu quả cao" if roe > th["roe"] else "Thấp" if roe < 0 else "Bình thường."
    detailed_assessment += f"\n- ROA: {roa:.2f}% (Ngưỡng ngành: {th['roa']}%). "
    detailed_assessment += "Hiệu quả" if roa > th["roa"] else "Thấp" if roa < 0 else "Ổn."
    detailed_assessment += f"\n- Khả năng trả lãi: {interest_coverage:.2f} (Ngưỡng ngành: {th['ic']}). "
    detailed_assessment += "Mạnh" if interest_coverage > th["ic"] else "Yếu" if interest_coverage < 1 else "Ổn."

    comments = [
        f"**Sức khỏe tài chính**: {symbol} đạt điểm {financial_score:.2f}/10.00, đánh giá '{assessment}' trong ngành {industry}.",
        f"**Net Profit Margin ({net_profit_margin:.2f}%)**: {'Cao hơn' if net_profit_margin > th['npm'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['npm']}%), cho thấy khả năng sinh lời {'vượt trội' if net_profit_margin > th['npm'] else 'cần cải thiện' if net_profit_margin < th['npm'] * 0.5 else 'ổn định'}.",
        f"**Debt to Equity ({debt_to_equity:.2f})**: {'Dưới' if debt_to_equity < th['dte'] else 'Trên hoặc bằng'} ngưỡng ngành ({th['dte']}), {'ít phụ thuộc vào nợ, rủi ro tài chính thấp' if debt_to_equity < th['dte'] else 'rủi ro tài chính cao' if debt_to_equity > th['dte'] * 1.5 else 'mức nợ chấp nhận được'}.",
        f"**P/E Ratio ({pe_ratio:.2f})**: {'Trong khoảng hợp lý' if 0 < pe_ratio < th['pe'] else 'Không hợp lý hoặc không xác định'} (<{th['pe']}), định giá cổ phiếu {'hấp dẫn' if 0 < pe_ratio < th['pe'] else 'cao hoặc không xác định'}.",
        f"**Current Ratio ({current_ratio:.2f})**: {'Vượt' if current_ratio > th['cr'] else 'Dưới hoặc bằng'} ngưỡng ngành ({th['cr']}), khả năng thanh toán ngắn hạn {'tốt' if current_ratio > th['cr'] else 'yếu' if current_ratio < 1 else 'ổn định'}.",
        f"**ROE ({roe:.2f}%)**: {'Cao hơn' if roe > th['roe'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['roe']}%), hiệu quả sử dụng vốn chủ sở hữu {'vượt trội' if roe > th['roe'] else 'thấp' if roe < 0 else 'bình thường'}.",
        f"**ROA ({roa:.2f}%)**: {'Vượt' if roa > th['roa'] else 'Dưới hoặc bằng'} ngưỡng ngành ({th['roa']}%), sử dụng tài sản {'hiệu quả' if roa > th['roa'] else 'thấp' if roa < 0 else 'ổn định'}.",
        f"**Interest Coverage ({interest_coverage:.2f})**: {'Cao hơn' if interest_coverage > th['ic'] else 'Thấp hơn hoặc bằng'} ngưỡng ngành ({th['ic']}), khả năng trả lãi vay {'tốt' if interest_coverage > th['ic'] else 'yếu' if interest_coverage < 1 else 'ổn định'}."
    ]

    return {
        "Net Profit Margin": round(net_profit_margin, 2),
        "Debt to Equity": round(debt_to_equity, 2) if debt_to_equity != float("inf") else "N/A",
        "P/E Ratio": round(pe_ratio, 2) if pe_ratio != float("inf") else "N/A",
        "Current Ratio": round(current_ratio, 2) if current_ratio != float("inf") else "N/A",
        "ROE": round(roe, 2),
        "ROA": round(roa, 2),
        "Interest Coverage": round(interest_coverage, 2) if interest_coverage != float("inf") else "N/A",
        "Financial Score": financial_score,
        "Assessment": assessment,
        "Detailed Assessment": detailed_assessment,
        "Comments": comments
    }

# 9. Phân tích cổ đông lớn
def analyze_major_shareholders(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        shareholders = company.shareholders()
        if shareholders.empty or 'share_own_percent' not in shareholders.columns:
            print(f"Dữ liệu cổ đông lớn cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_major_share = shareholders['share_own_percent'].sum() * 100
        top_shareholder = shareholders['share_own_percent'].iloc[0] * 100

        if total_major_share > 50:
            assessment = "Cao - Rủi ro quản trị cao, nhưng có thể tăng giá nếu cổ đông lớn mua thêm."
        elif total_major_share > 30:
            assessment = "Trung bình - Ảnh hưởng đáng kể từ cổ đông lớn."
        else:
            assessment = "Thấp - Quyền kiểm soát phân tán, ít rủi ro quản trị."

        return {
            "Total Major Share (%)": round(total_major_share, 2),
            "Top Shareholder (%)": round(top_shareholder, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích cổ đông lớn cho {symbol}: {e}")
        return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 10. Phân tích mức độ gắn bó của lãnh đạo
def analyze_management_commitment(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        officers = company.officers()
        if officers.empty or 'ownPercent' not in officers.columns:
            print(f"Dữ liệu lãnh đạo cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_management_share = officers['ownPercent'].sum()

        if total_management_share > 5:
            assessment = "Cao - Lãnh đạo có cam kết mạnh mẽ với công ty."
        elif total_management_share > 1:
            assessment = "Trung bình - Cam kết ở mức khá."
        else:
            assessment = "Thấp - Lãnh đạo ít gắn bó về mặt sở hữu."

        return {
            "Management Ownership (%)": round(total_management_share, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích mức độ gắn bó của lãnh đạo cho {symbol}: {e}")
        return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 11. Phân tích công ty con
def analyze_subsidiaries(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        subsidiaries = company.subsidiaries()
        if subsidiaries.empty or 'sub_own_percent' not in subsidiaries.columns:
            print(f"Dữ liệu công ty con cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        num_subsidiaries = len(subsidiaries)
        avg_ownership = subsidiaries['sub_own_percent'].mean() * 100

        if num_subsidiaries > 5 and avg_ownership > 50:
            assessment = "Đa dạng hóa tốt - Tiềm năng lợi nhuận từ công ty con cao."
        elif num_subsidiaries > 2:
            assessment = "Đa dạng hóa trung bình - Tiềm năng lợi nhuận khá."
        else:
            assessment = "Đa dạng hóa thấp - Ít phụ thuộc vào công ty con."

        return {
            "Number of Subsidiaries": num_subsidiaries,
            "Average Ownership (%)": round(avg_ownership, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích công ty con cho {symbol}: {e}")
        return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 12. Phát hiện tín hiệu nội bộ
def detect_insider_signals(symbol, days=30):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        insider_deals = company.insider_deals()
        if insider_deals.empty or 'deal_quantity' not in insider_deals.columns:
            print(f"Dữ liệu giao dịch nội bộ cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

        insider_deals['deal_announce_date'] = pd.to_datetime(insider_deals['deal_announce_date'])
        recent_deals = insider_deals[insider_deals['deal_announce_date'] > datetime.now() - timedelta(days=days)]

        if recent_deals.empty:
            return {"Net Insider Volume": 0, "Assessment": "Trung lập - Không có giao dịch gần đây."}

        net_volume = recent_deals['deal_quantity'].sum()

        if net_volume > 0:
            assessment = "Tích cực - Lãnh đạo mua ròng, dấu hiệu tốt cho triển vọng."
        elif net_volume < 0:
            assessment = "Tiêu cực - Lãnh đạo bán ròng, cần thận trọng."
        else:
            assessment = "Trung lập - Không có tín hiệu rõ ràng."

        return {
            "Net Insider Volume": int(net_volume),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phát hiện tín hiệu nội bộ cho {symbol}: {e}")
        return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

# 13. Dự đoán biến động giá quanh sự kiện
def predict_event_impact(symbol, days_before=5, days_after=5):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        events = company.events()
        if events.empty or 'exer_date' not in events.columns:
            print(f"Dữ liệu sự kiện cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

        events['exer_date'] = pd.to_datetime(events['exer_date'])
        recent_event = events[events['exer_date'] <= datetime.now()].iloc[-1]
        event_date = recent_event['exer_date']
        event_desc = recent_event['event_desc']

        start_date = (event_date - timedelta(days=days_before)).strftime("%Y-%m-%d")
        end_date = (event_date + timedelta(days=days_after)).strftime("%Y-%m-%d")
        price_data = get_stock_data(symbol, start_date, end_date)

        if price_data is None or len(price_data) < days_before + days_after:
            print(f"Không đủ dữ liệu giá quanh sự kiện cho {symbol}")
            return {"Event Date": event_date.strftime("%Y-%m-%d"), "Event Description": event_desc, "Price Change (%)": "N/A", "Assessment": "Không đủ dữ liệu giá"}

        pre_event_price = price_data['close'].iloc[days_before - 1]
        post_event_price = price_data['close'].iloc[-1]
        price_change = ((post_event_price - pre_event_price) / pre_event_price) * 100

        assessment = "Tăng giá" if price_change > 0 else "Giảm giá"

        return {
            "Event Date": event_date.strftime("%Y-%m-%d"),
            "Event Description": event_desc,
            "Price Change (%)": round(price_change, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi dự đoán biến động giá quanh sự kiện cho {symbol}: {e}")
        return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 14. Cập nhật tin tức nóng
def get_latest_news(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        news = company.news()
        if news.empty or 'publish_date' not in news.columns:
            print(f"Dữ liệu tin tức cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

        news['publish_date'] = pd.to_datetime(news['publish_date'])
        latest_news = news.iloc[0]

        return {
            "Date": latest_news['publish_date'].strftime("%Y-%m-%d"),
            "Title": latest_news['title'],
            "Source": latest_news['source']
        }
    except Exception as e:
        print(f"Lỗi khi lấy tin tức cho {symbol}: {e}")
        return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

# 15. Phân tích dòng tiền và cảnh báo bất thường
def analyze_cash_flow(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

        avg_volume = df['volume'].mean()
        latest_volume = df['volume'].iloc[-1]
        volume_spike = latest_volume > avg_volume * 2
        volume_warning = latest_volume > avg_volume * 1.5

        if volume_spike:
            assessment = "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền."
        elif volume_warning:
            assessment = "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình."
        else:
            assessment = "Không - Dòng tiền ổn định."

        return {
            "Data Range": f"{start_date} đến {end_date}",
            "Average Volume": round(avg_volume, 0),
            "Latest Volume": round(latest_volume, 0),
            "Volume Spike": volume_spike,
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích dòng tiền cho {symbol}: {e}")
        return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

# CSS để định dạng bảng
TABLE_STYLE = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-size: 14px;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""

# Hàm tạo dashboard tài chính với Matplotlib
def create_financial_dashboard(financial_results, symbol):
    if not financial_results:
        print("Không có dữ liệu để tạo dashboard.")
        return

    df = pd.DataFrame(financial_results)
    years = df["Year"].astype(str)

    for col in ["Net Profit Margin", "Debt to Equity", "P/E Ratio", "Current Ratio", "ROE", "ROA", "Interest Coverage"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    fig, axes = plt.subplots(4, 2, figsize=(12, 16))
    fig.suptitle(f"Dashboard Tài Chính cho mã {symbol} qua các năm", fontsize=16)

    axes[0, 0].plot(years, df["Net Profit Margin"], marker="o", linestyle="-", color="b")
    axes[0, 0].set_title("Net Profit Margin (%)")
    axes[0, 0].set_ylabel("%")

    axes[0, 1].plot(years, df["Debt to Equity"], marker="o", linestyle="-", color="g")
    axes[0, 1].set_title("Debt to Equity")
    axes[0, 1].set_ylabel("Tỷ lệ")

    axes[1, 0].plot(years, df["P/E Ratio"], marker="o", linestyle="-", color="r")
    axes[1, 0].set_title("P/E Ratio")
    axes[1, 0].set_ylabel("Lần")

    axes[1, 1].plot(years, df["Current Ratio"], marker="o", linestyle="-", color="c")
    axes[1, 1].set_title("Current Ratio")
    axes[1, 1].set_ylabel("Tỷ lệ")

    axes[2, 0].plot(years, df["ROE"], marker="o", linestyle="-", color="m")
    axes[2, 0].set_title("ROE (%)")
    axes[2, 0].set_ylabel("%")

    axes[2, 1].plot(years, df["ROA"], marker="o", linestyle="-", color="y")
    axes[2, 1].set_title("ROA (%)")
    axes[2, 1].set_ylabel("%")

    axes[3, 0].plot(years, df["Interest Coverage"], marker="o", linestyle="-", color="k")
    axes[3, 0].set_title("Interest Coverage")
    axes[3, 0].set_ylabel("Lần")

    axes[3, 1].axis("off")

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Hàm kiểm tra mã chứng khoán cơ bản
def is_valid_symbol(symbol):
    return symbol.isalpha() and 3 <= len(symbol) <= 5

# Hàm khởi tạo giao diện ban đầu
def create_initial_interface(output_prediction, output_financial, output_dashboard):
    end_date = datetime.now().strftime("%Y-%m-%d")

    date_picker = widgets.DatePicker(
        description='Chọn ngày bắt đầu:',
        value=datetime(2023, 1, 1),
        disabled=False
    )

    symbol_input = widgets.Text(
        description='Mã chứng khoán:',
        placeholder='Nhập mã (hoặc "exit" để thoát)',
    )

    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output_prediction:
            clear_output()
            custom_start_date = date_picker.value
            if custom_start_date is None:
                print("Vui lòng chọn ngày bắt đầu!")
                return
            custom_start_date = custom_start_date.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            if not is_valid_symbol(symbol):
                print(f"Mã chứng khoán '{symbol}' không hợp lệ! Vui lòng nhập mã gồm 3-5 chữ cái (ví dụ: VRE, SSI, FPT).")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")
            compare_predictions(symbol, custom_start_date, end_date)

            try:
                stock = Vnstock().stock(symbol=symbol, source="VCI")
                income_statement = stock.finance.income_statement(period="year", lang="en")
                balance_sheet = stock.finance.balance_sheet(period="year", lang="en")
            except ValueError as e:
                print(f"Lỗi: {e}. Mã '{symbol}' không được nhận diện bởi vnstock. Vui lòng kiểm tra lại mã hoặc thử mã khác (ví dụ: VRE, SSI).")
                return
            except Exception as e:
                print(f"Lỗi không xác định khi lấy dữ liệu cho {symbol}: {e}")
                return

            income_years = income_statement["yearReport"].unique().tolist() if not income_statement.empty else []
            balance_years = balance_sheet["yearReport"].unique().tolist() if not balance_sheet.empty else []
            all_years = sorted(set(income_years + balance_years))
            print(f"Danh sách các năm có dữ liệu tài chính cho {symbol}: {all_years}")

            num_years_picker = widgets.IntSlider(
                value=5,
                min=1,
                max=min(12, len(all_years)),
                step=1,
                description='Số năm phân tích:',
            )
            button_analyze_years = widgets.Button(description="Phân tích sức khỏe tài chính")

            def on_analyze_years_clicked(b):
                with output_financial:
                    clear_output()
                    num_years = num_years_picker.value
                    latest_year = 2024
                    selected_years = [latest_year - i for i in range(num_years) if (latest_year - i) in all_years]
                    selected_years.reverse()
                    print(f"\nPHÂN TÍCH SỨC KHỎE TÀI CHÍNH CHO MÃ {symbol} TRONG {num_years} NĂM GẦN NHẤT: {selected_years}")

                    financial_results = []
                    for year in selected_years:
                        financial_data = get_financial_data(symbol, year=year)
                        if financial_data:
                            net_profit_margin = (financial_data["net_profit"] / financial_data["revenue"]) * 100 if financial_data["revenue"] > 0 else 0
                            debt_to_equity = financial_data["total_debt"] / financial_data["equity"] if financial_data["equity"] > 0 else float("inf")
                            financial_results.append({
                                "Year": year,
                                "Net Profit Margin": round(net_profit_margin, 2),
                                "Debt to Equity": round(debt_to_equity, 2) if debt_to_equity != float("inf") else "N/A",
                                "P/E Ratio": round(financial_data["pe_ratio"], 2) if financial_data["pe_ratio"] != float("inf") else "N/A",
                                "Current Ratio": round(financial_data["current_ratio"], 2) if financial_data["current_ratio"] != float("inf") else "N/A",
                                "ROE": round(financial_data["roe"], 2),
                                "ROA": round(financial_data["roa"], 2),
                                "Interest Coverage": round(financial_data["interest_coverage"], 2) if financial_data["interest_coverage"] != float("inf") else "N/A"
                            })

                    if financial_results:
                        financial_df = pd.DataFrame(financial_results)
                        display(HTML(TABLE_STYLE + financial_df.to_html(index=False)))
                    else:
                        print("Không có dữ liệu tài chính để hiển thị.")

                with output_dashboard:
                    clear_output()
                    if financial_results:
                        print("Hiển thị dashboard tài chính:")
                        create_financial_dashboard(financial_results, symbol)
                    else:
                        print("Không có dữ liệu tài chính để hiển thị dashboard.")

                with output_financial:
                    if 2024 in selected_years:
                        print(f"\nPHÂN TÍCH TÀI CHÍNH CHI TIẾT CHO MÃ {symbol} NĂM 2024:")
                        financial_data_2024 = get_financial_data(symbol, year=2024)
                        if financial_data_2024:
                            financial_result_2024 = analyze_financials(financial_data_2024, symbol)
                            comments_df = pd.DataFrame(financial_result_2024["Comments"], columns=["Nhận xét chi tiết"])
                            display(HTML(TABLE_STYLE + comments_df.to_html(index=False, escape=False)))

                    print(f"\nPHÂN TÍCH CỔ ĐÔNG LỚN CHO MÃ {symbol}:")
                    shareholders_result = analyze_major_shareholders(symbol)
                    shareholders_df = pd.DataFrame([shareholders_result])
                    display(HTML(TABLE_STYLE + shareholders_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH MỨC ĐỘ GẮN BÓ CỦA LÃNH ĐẠO CHO MÃ {symbol}:")
                    management_result = analyze_management_commitment(symbol)
                    management_df = pd.DataFrame([management_result])
                    display(HTML(TABLE_STYLE + management_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH CÔNG TY CON CHO MÃ {symbol}:")
                    subsidiaries_result = analyze_subsidiaries(symbol)
                    subsidiaries_df = pd.DataFrame([subsidiaries_result])
                    display(HTML(TABLE_STYLE + subsidiaries_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH TÍN HIỆU NỘI BỘ (30 NGÀY GẦN NHẤT) CHO MÃ {symbol}:")
                    insider_result = detect_insider_signals(symbol)
                    insider_df = pd.DataFrame([insider_result])
                    display(HTML(TABLE_STYLE + insider_df.to_html(index=False)))

                    print(f"\nDỰ ĐOÁN BIẾN ĐỘNG GIÁ QUANH SỰ KIỆN GẦN NHẤT CHO MÃ {symbol}:")
                    event_result = predict_event_impact(symbol)
                    event_df = pd.DataFrame([event_result])
                    display(HTML(TABLE_STYLE + event_df.to_html(index=False)))

                    print(f"\nTIN TỨC NÓNG GẦN NHẤT CHO MÃ {symbol}:")
                    news_result = get_latest_news(symbol)
                    news_df = pd.DataFrame([news_result])
                    display(HTML(TABLE_STYLE + news_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH DÒNG TIỀN VÀ CẢNH BÁO BẤT THƯỜNG CHO MÃ {symbol}:")
                    cash_flow_result = analyze_cash_flow(symbol, custom_start_date, end_date)
                    cash_flow_df = pd.DataFrame([cash_flow_result])
                    styled_cash_flow_df = cash_flow_df.style.applymap(
                        lambda val: 'color: red' if val == "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền." else 'color: orange' if val == "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình." else 'color: black',
                        subset=["Assessment"]
                    )
                    display(HTML(TABLE_STYLE + styled_cash_flow_df.to_html(index=False)))

                    display(widgets.HBox([button_continue_yes, button_continue_no]))

            with output_prediction:
                print("\nChọn số năm để phân tích sức khỏe tài chính:")
                display(widgets.VBox([num_years_picker, button_analyze_years]))
            button_analyze_years.on_click(on_analyze_years_clicked)

    def on_yes_clicked(b):
        with output_prediction:
            clear_output()
            create_initial_interface(output_prediction, output_financial, output_dashboard)
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()

    def on_no_clicked(b):
        with output_prediction:
            clear_output()
            print("Đã thoát chương trình.")
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    display(date_picker, symbol_input, button_confirm, output_prediction, output_financial, output_dashboard)

# 16. Chạy trên Colab với widget
def run_prediction():
    output_prediction = widgets.Output()
    output_financial = widgets.Output()
    output_dashboard = widgets.Output()

    create_initial_interface(output_prediction, output_financial, output_dashboard)

if __name__ == "__main__":
    run_prediction()

DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Chọn ngày bắt đầu:')

Text(value='', description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')

Button(description='Xác nhận và dự đoán', style=ButtonStyle())

Output()

Output()

Output()

Dự báo, phân tích sức khỏe tài chính nhiều năm, vẽ DB

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datetime import datetime, timedelta
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import logging
import matplotlib.pyplot as plt

# Tắt cảnh báo từ vnstock
logging.getLogger("vnstock").setLevel(logging.ERROR)
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")

# Cấu hình renderer Plotly cho Colab
pio.renderers.default = "colab"

# CSS để định dạng bảng
TABLE_STYLE = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-size: 14px;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""

# Các hàm phân tích tài chính mới
def initialize_stock(symbol, source='TCBS'):
    return Vnstock().stock(symbol=symbol, source=source)

def get_financial_data(symbol, years, preferred_source='TCBS'):
    sources = [preferred_source, 'VCI'] if preferred_source == 'TCBS' else ['VCI', 'TCBS']
    selected_source = None
    income, balance, ratio, selected_years = None, None, None, []

    for source in sources:
        stock = initialize_stock(symbol, source)
        try:
            income_statement = stock.finance.income_statement(period='year')
            balance_sheet = stock.finance.balance_sheet(period='year')
            ratio = stock.finance.ratio(period='year')

            if any(df.empty for df in [income_statement, balance_sheet, ratio]):
                continue

            if source == 'TCBS':
                income_years = income_statement.index.unique().tolist()
                balance_years = balance_sheet.index.unique().tolist()
                ratio_years = ratio.index.unique().tolist()
            else:
                year_col = next((col for col in ['Năm', 'yearReport'] if col in income_statement.columns), None)
                if year_col is None:
                    continue
                income_years = [y for y in income_statement[year_col].unique().tolist() if isinstance(y, (int, float)) and y >= 2000]
                balance_years = [y for y in balance_sheet[year_col].unique().tolist() if isinstance(y, (int, float)) and y >= 2000]
                ratio_years = [y for y in ratio[year_col].unique().tolist() if isinstance(y, (int, float)) and y >= 2000]

            all_years = sorted(set(income_years + balance_years + ratio_years))
            selected_years = sorted(all_years[-years:]) if len(all_years) >= years else all_years

            if source == 'TCBS':
                income = income_statement.loc[income_statement.index.isin(selected_years)].copy()
                balance = balance_sheet.loc[balance_sheet.index.isin(selected_years)].copy()
                ratio = ratio.loc[ratio.index.isin(selected_years)].copy()
            else:
                income = income_statement[income_statement[year_col].isin(selected_years)].copy()
                balance = balance_sheet[balance_sheet[year_col].isin(selected_years)].copy()
                ratio = ratio[ratio[year_col].isin(selected_years)].copy()
                if not income.empty and year_col in income.columns:
                    income.set_index(year_col, inplace=True)
                if not balance.empty and year_col in balance.columns:
                    balance.set_index(year_col, inplace=True)
                if not ratio.empty and year_col in ratio.columns:
                    ratio.set_index(year_col, inplace=True)

            if source == 'TCBS':
                if 'short_liability' not in balance.columns and 'payable' in balance.columns:
                    balance.loc[:, 'short_liability'] = balance['payable']
                if 'short_asset' not in balance.columns and 'asset' in balance.columns:
                    balance.loc[:, 'short_asset'] = balance['asset']
                if 'long_asset' not in balance.columns and 'asset' in balance.columns:
                    balance.loc[:, 'long_asset'] = balance['asset'] - balance.get('short_asset', balance['asset'])
            else:
                if 'short_asset' not in balance.columns and 'CURRENT ASSETS (Bn. VND)' in balance.columns:
                    balance.loc[:, 'short_asset'] = balance['CURRENT ASSETS (Bn. VND)'] / 1e9
                if 'short_liability' not in balance.columns and 'Short-term borrowings (Bn. VND)' in balance.columns:
                    balance.loc[:, 'short_liability'] = balance['Short-term borrowings (Bn. VND)'] / 1e9
                if 'long_asset' not in balance.columns and 'LONG-TERM ASSETS (Bn. VND)' in balance.columns:
                    balance.loc[:, 'long_asset'] = balance['LONG-TERM ASSETS (Bn. VND)'] / 1e9
                if 'equity' not in balance.columns and "OWNER'S EQUITY(Bn.VND)" in balance.columns:
                    balance.loc[:, 'equity'] = balance["OWNER'S EQUITY(Bn.VND)"] / 1e9

            selected_source = source
            break

        except Exception:
            continue

    if selected_source is None:
        return None, None, None, [], None

    return income, balance, ratio, selected_years, selected_source

def calculate_financial_metrics(income, balance, ratio, selected_years, source='TCBS'):
    result = pd.DataFrame(index=selected_years, columns=[
        'Year', 'Net Profit Margin', 'Debt to Equity', 'P/E Ratio',
        'Current Ratio', 'ROE', 'ROA', 'Interest Coverage'
    ])

    for year in selected_years:
        result.loc[year, 'Year'] = year
        try:
            if source == 'TCBS':
                revenue = income.loc[year, 'revenue'] if year in income.index and 'revenue' in income.columns else np.nan
                profit = income.loc[year, 'post_tax_profit'] if year in income.index and 'post_tax_profit' in income.columns else np.nan
                equity = balance.loc[year, 'equity'] if year in balance.index and 'equity' in balance.columns else np.nan
                debt = balance.loc[year, 'payable'] if year in balance.index and 'payable' in balance.columns else np.nan
                pe = ratio.loc[year, 'price_to_earning'] if year in ratio.index and 'price_to_earning' in ratio.columns else np.nan
                short_asset = balance.loc[year, 'short_asset'] if year in balance.index and 'short_asset' in balance.columns else np.nan
                short_liability = balance.loc[year, 'short_liability'] if year in balance.index and 'short_liability' in balance.columns else np.nan
                total_asset = (balance.loc[year, 'short_asset'] + balance.loc[year, 'long_asset']) if year in balance.index else np.nan
                ebitda = income.loc[year, 'ebitda'] if year in income.index and 'ebitda' in income.columns else np.nan
            else:
                revenue = income.loc[year, 'Revenue (Bn. VND)'] / 1e9 if year in income.index and 'Revenue (Bn. VND)' in income.columns else np.nan
                profit = income.loc[year, 'Net Profit For the Year'] / 1e9 if year in income.index and 'Net Profit For the Year' in income.columns else np.nan
                equity = balance.loc[year, "OWNER'S EQUITY(Bn.VND)"] / 1e9 if year in balance.index and "OWNER'S EQUITY(Bn.VND)" in balance.columns else np.nan
                debt = (balance.loc[year, 'TOTAL ASSETS (Bn. VND)'] / 1e9 - equity) if year in balance.index and 'TOTAL ASSETS (Bn. VND)' in balance.columns else np.nan
                pe = ratio.loc[year, 'P/E'] if year in ratio.index and 'P/E' in ratio.columns else np.nan
                short_asset = balance.loc[year, 'short_asset'] if year in balance.index and 'short_asset' in balance.columns else np.nan
                short_liability = balance.loc[year, 'short_liability'] if year in balance.index and 'short_liability' in balance.columns else np.nan
                total_asset = balance.loc[year, 'TOTAL ASSETS (Bn. VND)'] / 1e9 if year in balance.index and 'TOTAL ASSETS (Bn. VND)' in balance.columns else np.nan
                ebitda = np.nan

            result.loc[year, 'Net Profit Margin'] = (profit / revenue * 100) if revenue and profit else np.nan
            result.loc[year, 'Debt to Equity'] = (debt / equity) if equity and debt else np.nan
            result.loc[year, 'P/E Ratio'] = pe
            result.loc[year, 'Current Ratio'] = (short_asset / short_liability) if short_asset and short_liability else np.nan
            result.loc[year, 'ROE'] = (profit / equity * 100) if equity and profit else np.nan
            result.loc[year, 'ROA'] = (profit / total_asset * 100) if total_asset and profit else np.nan
            result.loc[year, 'Interest Coverage'] = (ebitda / (ebitda - profit)) if ebitda and profit and (ebitda != profit) else np.nan

        except Exception:
            result.loc[year, :] = [year] + [np.nan] * 7

    return result.round(2)

# Hàm lấy dữ liệu với thông tin ngành
def get_financial_data_with_industry(symbol, years, preferred_source='TCBS'):
    income, balance, ratio, selected_years, selected_source = get_financial_data(symbol, years, preferred_source)
    if selected_source is None:
        return None, None, None, [], None, None

    # Lấy thông tin ngành từ TCBS
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        company_info = company.overview()
        industry = company_info['industry'].iloc[0] if not company_info.empty and 'industry' in company_info.columns else "Khác"
    except Exception:
        industry = "Khác"

    return income, balance, ratio, selected_years, selected_source, industry

# Hàm tính điểm sức khỏe tài chính
def calculate_financial_health_score(financial_metrics, industry):
    """
    Tính điểm sức khỏe tài chính và cung cấp nhận xét chi tiết dựa trên các chỉ số tài chính và ngưỡng theo ngành.

    Args:
        financial_metrics (pd.DataFrame): DataFrame chứa các chỉ số tài chính từ calculate_financial_metrics.
        industry (str): Ngành của công ty (Bất động sản, Công nghệ, Ngân hàng, hoặc Khác).

    Returns:
        dict: Chứa điểm tổng, đánh giá chi tiết, nhận xét, và gợi ý.
    """
    # Định nghĩa ngưỡng theo ngành
    industry_thresholds = {
        "Bất động sản": {
            "Net Profit Margin": 10,  # >10%
            "Debt to Equity": 2,      # <2
            "P/E Ratio": 20,          # <20
            "Current Ratio": 1,       # >1
            "ROE": 8,                 # >8%
            "ROA": 3,                 # >3%
            "Interest Coverage": 2    # >2
        },
        "Công nghệ": {
            "Net Profit Margin": 15,  # >15%
            "Debt to Equity": 1,      # <1
            "P/E Ratio": 25,          # <25
            "Current Ratio": 1.5,     # >1.5
            "ROE": 15,                # >15%
            "ROA": 10,                # >10%
            "Interest Coverage": 5    # >5
        },
        "Ngân hàng": {
            "Net Profit Margin": 20,  # >20%
            "Debt to Equity": 10,     # <10
            "P/E Ratio": 15,          # <15
            "Current Ratio": 1,       # >1
            "ROE": 12,                # >12%
            "ROA": 1,                 # >1%
            "Interest Coverage": 3    # >3
        },
        "Khác": {
            "Net Profit Margin": 10,  # >10%
            "Debt to Equity": 1,      # <1
            "P/E Ratio": 15,          # <15
            "Current Ratio": 1,       # >1
            "ROE": 10,                # >10%
            "ROA": 5,                 # >5%
            "Interest Coverage": 3    # >3
        }
    }

    # Định nghĩa trọng số
    weights = {
        "Net Profit Margin": 2.0,
        "Debt to Equity": 1.5,
        "P/E Ratio": 1.0,
        "Current Ratio": 1.5,
        "ROE": 2.0,
        "ROA": 1.5,
        "Interest Coverage": 0.5
    }

    # Lấy ngưỡng dựa trên ngành
    thresholds = industry_thresholds.get(industry, industry_thresholds["Khác"])

    # Khởi tạo điểm tổng
    total_score = 0
    comments = []
    detailed_assessment = []

    # Tính điểm và nhận xét cho từng chỉ số
    for metric in financial_metrics.columns[1:]:  # Bỏ cột 'Year'
        value = financial_metrics[metric].iloc[-1]  # Lấy giá trị mới nhất
        threshold = thresholds[metric]
        weight = weights[metric]

        if pd.isna(value):
            score = 0
            comment = f"{metric}: Không có dữ liệu."
            detail = f"{metric}: Không có dữ liệu để đánh giá."
        else:
            if metric in ["Net Profit Margin", "ROE", "ROA"]:
                # So sánh phần trăm
                if value > threshold:
                    score = weight
                    comment = f"{metric} ({value:.2f}%): Đạt ngưỡng (> {threshold}%), được {weight} điểm."
                    detail = f"{metric} ({value:.2f}%): Mức độ sinh lời vượt trội so với ngưỡng ngành."
                else:
                    score = 0
                    comment = f"{metric} ({value:.2f}%): Không đạt ngưỡng (> {threshold}%), 0 điểm."
                    detail = f"{metric} ({value:.2f}%): Mức độ sinh lời thấp hơn ngưỡng ngành, cần cải thiện hiệu quả."
            elif metric in ["Debt to Equity", "P/E Ratio"]:
                # So sánh nhỏ hơn
                if value < threshold:
                    score = weight
                    comment = f"{metric} ({value:.2f}): Đạt ngưỡng (< {threshold}), được {weight} điểm."
                    detail = f"{metric} ({value:.2f}): Mức độ rủi ro tài chính hoặc định giá hợp lý."
                else:
                    score = 0
                    comment = f"{metric} ({value:.2f}): Không đạt ngưỡng (< {threshold}), 0 điểm."
                    detail = f"{metric} ({value:.2f}): Mức độ rủi ro tài chính hoặc định giá cao hơn ngưỡng, cần thận trọng."
            elif metric in ["Current Ratio", "Interest Coverage"]:
                # So sánh lớn hơn
                if value > threshold:
                    score = weight
                    comment = f"{metric} ({value:.2f}): Đạt ngưỡng (> {threshold}), được {weight} điểm."
                    detail = f"{metric} ({value:.2f}): Khả năng thanh khoản hoặc chi trả lãi tốt."
                else:
                    score = 0
                    comment = f"{metric} ({value:.2f}): Không đạt ngưỡng (> {threshold}), 0 điểm."
                    detail = f"{metric} ({value:.2f}): Khả năng thanh khoản hoặc chi trả lãi yếu, tiềm ẩn rủi ro."

        total_score += score
        comments.append(comment)
        detailed_assessment.append(detail)

    # Đánh giá tổng quát
    assessment = "Tích cực" if total_score >= 7 else "Trung lập" if total_score >= 4 else "Tiêu cực"
    overall_comment = f"**Sức khỏe tài chính**: Điểm tổng {total_score:.2f}/10.00, đánh giá '{assessment}' trong ngành {industry}."
    comments.insert(0, overall_comment)

    # Gợi ý dựa trên điểm số
    suggestions = []
    if total_score < 4:
        suggestions.append("Gợi ý: Cân nhắc giảm đầu tư hoặc theo dõi sát sao các chỉ số tài chính, đặc biệt là Debt to Equity và Current Ratio.")
    elif total_score < 7:
        suggestions.append("Gợi ý: Duy trì giám sát, cải thiện các chỉ số dưới ngưỡng như ROE hoặc ROA để tăng sức khỏe tài chính.")
    else:
        suggestions.append("Gợi ý: Tiếp tục duy trì chiến lược hiện tại, đây là thời điểm tiềm năng để đầu tư hoặc mở rộng.")

    return {
        "Financial Score": round(total_score, 2),
        "Assessment": assessment,
        "Comments": comments,
        "Detailed Assessment": "\n".join(detailed_assessment),
        "Suggestions": suggestions
    }

# Hàm phân tích sức khỏe tài chính
def analyze_financial_health(symbol, years):
    income, balance, ratio, selected_years, source, industry = get_financial_data_with_industry(symbol, years)
    if income is None:
        return {
            "Assessment": "N/A",
            "Financial Score": 0,
            "Detailed Assessment": "Không có dữ liệu để phân tích.",
            "Comments": ["Không có dữ liệu tài chính."],
            "Suggestions": ["Không có dữ liệu để đưa ra gợi ý."],
            "Financial Metrics": None
        }

    financial_metrics = calculate_financial_metrics(income, balance, ratio, selected_years, source)
    health_score = calculate_financial_health_score(financial_metrics, industry)

    return {
        "Financial Metrics": financial_metrics,
        "Health Score": health_score
    }

# 1. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 5 or not all(col in df.columns for col in required_columns):
            print(f"Dữ liệu không đủ cho {symbol} (ít hơn 5 ngày hoặc thiếu cột).")
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")
        return None

# 2. Chỉ báo kỹ thuật
def calculate_indicators(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_3": ta.trend.ema_indicator(df["close"], window=3),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "Chaikin_Vol": (df["high"] - df["low"]).ewm(span=5, adjust=False).mean(),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "Daily_VWAP": (df["close"] * df["volume"]) / df["volume"],
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5),
            "StochRSI": ta.momentum.stochrsi(close=df["close"], window=5, smooth1=3, smooth2=3)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi tính chỉ báo: {e}")
        return None

# 3. Alligator
def add_alligator(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi thêm Alligator: {e}")
        return None

# 4. Chuẩn bị dữ liệu phân loại
def prepare_classification_data(df):
    if df is None or df.empty:
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty:
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        print(f"Lỗi khi chuẩn bị dữ liệu: {e}")
        return None, None, None

# 5. Dự báo và backtesting
def forecast_and_backtest(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators(df)
    if df is None:
        return None

    df = add_alligator(df)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 5 ngày. Bỏ qua {name}.")
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 10 ngày. Bỏ qua {name}.")
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for pred in predictions_5.values() if pred == 1)
    buy_votes_10 = sum(1 for pred in predictions_10.values() if pred == 1)
    final_rec = "MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN"

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df
    }

# 6. So sánh kết quả với "Toàn bộ từ ngày chọn" và "120 ngày"
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}

    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest(symbol, custom_start_date, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest(symbol, start_120, end_date)

    periods = ["Toàn bộ từ ngày chọn", "120 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else 120

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.applymap(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 7. Phân tích cổ đông lớn
def analyze_major_shareholders(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        shareholders = company.shareholders()
        if shareholders.empty or 'share_own_percent' not in shareholders.columns:
            print(f"Dữ liệu cổ đông lớn cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_major_share = shareholders['share_own_percent'].sum() * 100
        top_shareholder = shareholders['share_own_percent'].iloc[0] * 100

        if total_major_share > 50:
            assessment = "Cao - Rủi ro quản trị cao, nhưng có thể tăng giá nếu cổ đông lớn mua thêm."
        elif total_major_share > 30:
            assessment = "Trung bình - Ảnh hưởng đáng kể từ cổ đông lớn."
        else:
            assessment = "Thấp - Quyền kiểm soát phân tán, ít rủi ro quản trị."

        return {
            "Total Major Share (%)": round(total_major_share, 2),
            "Top Shareholder (%)": round(top_shareholder, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích cổ đông lớn cho {symbol}: {e}")
        return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 8. Phân tích mức độ gắn bó của lãnh đạo
def analyze_management_commitment(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        officers = company.officers()
        if officers.empty or 'ownPercent' not in officers.columns:
            print(f"Dữ liệu lãnh đạo cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_management_share = officers['ownPercent'].sum()

        if total_management_share > 5:
            assessment = "Cao - Lãnh đạo có cam kết mạnh mẽ với công ty."
        elif total_management_share > 1:
            assessment = "Trung bình - Cam kết ở mức khá."
        else:
            assessment = "Thấp - Lãnh đạo ít gắn bó về mặt sở hữu."

        return {
            "Management Ownership (%)": round(total_management_share, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích mức độ gắn bó của lãnh đạo cho {symbol}: {e}")
        return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 9. Phân tích công ty con
def analyze_subsidiaries(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        subsidiaries = company.subsidiaries()
        if subsidiaries.empty or 'sub_own_percent' not in subsidiaries.columns:
            print(f"Dữ liệu công ty con cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        num_subsidiaries = len(subsidiaries)
        avg_ownership = subsidiaries['sub_own_percent'].mean() * 100

        if num_subsidiaries > 5 and avg_ownership > 50:
            assessment = "Đa dạng hóa tốt - Tiềm năng lợi nhuận từ công ty con cao."
        elif num_subsidiaries > 2:
            assessment = "Đa dạng hóa trung bình - Tiềm năng lợi nhuận khá."
        else:
            assessment = "Đa dạng hóa thấp - Ít phụ thuộc vào công ty con."

        return {
            "Number of Subsidiaries": num_subsidiaries,
            "Average Ownership (%)": round(avg_ownership, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích công ty con cho {symbol}: {e}")
        return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 10. Phát hiện tín hiệu nội bộ
def detect_insider_signals(symbol, days=30):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        insider_deals = company.insider_deals()
        if insider_deals.empty or 'deal_quantity' not in insider_deals.columns:
            print(f"Dữ liệu giao dịch nội bộ cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

        insider_deals['deal_announce_date'] = pd.to_datetime(insider_deals['deal_announce_date'])
        recent_deals = insider_deals[insider_deals['deal_announce_date'] > datetime.now() - timedelta(days=days)]

        if recent_deals.empty:
            return {"Net Insider Volume": 0, "Assessment": "Trung lập - Không có giao dịch gần đây."}

        net_volume = recent_deals['deal_quantity'].sum()

        if net_volume > 0:
            assessment = "Tích cực - Lãnh đạo mua ròng, dấu hiệu tốt cho triển vọng."
        elif net_volume < 0:
            assessment = "Tiêu cực - Lãnh đạo bán ròng, cần thận trọng."
        else:
            assessment = "Trung lập - Không có tín hiệu rõ ràng."

        return {
            "Net Insider Volume": int(net_volume),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phát hiện tín hiệu nội bộ cho {symbol}: {e}")
        return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

# 11. Dự đoán biến động giá quanh sự kiện
def predict_event_impact(symbol, days_before=5, days_after=5):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        events = company.events()
        if events.empty or 'exer_date' not in events.columns:
            print(f"Dữ liệu sự kiện cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

        events['exer_date'] = pd.to_datetime(events['exer_date'])
        recent_event = events[events['exer_date'] <= datetime.now()].iloc[-1]
        event_date = recent_event['exer_date']
        event_desc = recent_event['event_desc']

        start_date = (event_date - timedelta(days=days_before)).strftime("%Y-%m-%d")
        end_date = (event_date + timedelta(days=days_after)).strftime("%Y-%m-%d")
        price_data = get_stock_data(symbol, start_date, end_date)

        if price_data is None or len(price_data) < days_before + days_after:
            print(f"Không đủ dữ liệu giá quanh sự kiện cho {symbol}")
            return {"Event Date": event_date.strftime("%Y-%m-%d"), "Event Description": event_desc, "Price Change (%)": "N/A", "Assessment": "Không đủ dữ liệu giá"}

        pre_event_price = price_data['close'].iloc[days_before - 1]
        post_event_price = price_data['close'].iloc[-1]
        price_change = ((post_event_price - pre_event_price) / pre_event_price) * 100

        assessment = "Tăng giá" if price_change > 0 else "Giảm giá"

        return {
            "Event Date": event_date.strftime("%Y-%m-%d"),
            "Event Description": event_desc,
            "Price Change (%)": round(price_change, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi dự đoán biến động giá quanh sự kiện cho {symbol}: {e}")
        return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 12. Cập nhật tin tức nóng
def get_latest_news(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        news = company.news()
        if news.empty or 'publish_date' not in news.columns:
            print(f"Dữ liệu tin tức cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

        news['publish_date'] = pd.to_datetime(news['publish_date'])
        latest_news = news.iloc[0]

        return {
            "Date": latest_news['publish_date'].strftime("%Y-%m-%d"),
            "Title": latest_news['title'],
            "Source": latest_news['source']
        }
    except Exception as e:
        print(f"Lỗi khi lấy tin tức cho {symbol}: {e}")
        return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

# 13. Phân tích dòng tiền và cảnh báo bất thường
def analyze_cash_flow(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

        avg_volume = df['volume'].mean()
        latest_volume = df['volume'].iloc[-1]
        volume_spike = latest_volume > avg_volume * 2
        volume_warning = latest_volume > avg_volume * 1.5

        if volume_spike:
            assessment = "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền."
        elif volume_warning:
            assessment = "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình."
        else:
            assessment = "Không - Dòng tiền ổn định."

        return {
            "Data Range": f"{start_date} đến {end_date}",
            "Average Volume": round(avg_volume, 0),
            "Latest Volume": round(latest_volume, 0),
            "Volume Spike": volume_spike,
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích dòng tiền cho {symbol}: {e}")
        return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

# Hàm tạo dashboard tài chính với Matplotlib
def create_financial_dashboard(financial_results, symbol):
    if not financial_results:
        print("Không có dữ liệu để tạo dashboard.")
        return

    df = pd.DataFrame(financial_results)
    years = df["Year"].astype(str)

    for col in ["Net Profit Margin", "Debt to Equity", "P/E Ratio", "Current Ratio", "ROE", "ROA", "Interest Coverage"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    fig, axes = plt.subplots(4, 2, figsize=(12, 16))
    fig.suptitle(f"Dashboard Tài Chính cho mã {symbol} qua các năm", fontsize=16)

    axes[0, 0].plot(years, df["Net Profit Margin"], marker="o", linestyle="-", color="b")
    axes[0, 0].set_title("Net Profit Margin (%)")
    axes[0, 0].set_ylabel("%")

    axes[0, 1].plot(years, df["Debt to Equity"], marker="o", linestyle="-", color="g")
    axes[0, 1].set_title("Debt to Equity")
    axes[0, 1].set_ylabel("Tỷ lệ")

    axes[1, 0].plot(years, df["P/E Ratio"], marker="o", linestyle="-", color="r")
    axes[1, 0].set_title("P/E Ratio")
    axes[1, 0].set_ylabel("Lần")

    axes[1, 1].plot(years, df["Current Ratio"], marker="o", linestyle="-", color="c")
    axes[1, 1].set_title("Current Ratio")
    axes[1, 1].set_ylabel("Tỷ lệ")

    axes[2, 0].plot(years, df["ROE"], marker="o", linestyle="-", color="m")
    axes[2, 0].set_title("ROE (%)")
    axes[2, 0].set_ylabel("%")

    axes[2, 1].plot(years, df["ROA"], marker="o", linestyle="-", color="y")
    axes[2, 1].set_title("ROA (%)")
    axes[2, 1].set_ylabel("%")

    axes[3, 0].plot(years, df["Interest Coverage"], marker="o", linestyle="-", color="k")
    axes[3, 0].set_title("Interest Coverage")
    axes[3, 0].set_ylabel("Lần")

    axes[3, 1].axis("off")

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Hàm kiểm tra mã chứng khoán cơ bản
def is_valid_symbol(symbol):
    return symbol.isalpha() and 3 <= len(symbol) <= 5

# Hàm khởi tạo giao diện ban đầu với hàm on_analyze_years_clicked được cập nhật
def create_initial_interface(output_prediction, output_financial, output_dashboard):
    end_date = datetime.now().strftime("%Y-%m-%d")

    date_picker = widgets.DatePicker(
        description='Chọn ngày bắt đầu:',
        value=datetime(2023, 1, 1),
        disabled=False
    )

    symbol_input = widgets.Text(
        description='Mã chứng khoán:',
        placeholder='Nhập mã (hoặc "exit" để thoát)',
    )

    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output_prediction:
            clear_output()
            custom_start_date = date_picker.value
            if custom_start_date is None:
                print("Vui lòng chọn ngày bắt đầu!")
                return
            custom_start_date = custom_start_date.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            if not is_valid_symbol(symbol):
                print(f"Mã chứng khoán '{symbol}' không hợp lệ! Vui lòng nhập mã gồm 3-5 chữ cái (ví dụ: VRE, SSI, FPT).")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")
            compare_predictions(symbol, custom_start_date, end_date)

            try:
                stock = Vnstock().stock(symbol=symbol, source="VCI")
                income_statement = stock.finance.income_statement(period="year", lang="en")
                balance_sheet = stock.finance.balance_sheet(period="year", lang="en")
            except ValueError as e:
                print(f"Lỗi: {e}. Mã '{symbol}' không được nhận diện bởi vnstock. Vui lòng kiểm tra lại mã hoặc thử mã khác (ví dụ: VRE, SSI).")
                return
            except Exception as e:
                print(f"Lỗi không xác định khi lấy dữ liệu cho {symbol}: {e}")
                return

            income_years = income_statement["yearReport"].unique().tolist() if not income_statement.empty else []
            balance_years = balance_sheet["yearReport"].unique().tolist() if not balance_sheet.empty else []
            all_years = sorted(set(income_years + balance_years))
            print(f"Danh sách các năm có dữ liệu tài chính cho {symbol}: {all_years}")

            num_years_picker = widgets.IntSlider(
                value=5,
                min=1,
                max=min(12, len(all_years)),
                step=1,
                description='Số năm phân tích:',
            )
            button_analyze_years = widgets.Button(description="Phân tích sức khỏe tài chính")

            def on_analyze_years_clicked(b):
                with output_financial:
                    clear_output()
                    num_years = num_years_picker.value
                    # Gọi hàm analyze_financial_health để phân tích sức khỏe tài chính
                    result = analyze_financial_health(symbol, num_years)

                    if result["Financial Metrics"] is None:
                        print(f"Không thể phân tích tài chính cho mã {symbol}.")
                    else:
                        financial_metrics = result["Financial Metrics"]
                        health_score = result["Health Score"]

                        # Hiển thị kết quả phân tích
                        print(f"\nPHÂN TÍCH SỨC KHỎE TÀI CHÍNH CHO MÃ {symbol} TRONG {num_years} NĂM GẦN NHẤT:")
                        income, balance, ratio, selected_years, source, industry = get_financial_data_with_industry(symbol, num_years)
                        print(f"Nguồn dữ liệu: {source}")
                        print(f"Các năm được phân tích: {selected_years}")
                        print(f"Ngành: {industry}")
                        display(HTML(TABLE_STYLE + financial_metrics.to_html(index=False)))

                        print(f"\nĐÁNH GIÁ SỨC KHỎE TÀI CHÍNH:")
                        for comment in health_score["Comments"]:
                            print(comment)

                        print(f"\nĐÁNH GIÁ CHI TIẾT:")
                        print(health_score["Detailed Assessment"])

                        print(f"\nGỢI Ý:")
                        for suggestion in health_score["Suggestions"]:
                            print(suggestion)

                with output_dashboard:
                    clear_output()
                    if result["Financial Metrics"] is not None:
                        print("Hiển thị dashboard tài chính:")
                        financial_data = result["Financial Metrics"].to_dict('records')
                        create_financial_dashboard(financial_data, symbol)
                    else:
                        print("Không có dữ liệu tài chính để hiển thị dashboard.")

                with output_financial:
                    print(f"\nPHÂN TÍCH CỔ ĐÔNG LỚN CHO MÃ {symbol}:")
                    shareholders_result = analyze_major_shareholders(symbol)
                    shareholders_df = pd.DataFrame([shareholders_result])
                    display(HTML(TABLE_STYLE + shareholders_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH MỨC ĐỘ GẮN BÓ CỦA LÃNH ĐẠO CHO MÃ {symbol}:")
                    management_result = analyze_management_commitment(symbol)
                    management_df = pd.DataFrame([management_result])
                    display(HTML(TABLE_STYLE + management_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH CÔNG TY CON CHO MÃ {symbol}:")
                    subsidiaries_result = analyze_subsidiaries(symbol)
                    subsidiaries_df = pd.DataFrame([subsidiaries_result])
                    display(HTML(TABLE_STYLE + subsidiaries_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH TÍN HIỆU NỘI BỘ (30 NGÀY GẦN NHẤT) CHO MÃ {symbol}:")
                    insider_result = detect_insider_signals(symbol)
                    insider_df = pd.DataFrame([insider_result])
                    display(HTML(TABLE_STYLE + insider_df.to_html(index=False)))

                    print(f"\nDỰ ĐOÁN BIẾN ĐỘNG GIÁ QUANH SỰ KIỆN GẦN NHẤT CHO MÃ {symbol}:")
                    event_result = predict_event_impact(symbol)
                    event_df = pd.DataFrame([event_result])
                    display(HTML(TABLE_STYLE + event_df.to_html(index=False)))

                    print(f"\nTIN TỨC NÓNG GẦN NHẤT CHO MÃ {symbol}:")
                    news_result = get_latest_news(symbol)
                    news_df = pd.DataFrame([news_result])
                    display(HTML(TABLE_STYLE + news_df.to_html(index=False)))

                    print(f"\nPHÂN TÍCH DÒNG TIỀN VÀ CẢNH BÁO BẤT THƯỜNG CHO MÃ {symbol}:")
                    cash_flow_result = analyze_cash_flow(symbol, custom_start_date, end_date)
                    cash_flow_df = pd.DataFrame([cash_flow_result])
                    styled_cash_flow_df = cash_flow_df.style.applymap(
                        lambda val: 'color: red' if val == "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền." else 'color: orange' if val == "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình." else 'color: black',
                        subset=["Assessment"]
                    )
                    display(HTML(TABLE_STYLE + styled_cash_flow_df.to_html(index=False)))

                    display(widgets.HBox([button_continue_yes, button_continue_no]))

            with output_prediction:
                print("\nChọn số năm để phân tích sức khỏe tài chính:")
                display(widgets.VBox([num_years_picker, button_analyze_years]))
            button_analyze_years.on_click(on_analyze_years_clicked)

    def on_yes_clicked(b):
        with output_prediction:
            clear_output()
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()
        create_initial_interface(output_prediction, output_financial, output_dashboard)

    def on_no_clicked(b):
        with output_prediction:
            clear_output()
            print("Đã thoát chương trình.")
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    display(date_picker, symbol_input, button_confirm, output_prediction, output_financial, output_dashboard)

# Chạy chương trình
def run_prediction():
    output_prediction = widgets.Output()
    output_financial = widgets.Output()
    output_dashboard = widgets.Output()

    create_initial_interface(output_prediction, output_financial, output_dashboard)

if __name__ == "__main__":
    run_prediction()


    Khi tiếp tục sử dụng Vnstock3, bạn xác nhận rằng bạn đã đọc, hiểu và đồng ý với Chính sách quyền riêng tư và Điều khoản, điều kiện về giấy phép sử dụng Vnstock3.

    Chi tiết:

    - Giấy phép sử dụng phần mềm: https://vnstocks.com/docs/tai-lieu/giay-phep-su-dung
    - Chính sách quyền riêng tư: https://vnstocks.com/docs/tai-lieu/chinh-sach-quyen-rieng-tu
    


DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Chọn ngày bắt đầu:')

Text(value='', description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')

Button(description='Xác nhận và dự đoán', style=ButtonStyle())

Output()

Output()

Output()

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datetime import datetime, timedelta
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import logging
import matplotlib.pyplot as plt

# Tắt cảnh báo từ vnstock
logging.getLogger("vnstock").setLevel(logging.ERROR)
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")

# CSS để định dạng bảng
TABLE_STYLE = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-size: 14px;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""

# Hàm lấy dữ liệu tài chính
def get_financial_data(symbol, num_years):
    try:
        stock = Vnstock().stock(symbol=symbol, source='TCBS')
        income = stock.finance.income_statement(period='year')
        balance = stock.finance.balance_sheet(period='year')
        ratio = stock.finance.ratio(period='year')

        income_years = income.index.tolist() if not income.empty else []
        balance_years = balance.index.tolist() if not balance.empty else []
        all_years = sorted(set(income_years + balance_years), reverse=True)
        selected_years = all_years[:min(num_years, len(all_years))]

        company_info = stock.company.overview()
        industry = company_info['industry'].iloc[0] if not company_info.empty else "Khác"

        return income, balance, ratio, selected_years, "TCBS", industry
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu tài chính: {e}")
        return None, None, None, [], "TCBS", "Khác"

# Hàm tính toán các chỉ số tài chính
def calculate_financial_metrics(income, balance, ratio, selected_years, source):
    financial_data = []
    for year in selected_years:
        income_year = income.loc[year] if year in income.index else None
        balance_year = balance.loc[year] if year in balance.index else None
        ratio_year = ratio.loc[year] if year in ratio.index else None

        if income_year is not None and balance_year is not None:
            net_profit = income_year.get("post_tax_profit", 0)
            revenue = income_year.get("revenue", 0)
            equity = balance_year.get("equity", 0)
            total_assets = balance_year.get("total_assets", 0)
            total_debt = balance_year.get("payable", 0)

            net_profit_margin = (net_profit / revenue * 100) if revenue != 0 else 0
            debt_to_equity = (total_debt / equity) if equity != 0 else 0
            current_ratio = (balance_year.get("short_asset", 0) / balance_year.get("short_liability", 0)) if balance_year.get("short_liability", 0) != 0 else 0
            roe = (net_profit / equity * 100) if equity != 0 else 0
            roa = (net_profit / total_assets * 100) if total_assets != 0 else 0
            pe_ratio = ratio_year.get("price_to_earning", 0) if ratio_year is not None else 0
            interest_coverage = (income_year.get("ebitda", 0) / income_year.get("interest_expense", 0)) if income_year.get("interest_expense", 0) != 0 else np.nan

            financial_data.append({
                "Year": year,
                "Net Profit Margin": net_profit_margin,
                "Debt to Equity": debt_to_equity,
                "P/E Ratio": pe_ratio,
                "Current Ratio": current_ratio,
                "ROE": roe,
                "ROA": roa,
                "Interest Coverage": interest_coverage
            })
    return pd.DataFrame(financial_data)

# Hàm phân tích sức khỏe tài chính
def analyze_financial_health(symbol, num_years):
    income, balance, ratio, selected_years, source, industry = get_financial_data(symbol, num_years)
    if income is None or balance is None:
        return {"Financial Metrics": None, "Health Score": None}
    financial_metrics = calculate_financial_metrics(income, balance, ratio, selected_years, source)
    health_score = {"Comments": ["Sức khỏe tài chính ổn định"], "Detailed Assessment": "ROE và ROA tích cực", "Suggestions": ["Tiếp tục theo dõi"]}
    return {"Financial Metrics": financial_metrics, "Health Score": health_score}

# Hàm gửi prompt đến Grok (sửa lỗi)
def send_prompt_to_grok(prompt, symbol, company_name=None):
    stock = Vnstock().stock(symbol=symbol, source='TCBS')

    if "thông tin cơ bản" in prompt:
        company_info = stock.company.overview()
        price_data = stock.quote.history(start=(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d"), end=datetime.now().strftime("%Y-%m-%d"))
        avg_volume = price_data['volume'].mean() if not price_data.empty else 0
        current_price = price_data['close'].iloc[-1] * 1000 if not price_data.empty else 0
        ratio = stock.finance.ratio(period='year')
        latest_ratio = ratio.iloc[-1] if not ratio.empty else {}

        outstanding_share = company_info['outstanding_share'].iloc[0] * 1e6 if not company_info.empty and 'outstanding_share' in company_info.columns else 0
        market_cap = (current_price * outstanding_share) / 1e9 if outstanding_share > 0 and current_price > 0 else 0

        return {
            "Tên công ty": company_info['short_name'].iloc[0] if not company_info.empty and 'short_name' in company_info.columns else "Không xác định",
            "Ngành nghề kinh doanh chính": company_info['industry'].iloc[0] if not company_info.empty else "Không xác định",
            "Vốn hóa thị trường": market_cap,
            "Khối lượng giao dịch trung bình": avg_volume,
            "Giá cổ phiếu hiện tại": current_price,
            "P/E": latest_ratio.get("price_to_earning", 0),
            "P/B": latest_ratio.get("price_to_book", 0),
            "EPS": latest_ratio.get("earning_per_share", 0)
        }

    elif "báo cáo tài chính" in prompt:
        income = stock.finance.income_statement(period='year')
        balance = stock.finance.balance_sheet(period='year')
        cash_flow = stock.finance.cash_flow(period='year')
        latest_income = income.iloc[-1] if not income.empty else {}
        latest_balance = balance.iloc[-1] if not balance.empty else {}
        latest_cash = cash_flow.iloc[-1] if not cash_flow.empty else {}

        revenue = latest_income.get("revenue", 0) / 1e9  # Chuyển sang tỷ VNĐ
        net_profit = latest_income.get("post_tax_profit", 0) / 1e9
        total_assets = latest_balance.get("total_assets", 0) / 1e9
        total_debt = latest_balance.get("payable", 0) / 1e9

        return {
            "Doanh thu": revenue,
            "Lợi nhuận ròng": net_profit,
            "Tổng tài sản": total_assets,
            "Tổng nợ": total_debt,
            "Biên lợi nhuận gộp": latest_income.get("gross_profit", 0) / latest_income.get("revenue", 1) * 100 if latest_income.get("revenue", 0) != 0 else 0,
            "Biên lợi nhuận ròng": net_profit / revenue * 100 if revenue != 0 else 0,
            "Tỷ lệ nợ trên vốn chủ sở hữu (D/E)": total_debt / latest_balance.get("equity", 1) if latest_balance.get("equity", 0) != 0 else 0,
            "Dòng tiền từ hoạt động kinh doanh": latest_cash.get("from_sale", 0) / 1e9 if not cash_flow.empty else 0
        }

    elif "phân tích kỹ thuật" in prompt:
        price_data = stock.quote.history(start=(datetime.now() - timedelta(days=200)).strftime("%Y-%m-%d"), end=datetime.now().strftime("%Y-%m-%d"))
        if not price_data.empty and len(price_data) >= 50:  # Đảm bảo đủ dữ liệu
            df = ta.add_all_ta_features(price_data, "open", "high", "low", "close", "volume", fillna=True)
            latest = df.iloc[-1]
            trend = "tăng" if latest["trend_ema_fast"] > latest["trend_ema_slow"] else "giảm" if latest["trend_ema_fast"] < latest["trend_ema_slow"] else "đi ngang"
            support = df["close"].min()
            resistance = df["close"].max()
            return {
                "Xu hướng chung": trend,
                "Hỗ trợ": support * 1000,
                "Kháng cự": resistance * 1000,
                "RSI": latest["momentum_rsi"],
                "MACD": latest["trend_macd"],
                "MA50": latest["trend_sma_50"] * 1000,
                "MA200": latest["trend_sma_200"] * 1000,
                "Khuyến nghị": "Mua" if latest["momentum_rsi"] < 30 else "Bán" if latest["momentum_rsi"] > 70 else "Giữ"
            }
        return {"Error": "Không đủ dữ liệu giá để phân tích kỹ thuật"}

    elif "định giá cổ phiếu" in prompt:
        ratio = stock.finance.ratio(period='year')
        latest_ratio = ratio.iloc[-1] if not ratio.empty else {}
        income = stock.finance.income_statement(period='year')
        growth_rate = (income["revenue"].pct_change().mean() * 100) if not income.empty and len(income) > 1 else 0
        pe = latest_ratio.get("price_to_earning", 0)
        return {
            "P/E": pe,
            "P/B": latest_ratio.get("price_to_book", 0),
            "EV/EBITDA": latest_ratio.get("value_before_ebitda", 0),
            "Tốc độ tăng trưởng": growth_rate,
            "Định giá": "Thấp" if pe < 10 and pe > 0 else "Cao" if pe > 20 else "Hợp lý"
        }

    elif "tin tức mới nhất" in prompt:
        news = stock.company.news()
        if not news.empty:
            latest_news = news.iloc[0]
            return {
                "Tiêu đề": latest_news["title"],
                "Ngày": latest_news["time"] if "time" in latest_news else "Không có ngày",
                "Tóm tắt": latest_news["content"][:200] + "..." if "content" in latest_news else "Không có nội dung"
            }
        return {"Error": "Không có tin tức"}

# Hàm tạo báo cáo tổng hợp
def generate_comprehensive_report(symbol, output):
    with output:
        clear_output()
        stock = Vnstock().stock(symbol=symbol, source='TCBS')
        company_info = stock.company.overview()
        print("Dữ liệu thô từ company.overview():")
        print(company_info)
        print("Các cột có sẵn:", list(company_info.columns))

        if not company_info.empty:
            if 'companyName' in company_info.columns:
                company_name = company_info['companyName'].iloc[0]
            elif 'company_name' in company_info.columns:
                company_name = company_info['company_name'].iloc[0]
            elif 'short_name' in company_info.columns:
                company_name = company_info['short_name'].iloc[0]
            else:
                company_name = symbol
        else:
            company_name = symbol

        # Prompt 1: Thông tin cơ bản
        print("\n1. THÔNG TIN CƠ BẢN")
        basic_info = send_prompt_to_grok(
            "Hãy tìm thông tin cơ bản về mã chứng khoán [MÃ CỔ PHIẾU] trên thị trường chứng khoán Việt Nam...",
            symbol
        )
        display(HTML(TABLE_STYLE + pd.DataFrame([basic_info]).to_html(index=False)))

        # Prompt 2: Báo cáo tài chính
        print("\n2. BÁO CÁO TÀI CHÍNH")
        financial_report = send_prompt_to_grok(
            f"Tìm báo cáo tài chính mới nhất của công ty {company_name} với mã chứng khoán {symbol}...",
            symbol, company_name
        )
        display(HTML(TABLE_STYLE + pd.DataFrame([financial_report]).to_html(index=False)))

        # Prompt 3: Phân tích kỹ thuật
        print("\n3. PHÂN TÍCH KỸ THUẬT")
        technical_analysis = send_prompt_to_grok(
            f"Phân tích kỹ thuật cho mã cổ phiếu {symbol} dựa trên dữ liệu giá gần đây...",
            symbol
        )
        display(HTML(TABLE_STYLE + pd.DataFrame([technical_analysis]).to_html(index=False)))

        # Prompt 4: Định giá & triển vọng
        print("\n4. ĐỊNH GIÁ & TRIỂN VỌNG ĐẦU TƯ")
        valuation = send_prompt_to_grok(
            f"Định giá cổ phiếu {symbol} dựa trên các phương pháp P/E, P/B, EV/EBITDA, DCF...",
            symbol
        )
        display(HTML(TABLE_STYLE + pd.DataFrame([valuation]).to_html(index=False)))

        # Prompt 5: Tin tức & sự kiện
        print("\n5. TIN TỨC & SỰ KIỆN")
        news_analysis = send_prompt_to_grok(
            f"Tìm các tin tức mới nhất về công ty {company_name} và mã chứng khoán {symbol}...",
            symbol, company_name
        )
        display(HTML(TABLE_STYLE + pd.DataFrame([news_analysis]).to_html(index=False)))

# Cập nhật giao diện
def create_initial_interface(output_prediction, output_financial, output_dashboard, output_report):
    end_date = datetime.now().strftime("%Y-%m-%d")

    date_picker = widgets.DatePicker(description='Chọn ngày bắt đầu:', value=datetime(2023, 1, 1))
    symbol_input = widgets.Text(description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')
    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_full_report = widgets.Button(description="Báo cáo tổng hợp")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output_prediction:
            clear_output()
            custom_start_date = date_picker.value
            if custom_start_date is None:
                print("Vui lòng chọn ngày bắt đầu!")
                return
            custom_start_date = custom_start_date.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")

            income, balance, ratio, selected_years, source, industry = get_financial_data(symbol, 5)
            if income is None:
                print(f"Không thể lấy dữ liệu cho {symbol}")
                return

            num_years_picker = widgets.IntSlider(value=5, min=1, max=12, step=1, description='Số năm phân tích:')
            button_analyze_years = widgets.Button(description="Phân tích sức khỏe tài chính")

            def on_analyze_years_clicked(b):
                with output_financial:
                    clear_output()
                    num_years = num_years_picker.value
                    result = analyze_financial_health(symbol, num_years)
                    if result["Financial Metrics"] is not None:
                        print(f"\nPHÂN TÍCH SỨC KHỎE TÀI CHÍNH CHO MÃ {symbol} TRONG {num_years} NĂM GẦN NHẤT:")
                        print(f"Nguồn dữ liệu: {source}")
                        print(f"Các năm được phân tích: {selected_years}")
                        display(HTML(TABLE_STYLE + result["Financial Metrics"].to_html(index=False)))

                with output_dashboard:
                    clear_output()
                    if result["Financial Metrics"] is not None:
                        print("Hiển thị dashboard tài chính:")
                        # Thêm logic dashboard nếu cần

            def on_full_report_clicked(b):
                generate_comprehensive_report(symbol, output_report)

            with output_prediction:
                print("\nChọn số năm để phân tích sức khỏe tài chính hoặc xem báo cáo tổng hợp:")
                display(widgets.VBox([num_years_picker, button_analyze_years, button_full_report]))
            button_analyze_years.on_click(on_analyze_years_clicked)
            button_full_report.on_click(on_full_report_clicked)

    def on_yes_clicked(b):
        with output_prediction:
            clear_output()
        with output_financial:
            clear_output()
        with output_dashboard:
            clear_output()
        with output_report:
            clear_output()
        create_initial_interface(output_prediction, output_financial, output_dashboard, output_report)

    def on_no_clicked(b):
        with output_prediction:
            clear_output()
            print("Đã thoát chương trình.")

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    display(date_picker, symbol_input, button_confirm, output_prediction, output_financial, output_dashboard, output_report)

# Chạy chương trình
def run_prediction():
    output_prediction = widgets.Output()
    output_financial = widgets.Output()
    output_dashboard = widgets.Output()
    output_report = widgets.Output()
    create_initial_interface(output_prediction, output_financial, output_dashboard, output_report)

if __name__ == "__main__":
    run_prediction()

DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Chọn ngày bắt đầu:')

Text(value='', description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')

Button(description='Xác nhận và dự đoán', style=ButtonStyle())

Output()

Output()

Output()

Output()

Quét toàn sàn, đưa ra khuyến nghị

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
import logging
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import requests

warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")
logging.getLogger("vnstock").setLevel(logging.CRITICAL)
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.CRITICAL)

log_file = "/content/stock_forecast.log"
with open(log_file, 'w') as f:
    f.write("Bắt đầu ghi log...\n")
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s', filename=log_file, filemode='a')

error_summary = Counter()

# 1. Lấy danh sách mã cổ phiếu từ API FPTS
def get_stock_symbols_from_cafef():
    api_url = "https://ezir.fpts.com.vn/ThongTinDoanhNghiep/ThongTinCompanyViEn?culture=vi-VN"
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        df = df[df["aposT_TO"].isin(["HOSE", "HNX", "UPCOM"])]
        df = df[["astock_CODE", "aposT_TO"]]
        df.rename(columns={"astock_CODE": "Mã CK", "aposT_TO": "Sàn"}, inplace=True)
        symbols = df["Mã CK"].unique().tolist()
        print(f"Tổng số mã từ API FPTS: {len(symbols)}")
        df.to_csv("filtered_stock_data.csv", index=False)
        print("Dữ liệu đã được lưu vào filtered_stock_data.csv")
        return symbols
    except requests.exceptions.RequestException as e:
        logging.error(f"Lỗi khi gọi API: {e}")
        return []
    except Exception as e:
        logging.error(f"Lỗi khi xử lý dữ liệu từ API: {e}")
        return []

# 2. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 5 or not all(col in df.columns for col in required_columns):
            error_summary["Dữ liệu không đủ 5 ngày"] += 1
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        error_summary["Lỗi API từ vnstock"] += 1
        return None

# 3. Chỉ báo kỹ thuật (đã sửa cho xu hướng ngắn hạn 5-10 ngày)
def calculate_indicators(df, symbol="Unknown"):
    if df is None or df.empty or len(df) < 5:
        error_summary["Dữ liệu không đủ 5 ngày"] += 1
        return None
    try:
        required_columns = ["close", "high", "low", "volume"]
        if any(col not in df.columns or df[col].isna().any() for col in required_columns):
            error_summary["Thiếu cột hoặc chứa NaN"] += 1
            return None

        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty or df.isna().all().any():
            error_summary["Dữ liệu trống sau tính toán"] += 1
            return None
        return df
    except Exception as e:
        error_summary["Lỗi khác"] += 1
        return None

# 4. Alligator
def add_alligator(df, symbol="Unknown"):
    if df is None or df.empty or len(df) < 5:
        error_summary["Dữ liệu không đủ 5 ngày"] += 1
        return None
    try:
        if df["close"].isna().any():
            error_summary["Thiếu cột hoặc chứa NaN"] += 1
            return None
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty or df.isna().all().any():
            error_summary["Dữ liệu trống sau tính toán"] += 1
            return None
        return df
    except Exception as e:
        error_summary["Lỗi khác"] += 1
        return None

# 5. Chuẩn bị dữ liệu phân loại
def prepare_classification_data(df, symbol="Unknown"):
    if df is None or df.empty or df.isna().all().any():
        error_summary["Thiếu cột hoặc chứa NaN"] += 1
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty or len(df) < 20:
            error_summary["Dữ liệu trống hoặc quá ít sau loại NaN"] += 1
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        error_summary["Chỉ có một lớp dữ liệu hoặc lỗi khác"] += 1
        return None, None, None

# 6. Dự báo và backtesting (cho toàn bộ danh sách)
def forecast_and_backtest(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        df = calculate_indicators(df, symbol)
        if df is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        df = add_alligator(df, symbol)
        if df is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        if 'RSI_5' not in df.columns:
            error_summary["Lỗi khác"] += 1
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df, symbol)
        if X_train_5 is None or X_train_10 is None:
            return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                    "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

        models = {
            "Logistic Regression": LogisticRegression(max_iter=1000, C=0.5, random_state=42),
            "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=5, random_state=42),
            "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=4, subsample=0.8, colsample_bytree=0.8, random_state=42)
        }

        predictions_5 = {}
        predictions_10 = {}
        accuracy_5 = {}
        accuracy_10 = {}

        for name, model in models.items():
            if len(np.unique(y_train_5)) < 2:
                predictions_5[name] = np.unique(y_train_5)[0]
                accuracy_5[name] = 0
            else:
                model.fit(X_train_5, y_train_5)
                y_pred_5 = model.predict(X_test_5)
                latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
                predictions_5[name] = model.predict(latest_data)[0]
                accuracy_5[name] = accuracy_score(y_test_5, y_pred_5)

            if len(np.unique(y_train_10)) < 2:
                predictions_10[name] = np.unique(y_train_10)[0]
                accuracy_10[name] = 0
            else:
                model.fit(X_train_10, y_train_10)
                y_pred_10 = model.predict(X_test_10)
                latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
                predictions_10[name] = model.predict(latest_data)[0]
                accuracy_10[name] = accuracy_score(y_test_10, y_pred_10)

        buy_votes_5 = sum(1 for name, pred in predictions_5.items() if pred == 1 and accuracy_5[name] >= 0.6)
        buy_votes_10 = sum(1 for name, pred in predictions_10.items() if pred == 1 and accuracy_10[name] >= 0.6)
        total_valid_models_5 = sum(1 for acc in accuracy_5.values() if acc >= 0.6)
        total_valid_models_10 = sum(1 for acc in accuracy_10.values() if acc >= 0.6)

        final_rec = "N/A"
        if total_valid_models_5 > 0 or total_valid_models_10 > 0:
            final_rec = "MUA" if (buy_votes_5 >= 2 or buy_votes_10 >= 2) else "BÁN"

        avg_accuracy_5 = np.mean(list(accuracy_5.values())) if accuracy_5 else 0
        avg_accuracy_10 = np.mean(list(accuracy_10.values())) if accuracy_10 else 0

        rf_model = models["Random Forest"]
        rf_model.fit(X_train_5, y_train_5)
        feature_importance = pd.DataFrame({"Feature": feature_columns, "Importance": rf_model.feature_importances_})
        top_features = feature_importance.sort_values("Importance", ascending=False).head(5).to_dict()

        return {
            "Symbol": symbol,
            "Final Recommendation": final_rec,
            "Buy Votes 5 Days": buy_votes_5,
            "Buy Votes 10 Days": buy_votes_10,
            "Accuracy_5": avg_accuracy_5,
            "Accuracy_10": avg_accuracy_10,
            "Top_Features": top_features
        }
    except Exception as e:
        error_summary["Lỗi khác"] += 1
        return {"Symbol": symbol, "Final Recommendation": "N/A", "Buy Votes 5 Days": 0, "Buy Votes 10 Days": 0,
                "Accuracy_5": 0, "Accuracy_10": 0, "Top_Features": {}}

# 7. Xử lý song song cho toàn bộ danh sách
def process_all_symbols(symbols, days_window=120):
    end_date = datetime.now().strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=days_window)).strftime("%Y-%m-%d")
    print(f"Đang xử lý dự báo xu hướng 5-10 ngày với 120 ngày lịch sử từ {start_date} đến {end_date}...")

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda s: forecast_and_backtest(s, start_date, end_date), symbols))

    recommendations = [r for r in results if r]
    return pd.DataFrame(recommendations)

# 8. Lấy top 10 mã "MUA" hoặc "BÁN"
def get_top_recommendations(df, recommendation_type="MUA"):
    df["Total Buy Votes"] = df["Buy Votes 5 Days"] + df["Buy Votes 10 Days"]
    df["Avg Accuracy"] = (df["Accuracy_5"] + df["Accuracy_10"]) / 2

    filtered_df = df[df["Final Recommendation"] == recommendation_type].copy()

    if filtered_df.empty:
        print(f"Không có mã nào được khuyến nghị '{recommendation_type}'.")
        return pd.DataFrame()

    top_df = filtered_df.sort_values(by=["Total Buy Votes", "Avg Accuracy"], ascending=[False, False])
    return top_df.head(10)[["Symbol", "Final Recommendation", "Buy Votes 5 Days", "Buy Votes 10 Days",
                            "Accuracy_5", "Accuracy_10", "Avg Accuracy", "Total Buy Votes"]]

# 9. Dự báo và backtesting (cho mã cụ thể - đã sửa chỉ báo)
def forecast_and_backtest_specific(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators(df, symbol)
    if df is None:
        return None

    df = add_alligator(df, symbol)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df, symbol)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for pred in predictions_5.values() if pred == 1)
    buy_votes_10 = sum(1 for pred in predictions_10.values() if pred == 1)
    final_rec = "MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN"

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df
    }

# 10. So sánh kết quả với các khoảng thời gian
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}
    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest_specific(symbol, custom_start_date, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest_specific(symbol, start_120, end_date)

    periods = ["Toàn bộ từ ngày chọn", "120 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else 120

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.applymap(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 11. Giao diện chính với widget
def run_program():
    end_date = datetime.now().strftime("%Y-%m-%d")
    output = widgets.Output()

    symbols_list = get_stock_symbols_from_cafef()
    if not symbols_list:
        print("Không thể lấy danh sách mã cổ phiếu.")
        return

    print(f"Tổng số mã đưa vào phân tích: {len(symbols_list)}")
    recommendations_df = process_all_symbols(symbols_list, days_window=120)

    if not recommendations_df.empty:
        recommendations_df.to_csv("recommendations_120_days_ml_optimized_new_indicators.csv", index=False)
        print(f"Kết quả dự báo xu hướng 5-10 ngày với 120 ngày lịch sử đã được lưu vào recommendations_120_days_ml_optimized_new_indicators.csv")
        print("Kết quả khuyến nghị:")
        print(recommendations_df[["Symbol", "Final Recommendation", "Buy Votes 5 Days", "Buy Votes 10 Days",
                                 "Accuracy_5", "Accuracy_10", "Top_Features"]])

        successful = len(recommendations_df[recommendations_df["Final Recommendation"].isin(["MUA", "BÁN"])])
        unsuccessful = len(recommendations_df[recommendations_df["Final Recommendation"] == "N/A"])
        buy_count = len(recommendations_df[recommendations_df["Final Recommendation"] == "MUA"])
        total_analyzed = len(recommendations_df)

        print(f"\nTóm tắt kết quả:")
        print(f"Số mã phân tích thành công: {successful}/{len(symbols_list)}")
        print(f"Số mã không thành công: {unsuccessful}")
        print(f"Tỷ lệ khuyến nghị MUA: {buy_count/total_analyzed*100:.2f}%")

        top_buy = get_top_recommendations(recommendations_df, "MUA")
        top_sell = get_top_recommendations(recommendations_df, "BÁN")

        print("\nTop 10 mã khuyến nghị 'MUA' cao nhất:")
        if not top_buy.empty:
            print(top_buy)
        else:
            print("Không có mã nào.")

        print("\nTop 10 mã khuyến nghị 'BÁN' cao nhất:")
        if not top_sell.empty:
            print(top_sell)
        else:
            print("Không có mã nào.")

    date_picker = widgets.DatePicker(description='Chọn ngày bắt đầu:', value=datetime(2023, 1, 1))
    symbol_input = widgets.Text(description='Mã chứng khoán:', placeholder='Nhập mã (hoặc "exit" để thoát)')
    button_confirm = widgets.Button(description="Xác nhận và dự đoán")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output:
            clear_output()
            custom_start_date = date_picker.value.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            if symbol.lower() == 'exit':
                print("Đã thoát chương trình.")
                return

            print(f"Dữ liệu lịch sử được lấy từ {custom_start_date} đến {end_date}")
            compare_predictions(symbol, custom_start_date, end_date)
            display(widgets.HBox([button_continue_yes, button_continue_no]))

    def on_yes_clicked(b):
        with output:
            clear_output()
            display(date_picker, symbol_input, button_confirm, output)

    def on_no_clicked(b):
        with output:
            clear_output()
            print("Đã thoát chương trình.")

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    with output:
        print("\nPhân tích mã cụ thể sau khi hoàn tất phân tích toàn bộ danh sách:")
        display(date_picker, symbol_input, button_confirm, output)

    display(output)

# 12. Thực thi chương trình
if __name__ == "__main__":
    run_program()


    Khi tiếp tục sử dụng Vnstock3, bạn xác nhận rằng bạn đã đọc, hiểu và đồng ý với Chính sách quyền riêng tư và Điều khoản, điều kiện về giấy phép sử dụng Vnstock3.

    Chi tiết:

    - Giấy phép sử dụng phần mềm: https://vnstocks.com/docs/tai-lieu/giay-phep-su-dung
    - Chính sách quyền riêng tư: https://vnstocks.com/docs/tai-lieu/chinh-sach-quyen-rieng-tu
    
Tổng số mã từ API FPTS: 1683
Dữ liệu đã được lưu vào filtered_stock_data.csv
Tổng số mã đưa vào phân tích: 1683
Đang xử lý dự báo xu hướng 5-10 ngày với 120 ngày lịch sử từ 2024-11-13 đến 2025-03-13...
Kết quả dự báo xu hướng 5-10 ngày với 120 ngày lịch sử đã được lưu vào recommendations_120_days_ml_optimized_new_indicators.csv
Kết quả khuyến nghị:
     Symbol Final Recommendation  Buy Votes 5 Days  Buy Votes 10 Days  \
0       A32                  N/A                 0                  0   
1       AAA                  BÁN                 0                  0   
2       AAH                  BÁN                 1           

Output()

In [ ]:
Tập trung một mã cụ thể

In [ ]:
import pandas as pd
import numpy as np
import ta
from vnstock import Vnstock
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datetime import datetime, timedelta
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import logging
import matplotlib.pyplot as plt

# Tắt cảnh báo từ vnstock
logging.getLogger("vnstock").setLevel(logging.ERROR)
logging.getLogger("vnstock.common.data.data_explorer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="vnstock")

# Cấu hình renderer Plotly cho Colab
pio.renderers.default = "colab"

# CSS để định dạng bảng
TABLE_STYLE = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-size: 14px;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""

# Các hàm phân tích tài chính (giữ nguyên)
def initialize_stock(symbol, source='TCBS'):
    return Vnstock().stock(symbol=symbol, source=source)

def get_financial_data(symbol, years, preferred_source='TCBS'):
    sources = [preferred_source, 'VCI'] if preferred_source == 'TCBS' else ['VCI', 'TCBS']
    selected_source = None
    income, balance, ratio, selected_years = None, None, None, []

    for source in sources:
        stock = initialize_stock(symbol, source)
        try:
            income_statement = stock.finance.income_statement(period='year')
            balance_sheet = stock.finance.balance_sheet(period='year')
            ratio = stock.finance.ratio(period='year')

            if any(df.empty for df in [income_statement, balance_sheet, ratio]):
                continue

            if source == 'TCBS':
                income_years = income_statement.index.unique().tolist()
                balance_years = balance_sheet.index.unique().tolist()
                ratio_years = ratio.index.unique().tolist()
            else:
                year_col = next((col for col in ['Năm', 'yearReport'] if col in income_statement.columns), None)
                if year_col is None:
                    continue
                income_years = [y for y in income_statement[year_col].unique().tolist() if isinstance(y, (int, float)) and y >= 2000]
                balance_years = [y for y in balance_sheet[year_col].unique().tolist() if isinstance(y, (int, float)) and y >= 2000]
                ratio_years = [y for y in ratio[year_col].unique().tolist() if isinstance(y, (int, float)) and y >= 2000]

            all_years = sorted(set(income_years + balance_years + ratio_years))
            selected_years = sorted(all_years[-years:]) if len(all_years) >= years else all_years

            if source == 'TCBS':
                income = income_statement.loc[income_statement.index.isin(selected_years)].copy()
                balance = balance_sheet.loc[balance_sheet.index.isin(selected_years)].copy()
                ratio = ratio.loc[ratio.index.isin(selected_years)].copy()
            else:
                income = income_statement[income_statement[year_col].isin(selected_years)].copy()
                balance = balance_sheet[balance_sheet[year_col].isin(selected_years)].copy()
                ratio = ratio[ratio[year_col].isin(selected_years)].copy()
                if not income.empty and year_col in income.columns:
                    income.set_index(year_col, inplace=True)
                if not balance.empty and year_col in balance.columns:
                    balance.set_index(year_col, inplace=True)
                if not ratio.empty and year_col in ratio.columns:
                    ratio.set_index(year_col, inplace=True)

            if source == 'TCBS':
                if 'short_liability' not in balance.columns and 'payable' in balance.columns:
                    balance.loc[:, 'short_liability'] = balance['payable']
                if 'short_asset' not in balance.columns and 'asset' in balance.columns:
                    balance.loc[:, 'short_asset'] = balance['asset']
                if 'long_asset' not in balance.columns and 'asset' in balance.columns:
                    balance.loc[:, 'long_asset'] = balance['asset'] - balance.get('short_asset', balance['asset'])
            else:
                if 'short_asset' not in balance.columns and 'CURRENT ASSETS (Bn. VND)' in balance.columns:
                    balance.loc[:, 'short_asset'] = balance['CURRENT ASSETS (Bn. VND)'] / 1e9
                if 'short_liability' not in balance.columns and 'Short-term borrowings (Bn. VND)' in balance.columns:
                    balance.loc[:, 'short_liability'] = balance['Short-term borrowings (Bn. VND)'] / 1e9
                if 'long_asset' not in balance.columns and 'LONG-TERM ASSETS (Bn. VND)' in balance.columns:
                    balance.loc[:, 'long_asset'] = balance['LONG-TERM ASSETS (Bn. VND)'] / 1e9
                if 'equity' not in balance.columns and "OWNER'S EQUITY(Bn.VND)" in balance.columns:
                    balance.loc[:, 'equity'] = balance["OWNER'S EQUITY(Bn.VND)"] / 1e9

            selected_source = source
            break

        except Exception:
            continue

    if selected_source is None:
        return None, None, None, [], None

    return income, balance, ratio, selected_years, selected_source

def calculate_financial_metrics(income, balance, ratio, selected_years, source='TCBS'):
    result = pd.DataFrame(index=selected_years, columns=[
        'Year', 'Net Profit Margin', 'Debt to Equity', 'P/E Ratio',
        'Current Ratio', 'ROE', 'ROA', 'Interest Coverage'
    ])

    for year in selected_years:
        result.loc[year, 'Year'] = year
        try:
            if source == 'TCBS':
                revenue = income.loc[year, 'revenue'] if year in income.index and 'revenue' in income.columns else np.nan
                profit = income.loc[year, 'post_tax_profit'] if year in income.index and 'post_tax_profit' in income.columns else np.nan
                equity = balance.loc[year, 'equity'] if year in balance.index and 'equity' in balance.columns else np.nan
                debt = balance.loc[year, 'payable'] if year in balance.index and 'payable' in balance.columns else np.nan
                pe = ratio.loc[year, 'price_to_earning'] if year in ratio.index and 'price_to_earning' in ratio.columns else np.nan
                short_asset = balance.loc[year, 'short_asset'] if year in balance.index and 'short_asset' in balance.columns else np.nan
                short_liability = balance.loc[year, 'short_liability'] if year in balance.index and 'short_liability' in balance.columns else np.nan
                total_asset = (balance.loc[year, 'short_asset'] + balance.loc[year, 'long_asset']) if year in balance.index else np.nan
                ebitda = income.loc[year, 'ebitda'] if year in income.index and 'ebitda' in income.columns else np.nan
            else:
                revenue = income.loc[year, 'Revenue (Bn. VND)'] / 1e9 if year in income.index and 'Revenue (Bn. VND)' in income.columns else np.nan
                profit = income.loc[year, 'Net Profit For the Year'] / 1e9 if year in income.index and 'Net Profit For the Year' in income.columns else np.nan
                equity = balance.loc[year, "OWNER'S EQUITY(Bn.VND)"] / 1e9 if year in balance.index and "OWNER'S EQUITY(Bn.VND)" in balance.columns else np.nan
                debt = (balance.loc[year, 'TOTAL ASSETS (Bn. VND)'] / 1e9 - equity) if year in balance.index and 'TOTAL ASSETS (Bn. VND)' in balance.columns else np.nan
                pe = ratio.loc[year, 'P/E'] if year in ratio.index and 'P/E' in ratio.columns else np.nan
                short_asset = balance.loc[year, 'short_asset'] if year in balance.index and 'short_asset' in balance.columns else np.nan
                short_liability = balance.loc[year, 'short_liability'] if year in balance.index and 'short_liability' in balance.columns else np.nan
                total_asset = balance.loc[year, 'TOTAL ASSETS (Bn. VND)'] / 1e9 if year in balance.index and 'TOTAL ASSETS (Bn. VND)' in balance.columns else np.nan
                ebitda = np.nan

            result.loc[year, 'Net Profit Margin'] = (profit / revenue * 100) if revenue and profit else np.nan
            result.loc[year, 'Debt to Equity'] = (debt / equity) if equity and debt else np.nan
            result.loc[year, 'P/E Ratio'] = pe
            result.loc[year, 'Current Ratio'] = (short_asset / short_liability) if short_asset and short_liability else np.nan
            result.loc[year, 'ROE'] = (profit / equity * 100) if equity and profit else np.nan
            result.loc[year, 'ROA'] = (profit / total_asset * 100) if total_asset and profit else np.nan
            result.loc[year, 'Interest Coverage'] = (ebitda / (ebitda - profit)) if ebitda and profit and (ebitda != profit) else np.nan

        except Exception:
            result.loc[year, :] = [year] + [np.nan] * 7

    return result.round(2)

def get_financial_data_with_industry(symbol, years, preferred_source='TCBS'):
    income, balance, ratio, selected_years, selected_source = get_financial_data(symbol, years, preferred_source)
    if selected_source is None:
        return None, None, None, [], None, None

    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        company_info = company.overview()
        industry = company_info['industry'].iloc[0] if not company_info.empty and 'industry' in company_info.columns else "Khác"
    except Exception:
        industry = "Khác"

    return income, balance, ratio, selected_years, selected_source, industry

def calculate_financial_health_score(financial_metrics, industry):
    industry_thresholds = {
        "Bất động sản": {"Net Profit Margin": 10, "Debt to Equity": 2, "P/E Ratio": 20, "Current Ratio": 1, "ROE": 8, "ROA": 3, "Interest Coverage": 2},
        "Công nghệ": {"Net Profit Margin": 15, "Debt to Equity": 1, "P/E Ratio": 25, "Current Ratio": 1.5, "ROE": 15, "ROA": 10, "Interest Coverage": 5},
        "Ngân hàng": {"Net Profit Margin": 20, "Debt to Equity": 10, "P/E Ratio": 15, "Current Ratio": 1, "ROE": 12, "ROA": 1, "Interest Coverage": 3},
        "Khác": {"Net Profit Margin": 10, "Debt to Equity": 1, "P/E Ratio": 15, "Current Ratio": 1, "ROE": 10, "ROA": 5, "Interest Coverage": 3}
    }
    weights = {"Net Profit Margin": 2.0, "Debt to Equity": 1.5, "P/E Ratio": 1.0, "Current Ratio": 1.5, "ROE": 2.0, "ROA": 1.5, "Interest Coverage": 0.5}
    thresholds = industry_thresholds.get(industry, industry_thresholds["Khác"])

    total_score = 0
    comments = []
    detailed_assessment = []

    for metric in financial_metrics.columns[1:]:
        value = financial_metrics[metric].iloc[-1]
        threshold = thresholds[metric]
        weight = weights[metric]

        if pd.isna(value):
            score = 0
            comment = f"{metric}: Không có dữ liệu."
            detail = f"{metric}: Không có dữ liệu để đánh giá."
        else:
            if metric in ["Net Profit Margin", "ROE", "ROA"]:
                score = weight if value > threshold else 0
                comment = f"{metric} ({value:.2f}%): {'Đạt' if value > threshold else 'Không đạt'} ngưỡng (> {threshold}%), được {score} điểm."
                detail = f"{metric} ({value:.2f}%): {'Mức độ sinh lời vượt trội' if value > threshold else 'Mức độ sinh lời thấp hơn ngưỡng ngành, cần cải thiện'}."
            elif metric in ["Debt to Equity", "P/E Ratio"]:
                score = weight if value < threshold else 0
                comment = f"{metric} ({value:.2f}): {'Đạt' if value < threshold else 'Không đạt'} ngưỡng (< {threshold}), được {score} điểm."
                detail = f"{metric} ({value:.2f}): {'Mức độ rủi ro tài chính hoặc định giá hợp lý' if value < threshold else 'Mức độ rủi ro tài chính hoặc định giá cao, cần thận trọng'}."
            elif metric in ["Current Ratio", "Interest Coverage"]:
                score = weight if value > threshold else 0
                comment = f"{metric} ({value:.2f}): {'Đạt' if value > threshold else 'Không đạt'} ngưỡng (> {threshold}), được {score} điểm."
                detail = f"{metric} ({value:.2f}): {'Khả năng thanh khoản hoặc chi trả lãi tốt' if value > threshold else 'Khả năng thanh khoản hoặc chi trả lãi yếu, tiềm ẩn rủi ro'}."

        total_score += score
        comments.append(comment)
        detailed_assessment.append(detail)

    assessment = "Tích cực" if total_score >= 7 else "Trung lập" if total_score >= 4 else "Tiêu cực"
    overall_comment = f"**Sức khỏe tài chính**: Điểm tổng {total_score:.2f}/10.00, đánh giá '{assessment}' trong ngành {industry}."
    comments.insert(0, overall_comment)

    suggestions = []
    if total_score < 4:
        suggestions.append("Gợi ý: Cân nhắc giảm đầu tư hoặc theo dõi sát sao các chỉ số tài chính.")
    elif total_score < 7:
        suggestions.append("Gợi ý: Duy trì giám sát, cải thiện các chỉ số dưới ngưỡng để tăng sức khỏe tài chính.")
    else:
        suggestions.append("Gợi ý: Tiếp tục duy trì chiến lược hiện tại, tiềm năng đầu tư hoặc mở rộng.")

    return {
        "Financial Score": round(total_score, 2),
        "Assessment": assessment,
        "Comments": comments,
        "Detailed Assessment": "\n".join(detailed_assessment),
        "Suggestions": suggestions
    }

def analyze_financial_health(symbol, years):
    income, balance, ratio, selected_years, source, industry = get_financial_data_with_industry(symbol, years)
    if income is None:
        return {
            "Assessment": "N/A",
            "Financial Score": 0,
            "Detailed Assessment": "Không có dữ liệu để phân tích.",
            "Comments": ["Không có dữ liệu tài chính."],
            "Suggestions": ["Không có dữ liệu để đưa ra gợi ý."],
            "Financial Metrics": None
        }

    financial_metrics = calculate_financial_metrics(income, balance, ratio, selected_years, source)
    health_score = calculate_financial_health_score(financial_metrics, industry)

    return {
        "Financial Metrics": financial_metrics,
        "Health Score": health_score
    }

# 1. Lấy dữ liệu chứng khoán
def get_stock_data(symbol, start_date, end_date):
    try:
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        df = stock.quote.history(start=start_date, end=end_date)
        required_columns = ["close", "high", "low", "volume"]
        if df.empty or len(df) < 5 or not all(col in df.columns for col in required_columns):
            print(f"Dữ liệu không đủ cho {symbol} (ít hơn 5 ngày hoặc thiếu cột).")
            return None
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        df[required_columns] = df[required_columns].ffill().bfill()
        return df
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")
        return None

# 2. Chỉ báo kỹ thuật (đã sửa cho xu hướng ngắn hạn 5-10 ngày)
def calculate_indicators(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        indicators = {
            "SMA_5": ta.trend.sma_indicator(df["close"], window=5),
            "EMA_5": ta.trend.ema_indicator(df["close"], window=5),
            "MACD": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd(),
            "Signal": ta.trend.MACD(df["close"], window_fast=6, window_slow=13, window_sign=5).macd_signal(),
            "RSI_5": ta.momentum.rsi(df["close"], window=5),
            "Williams_%R": ta.momentum.williams_r(df["high"], df["low"], df["close"], lbp=5),
            "Stochastic": ta.momentum.stoch(df["high"], df["low"], df["close"], window=5),
            "Momentum_5": df["close"].diff(5),
            "ROC_5": ta.momentum.roc(df["close"], window=5),
            "OBV": ta.volume.on_balance_volume(df["close"], df["volume"]),
            "VROC": np.where(df["volume"].shift(5) == 0, 0, (df["volume"] - df["volume"].shift(5)) / df["volume"].shift(5) * 100),
            "AD_Line": ta.volume.acc_dist_index(df["high"], df["low"], df["close"], df["volume"]),
            "ATR": ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=5),
            "Bollinger_Upper": ta.volatility.bollinger_hband(df["close"], window=5),
            "Bollinger_Lower": ta.volatility.bollinger_lband(df["close"], window=5),
            "ADX": ta.trend.adx(df["high"], df["low"], df["close"], window=5)
        }
        df = df.assign(**indicators)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi tính chỉ báo: {e}")
        return None

# 3. Alligator
def add_alligator(df):
    if df is None or df.empty or len(df) < 5:
        return None
    try:
        df["Jaw"] = df["close"].rolling(5).mean().shift(3)
        df["Teeth"] = df["close"].rolling(4).mean().shift(2)
        df["Lips"] = df["close"].rolling(3).mean().shift(1)
        df.dropna(inplace=True)
        if df.empty:
            return None
        return df
    except Exception as e:
        print(f"Lỗi khi thêm Alligator: {e}")
        return None

# 4. Chuẩn bị dữ liệu phân loại
def prepare_classification_data(df):
    if df is None or df.empty:
        return None, None, None
    try:
        df["Future_Close_5"] = df["close"].shift(-5)
        df["Future_Close_10"] = df["close"].shift(-10)
        df["Target_5"] = (df["Future_Close_5"] > df["close"]).astype(int)
        df["Target_10"] = (df["Future_Close_10"] > df["close"]).astype(int)
        feature_columns = df.columns.difference(["Future_Close_5", "Future_Close_10", "Target_5", "Target_10"])

        df.dropna(inplace=True)
        if df.empty:
            return None, None, None

        X = df[feature_columns]
        y_5 = df["Target_5"]
        y_10 = df["Target_10"]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns, index=df.index)
        split_5 = train_test_split(X_scaled, y_5, test_size=0.2, random_state=42)
        split_10 = train_test_split(X_scaled, y_10, test_size=0.2, random_state=42)
        return split_5, split_10, feature_columns
    except Exception as e:
        print(f"Lỗi khi chuẩn bị dữ liệu: {e}")
        return None, None, None

# 5. Tính lợi nhuận kỳ vọng
def calculate_expected_profit(df, buy_votes_5, buy_votes_10):
    if df is None or df.empty or len(df) < 10:
        return {"Expected Profit 5 Days (%)": "N/A", "Expected Profit 10 Days (%)": "N/A"}

    df["Price_Change_5"] = df["close"].pct_change(periods=5) * 100
    df["Price_Change_10"] = df["close"].pct_change(periods=10) * 100
    avg_profit_5 = df[df["Price_Change_5"] > 0]["Price_Change_5"].mean()
    avg_profit_10 = df[df["Price_Change_10"] > 0]["Price_Change_10"].mean()

    prob_5 = buy_votes_5 / 3
    prob_10 = buy_votes_10 / 3

    expected_profit_5 = avg_profit_5 * prob_5 if not pd.isna(avg_profit_5) else 0
    expected_profit_10 = avg_profit_10 * prob_10 if not pd.isna(avg_profit_10) else 0

    return {
        "Expected Profit 5 Days (%)": round(expected_profit_5, 2) if expected_profit_5 > 0 else "N/A",
        "Expected Profit 10 Days (%)": round(expected_profit_10, 2) if expected_profit_10 > 0 else "N/A"
    }

# 6. Tính rủi ro
def calculate_risk(df):
    if df is None or df.empty or len(df) < 10:
        return {"Risk 5 Days (%)": "N/A", "Risk 10 Days (%)": "N/A"}

    df["Price_Change_5"] = df["close"].pct_change(periods=5) * 100
    df["Price_Change_10"] = df["close"].pct_change(periods=10) * 100
    risk_5 = df["Price_Change_5"].std()
    risk_10 = df["Price_Change_10"].std()

    return {
        "Risk 5 Days (%)": round(risk_5, 2) if not pd.isna(risk_5) else "N/A",
        "Risk 10 Days (%)": round(risk_10, 2) if not pd.isna(risk_10) else "N/A"
    }

# 7. Dự báo và backtesting (bổ sung rủi ro)
def forecast_and_backtest(symbol, start_date, end_date):
    df = get_stock_data(symbol, start_date, end_date)
    if df is None:
        return None

    df = calculate_indicators(df)
    if df is None:
        return None

    df = add_alligator(df)
    if df is None:
        return None

    (X_train_5, X_test_5, y_train_5, y_test_5), (X_train_10, X_test_10, y_train_10, y_test_10), feature_columns = prepare_classification_data(df)
    if X_train_5 is None or X_train_10 is None:
        return None

    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
        "XGBoost": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=6, random_state=42)
    }

    model_scores_5 = {}
    model_scores_10 = {}
    predictions_5 = {}
    predictions_10 = {}

    for name, model in models.items():
        if len(np.unique(y_train_5)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 5 ngày. Bỏ qua {name}.")
            model_scores_5[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_5[name] = 1 if np.unique(y_train_5)[0] == 1 else 0
        else:
            model.fit(X_train_5, y_train_5)
            y_pred_5 = model.predict(X_test_5)
            model_scores_5[name] = {
                "Accuracy": accuracy_score(y_test_5, y_pred_5),
                "Precision": precision_score(y_test_5, y_pred_5, zero_division=0),
                "Recall": recall_score(y_test_5, y_pred_5, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_5[name] = model.predict(latest_data)[0]

        if len(np.unique(y_train_10)) < 2:
            print(f"Cảnh báo: Dữ liệu cho {symbol} ({start_date} - {end_date}) chỉ có một lớp cho 10 ngày. Bỏ qua {name}.")
            model_scores_10[name] = {"Accuracy": 0, "Precision": 0, "Recall": 0}
            predictions_10[name] = 1 if np.unique(y_train_10)[0] == 1 else 0
        else:
            model.fit(X_train_10, y_train_10)
            y_pred_10 = model.predict(X_test_10)
            model_scores_10[name] = {
                "Accuracy": accuracy_score(y_test_10, y_pred_10),
                "Precision": precision_score(y_test_10, y_pred_10, zero_division=0),
                "Recall": recall_score(y_test_10, y_pred_10, zero_division=0)
            }
            latest_data = pd.DataFrame([df.loc[df.index[-1], sorted(feature_columns)]], columns=sorted(feature_columns))
            predictions_10[name] = model.predict(latest_data)[0]

    buy_votes_5 = sum(1 for name, pred in predictions_5.items() if pred == 1 and model_scores_5[name]["Accuracy"] >= 0.55)
    buy_votes_10 = sum(1 for name, pred in predictions_10.items() if pred == 1 and model_scores_10[name]["Accuracy"] >= 0.55)
    valid_models = sum(1 for scores in model_scores_5.values() if scores["Accuracy"] >= 0.55) + \
                   sum(1 for scores in model_scores_10.values() if scores["Accuracy"] >= 0.55)

    final_rec = "TRUNG LẬP" if valid_models == 0 else ("MUA" if buy_votes_5 >= 2 or buy_votes_10 >= 2 else "BÁN")
    avg_accuracy = np.mean([scores["Accuracy"] for scores in list(model_scores_5.values()) + list(model_scores_10.values())
                           if scores["Accuracy"] >= 0.55]) if valid_models > 0 else 0

    actual_trend_5 = actual_trend_10 = None
    if len(df) >= 10:
        actual_close_5 = df["close"].iloc[-6]
        actual_trend_5 = 1 if df["close"].iloc[-1] > actual_close_5 else 0
        actual_close_10 = df["close"].iloc[-11]
        actual_trend_10 = 1 if df["close"].iloc[-1] > actual_close_10 else 0

    # Tính lợi nhuận kỳ vọng và rủi ro
    expected_profit = calculate_expected_profit(df, buy_votes_5, buy_votes_10)
    risk = calculate_risk(df)

    return {
        "Symbol": symbol,
        "Final Recommendation": final_rec,
        "Buy Votes 5 Days": buy_votes_5,
        "Buy Votes 10 Days": buy_votes_10,
        "Model Scores 5 Days": model_scores_5,
        "Model Scores 10 Days": model_scores_10,
        "Actual Trend 5 Days": actual_trend_5,
        "Actual Trend 10 Days": actual_trend_10,
        "Predictions 5 Days": predictions_5,
        "Predictions 10 Days": predictions_10,
        "Dataframe": df,
        "Average Accuracy": round(avg_accuracy, 2),
        "Expected Profit": expected_profit,
        "Risk": risk
    }

# 8. So sánh kết quả với "Toàn bộ từ ngày chọn" và "120 ngày" (bổ sung rủi ro)
def compare_predictions(symbol, custom_start_date, end_date):
    results = {}

    start_dt = datetime.strptime(custom_start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    full_days = (end_dt - start_dt).days

    print(f"Đang phân tích với toàn bộ dữ liệu từ {custom_start_date} đến {end_date}...")
    results["Toàn bộ từ ngày chọn"] = forecast_and_backtest(symbol, custom_start_date, end_date)

    start_120 = (end_dt - timedelta(days=120)).strftime("%Y-%m-%d")
    print(f"Đang phân tích với 120 ngày từ {start_120} đến {end_date}...")
    results["120 ngày"] = forecast_and_backtest(symbol, start_120, end_date)

    periods = ["Toàn bộ từ ngày chọn", "120 ngày"]
    data = {
        "Khoảng thời gian": [],
        "Số ngày": [],
        "Khuyến nghị": [],
        "Phiếu MUA (5 ngày)": [],
        "Phiếu MUA (10 ngày)": [],
        "Accuracy (5 ngày)": [],
        "Accuracy (10 ngày)": [],
        "Average Accuracy": [],
        "Expected Profit 5 Days (%)": [],
        "Expected Profit 10 Days (%)": [],
        "Risk 5 Days (%)": [],
        "Risk 10 Days (%)": [],
        "Xu hướng thực tế (5 ngày)": [],
        "Xu hướng thực tế (10 ngày)": [],
        "Cảnh báo": []
    }

    for period in periods:
        result = results[period]
        days_used = full_days if period == "Toàn bộ từ ngày chọn" else 120

        if result is None:
            data["Khoảng thời gian"].append(period)
            data["Số ngày"].append(days_used)
            data["Khuyến nghị"].append("N/A")
            data["Phiếu MUA (5 ngày)"].append("N/A")
            data["Phiếu MUA (10 ngày)"].append("N/A")
            data["Accuracy (5 ngày)"].append("N/A")
            data["Accuracy (10 ngày)"].append("N/A")
            data["Average Accuracy"].append("N/A")
            data["Expected Profit 5 Days (%)"].append("N/A")
            data["Expected Profit 10 Days (%)"].append("N/A")
            data["Risk 5 Days (%)"].append("N/A")
            data["Risk 10 Days (%)"].append("N/A")
            data["Xu hướng thực tế (5 ngày)"].append("N/A")
            data["Xu hướng thực tế (10 ngày)"].append("N/A")
            data["Cảnh báo"].append("Không đủ dữ liệu")
            continue

        df = result["Dataframe"]
        data["Khoảng thời gian"].append(period)
        data["Số ngày"].append(days_used)
        data["Khuyến nghị"].append(result["Final Recommendation"])
        data["Phiếu MUA (5 ngày)"].append(f"{result['Buy Votes 5 Days']}/3")
        data["Phiếu MUA (10 ngày)"].append(f"{result['Buy Votes 10 Days']}/3")

        acc_5 = np.mean([scores["Accuracy"] for scores in result["Model Scores 5 Days"].values()])
        acc_10 = np.mean([scores["Accuracy"] for scores in result["Model Scores 10 Days"].values()])
        data["Accuracy (5 ngày)"].append(f"{acc_5:.2f}")
        data["Accuracy (10 ngày)"].append(f"{acc_10:.2f}")
        data["Average Accuracy"].append(f"{result['Average Accuracy']:.2f}")

        data["Expected Profit 5 Days (%)"].append(result["Expected Profit"]["Expected Profit 5 Days (%)"])
        data["Expected Profit 10 Days (%)"].append(result["Expected Profit"]["Expected Profit 10 Days (%)"])
        data["Risk 5 Days (%)"].append(result["Risk"]["Risk 5 Days (%)"])
        data["Risk 10 Days (%)"].append(result["Risk"]["Risk 10 Days (%)"])

        data["Xu hướng thực tế (5 ngày)"].append("Tăng" if result["Actual Trend 5 Days"] == 1 else "Giảm" if result["Actual Trend 5 Days"] == 0 else "N/A")
        data["Xu hướng thực tế (10 ngày)"].append("Tăng" if result["Actual Trend 10 Days"] == 1 else "Giảm" if result["Actual Trend 10 Days"] == 0 else "N/A")

        warning = "Có" if result["Final Recommendation"] == "BÁN" and (result["Actual Trend 5 Days"] == 1 or result["Actual Trend 10 Days"] == 1) else "Không"
        data["Cảnh báo"].append(warning)

    df_results = pd.DataFrame(data)

    def highlight_warning(val):
        color = 'red' if val == "Có" else 'black'
        return f'color: {color}'

    styled_df = df_results.style.applymap(highlight_warning, subset=["Cảnh báo"])
    print(f"\nSO SÁNH KẾT QUẢ DỰ ĐOÁN CHO MÃ {symbol}")
    display(HTML(styled_df.to_html(index=False)))
    return df_results

# 9. Phân tích cổ đông lớn
def analyze_major_shareholders(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        shareholders = company.shareholders()
        if shareholders.empty or 'share_own_percent' not in shareholders.columns:
            print(f"Dữ liệu cổ đông lớn cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_major_share = shareholders['share_own_percent'].sum() * 100
        top_shareholder = shareholders['share_own_percent'].iloc[0] * 100

        if total_major_share > 50:
            assessment = "Cao - Rủi ro quản trị cao, nhưng có thể tăng giá nếu cổ đông lớn mua thêm."
        elif total_major_share > 30:
            assessment = "Trung bình - Ảnh hưởng đáng kể từ cổ đông lớn."
        else:
            assessment = "Thấp - Quyền kiểm soát phân tán, ít rủi ro quản trị."

        return {
            "Total Major Share (%)": round(total_major_share, 2),
            "Top Shareholder (%)": round(top_shareholder, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích cổ đông lớn cho {symbol}: {e}")
        return {"Total Major Share (%)": "N/A", "Top Shareholder (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 10. Phân tích mức độ gắn bó của lãnh đạo
def analyze_management_commitment(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        officers = company.officers()
        if officers.empty or 'ownPercent' not in officers.columns:
            print(f"Dữ liệu lãnh đạo cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        total_management_share = officers['ownPercent'].sum()

        if total_management_share > 5:
            assessment = "Cao - Lãnh đạo có cam kết mạnh mẽ với công ty."
        elif total_management_share > 1:
            assessment = "Trung bình - Cam kết ở mức khá."
        else:
            assessment = "Thấp - Lãnh đạo ít gắn bó về mặt sở hữu."

        return {
            "Management Ownership (%)": round(total_management_share, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích mức độ gắn bó của lãnh đạo cho {symbol}: {e}")
        return {"Management Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 11. Phân tích công ty con
def analyze_subsidiaries(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        subsidiaries = company.subsidiaries()
        if subsidiaries.empty or 'sub_own_percent' not in subsidiaries.columns:
            print(f"Dữ liệu công ty con cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

        num_subsidiaries = len(subsidiaries)
        avg_ownership = subsidiaries['sub_own_percent'].mean() * 100

        if num_subsidiaries > 5 and avg_ownership > 50:
            assessment = "Đa dạng hóa tốt - Tiềm năng lợi nhuận từ công ty con cao."
        elif num_subsidiaries > 2:
            assessment = "Đa dạng hóa trung bình - Tiềm năng lợi nhuận khá."
        else:
            assessment = "Đa dạng hóa thấp - Ít phụ thuộc vào công ty con."

        return {
            "Number of Subsidiaries": num_subsidiaries,
            "Average Ownership (%)": round(avg_ownership, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích công ty con cho {symbol}: {e}")
        return {"Number of Subsidiaries": "N/A", "Average Ownership (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 12. Phát hiện tín hiệu nội bộ
def detect_insider_signals(symbol, days=30):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        insider_deals = company.insider_deals()
        if insider_deals.empty or 'deal_quantity' not in insider_deals.columns:
            print(f"Dữ liệu giao dịch nội bộ cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

        insider_deals['deal_announce_date'] = pd.to_datetime(insider_deals['deal_announce_date'])
        recent_deals = insider_deals[insider_deals['deal_announce_date'] > datetime.now() - timedelta(days=days)]

        if recent_deals.empty:
            return {"Net Insider Volume": 0, "Assessment": "Trung lập - Không có giao dịch gần đây."}

        net_volume = recent_deals['deal_quantity'].sum()

        if net_volume > 0:
            assessment = "Tích cực - Lãnh đạo mua ròng, dấu hiệu tốt cho triển vọng."
        elif net_volume < 0:
            assessment = "Tiêu cực - Lãnh đạo bán ròng, cần thận trọng."
        else:
            assessment = "Trung lập - Không có tín hiệu rõ ràng."

        return {
            "Net Insider Volume": int(net_volume),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phát hiện tín hiệu nội bộ cho {symbol}: {e}")
        return {"Net Insider Volume": "N/A", "Assessment": "Không có dữ liệu"}

# 13. Dự đoán biến động giá quanh sự kiện
def predict_event_impact(symbol, days_before=5, days_after=5):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        events = company.events()
        if events.empty or 'exer_date' not in events.columns:
            print(f"Dữ liệu sự kiện cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

        events['exer_date'] = pd.to_datetime(events['exer_date'])
        recent_event = events[events['exer_date'] <= datetime.now()].iloc[-1]
        event_date = recent_event['exer_date']
        event_desc = recent_event['event_desc']

        start_date = (event_date - timedelta(days=days_before)).strftime("%Y-%m-%d")
        end_date = (event_date + timedelta(days=days_after)).strftime("%Y-%m-%d")
        price_data = get_stock_data(symbol, start_date, end_date)

        if price_data is None or len(price_data) < days_before + days_after:
            print(f"Không đủ dữ liệu giá quanh sự kiện cho {symbol}")
            return {"Event Date": event_date.strftime("%Y-%m-%d"), "Event Description": event_desc, "Price Change (%)": "N/A", "Assessment": "Không đủ dữ liệu giá"}

        pre_event_price = price_data['close'].iloc[days_before - 1]
        post_event_price = price_data['close'].iloc[-1]
        price_change = ((post_event_price - pre_event_price) / pre_event_price) * 100

        assessment = "Tăng giá" if price_change > 0 else "Giảm giá"

        return {
            "Event Date": event_date.strftime("%Y-%m-%d"),
            "Event Description": event_desc,
            "Price Change (%)": round(price_change, 2),
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi dự đoán biến động giá quanh sự kiện cho {symbol}: {e}")
        return {"Event Date": "N/A", "Event Description": "N/A", "Price Change (%)": "N/A", "Assessment": "Không có dữ liệu"}

# 14. Cập nhật tin tức nóng
def get_latest_news(symbol):
    try:
        company = Vnstock().stock(symbol=symbol, source="TCBS").company
        news = company.news()
        if news.empty or 'publish_date' not in news.columns:
            print(f"Dữ liệu tin tức cho {symbol} chưa được cập nhật từ TCBS.")
            return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

        news['publish_date'] = pd.to_datetime(news['publish_date'])
        latest_news = news.iloc[0]

        return {
            "Date": latest_news['publish_date'].strftime("%Y-%m-%d"),
            "Title": latest_news['title'],
            "Source": latest_news['source']
        }
    except Exception as e:
        print(f"Lỗi khi lấy tin tức cho {symbol}: {e}")
        return {"Date": "N/A", "Title": "N/A", "Source": "N/A"}

# 15. Phân tích dòng tiền và cảnh báo bất thường
def analyze_cash_flow(symbol, start_date, end_date):
    try:
        df = get_stock_data(symbol, start_date, end_date)
        if df is None:
            return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

        avg_volume = df['volume'].mean()
        latest_volume = df['volume'].iloc[-1]
        volume_spike = latest_volume > avg_volume * 2
        volume_warning = latest_volume > avg_volume * 1.5

        if volume_spike:
            assessment = "Có - Khối lượng tăng đột biến, cần chú ý dòng tiền."
        elif volume_warning:
            assessment = "Cảnh báo nhẹ - Khối lượng tăng đáng kể so với trung bình."
        else:
            assessment = "Không - Dòng tiền ổn định."

        return {
            "Data Range": f"{start_date} đến {end_date}",
            "Average Volume": round(avg_volume, 0),
            "Latest Volume": round(latest_volume, 0),
            "Volume Spike": volume_spike,
            "Assessment": assessment
        }
    except Exception as e:
        print(f"Lỗi khi phân tích dòng tiền cho {symbol}: {e}")
        return {"Data Range": "N/A", "Average Volume": "N/A", "Latest Volume": "N/A", "Volume Spike": "N/A", "Assessment": "Không có dữ liệu"}

# Hàm tạo dashboard tài chính với Matplotlib
def create_financial_dashboard(financial_results, symbol):
    if not financial_results:
        print("Không có dữ liệu để tạo dashboard.")
        return

    df = pd.DataFrame(financial_results)
    years = df["Year"].astype(str)

    for col in ["Net Profit Margin", "Debt to Equity", "P/E Ratio", "Current Ratio", "ROE", "ROA", "Interest Coverage"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    fig, axes = plt.subplots(4, 2, figsize=(12, 16))
    fig.suptitle(f"Dashboard Tài Chính cho mã {symbol} qua các năm", fontsize=16)

    axes[0, 0].plot(years, df["Net Profit Margin"], marker="o", linestyle="-", color="b")
    axes[0, 0].set_title("Net Profit Margin (%)")
    axes[0, 0].set_ylabel("%")

    axes[0, 1].plot(years, df["Debt to Equity"], marker="o", linestyle="-", color="g")
    axes[0, 1].set_title("Debt to Equity")
    axes[0, 1].set_ylabel("Tỷ lệ")

    axes[1, 0].plot(years, df["P/E Ratio"], marker="o", linestyle="-", color="r")
    axes[1, 0].set_title("P/E Ratio")
    axes[1, 0].set_ylabel("Lần")

    axes[1, 1].plot(years, df["Current Ratio"], marker="o", linestyle="-", color="c")
    axes[1, 1].set_title("Current Ratio")
    axes[1, 1].set_ylabel("Tỷ lệ")

    axes[2, 0].plot(years, df["ROE"], marker="o", linestyle="-", color="m")
    axes[2, 0].set_title("ROE (%)")
    axes[2, 0].set_ylabel("%")

    axes[2, 1].plot(years, df["ROA"], marker="o", linestyle="-", color="y")
    axes[2, 1].set_title("ROA (%)")
    axes[2, 1].set_ylabel("%")

    axes[3, 0].plot(years, df["Interest Coverage"], marker="o", linestyle="-", color="k")
    axes[3, 0].set_title("Interest Coverage")
    axes[3, 0].set_ylabel("Lần")

    axes[3, 1].axis("off")

    for ax in axes.flat:
        ax.grid(True, linestyle="--", alpha=0.7)
        ax.set_xlabel("Năm")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Hàm tạo biểu đồ kỹ thuật với Plotly
def create_technical_chart(df, symbol):
    if df is None or df.empty:
        print("Không có dữ liệu để tạo biểu đồ kỹ thuật.")
        return None

    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=("Giá đóng cửa & Bollinger Bands", "RSI", "MACD", "Khối lượng"),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    fig.add_trace(go.Scatter(x=df.index, y=df["close"], name="Giá đóng cửa", line=dict(color="blue")), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["Bollinger_Upper"], name="Bollinger Upper", line=dict(color="orange", dash="dash")), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["Bollinger_Lower"], name="Bollinger Lower", line=dict(color="orange", dash="dash")), row=1, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df["RSI_5"], name="RSI", line=dict(color="purple")), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df["MACD"], name="MACD", line=dict(color="blue")), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["Signal"], name="Signal", line=dict(color="orange")), row=3, col=1)

    fig.add_trace(go.Bar(x=df.index, y=df["volume"], name="Khối lượng", marker_color="gray"), row=4, col=1)

    fig.update_layout(
        title=f"Biểu đồ Kỹ thuật cho {symbol}",
        height=800,
        showlegend=True,
        xaxis4_title="Ngày",
        yaxis1_title="Giá (VND)",
        yaxis2_title="RSI",
        yaxis3_title="MACD",
        yaxis4_title="Khối lượng"
    )
    fig.update_xaxes(rangeslider_visible=False)
    return fig

# Giao diện chính với widget
def main_interface():
    end_date = datetime.now().strftime("%Y-%m-%d")
    output = widgets.Output()

    date_picker = widgets.DatePicker(description="Ngày bắt đầu:", value=datetime.now() - timedelta(days=365))
    symbol_input = widgets.Text(description="Mã CK:", placeholder="Nhập mã chứng khoán")
    button_confirm = widgets.Button(description="Xác nhận")
    button_continue_yes = widgets.Button(description="Tiếp tục (Yes)")
    button_continue_no = widgets.Button(description="Thoát (No)")

    def on_confirm_clicked(b):
        with output:
            clear_output()
            custom_start_date = date_picker.value
            if custom_start_date is None:
                print("Vui lòng chọn ngày bắt đầu!")
                return
            custom_start_date = custom_start_date.strftime("%Y-%m-%d")
            symbol = symbol_input.value.strip().upper()

            if custom_start_date > end_date:
                print("Ngày bắt đầu phải trước hoặc bằng ngày hiện tại!")
                return

            if not symbol:
                print("Vui lòng nhập mã chứng khoán!")
                return

            print(f"\n=== PHÂN TÍCH MÃ {symbol} ===")
            print(f"Dữ liệu lịch sử từ {custom_start_date} đến {end_date}")

            # Phân tích tài chính
            print("\n1. PHÂN TÍCH SỨC KHỎE TÀI CHÍNH")
            financial_analysis = analyze_financial_health(symbol, years=5)
            if financial_analysis["Financial Metrics"] is not None:
                financial_df = financial_analysis["Financial Metrics"]
                display(HTML(TABLE_STYLE + financial_df.to_html(index=False)))
                create_financial_dashboard(financial_df, symbol)
                print("\nĐÁNH GIÁ SỨC KHỎE TÀI CHÍNH:")
                for comment in financial_analysis["Health Score"]["Comments"]:
                    print(comment)
                print(f"\nĐánh giá chi tiết:\n{financial_analysis['Health Score']['Detailed Assessment']}")
                print("\nGợi ý:")
                for suggestion in financial_analysis["Health Score"]["Suggestions"]:
                    print(suggestion)
            else:
                print("Không có dữ liệu tài chính để phân tích.")

            # Dự báo xu hướng kỹ thuật và lợi nhuận kỳ vọng + rủi ro
            print("\n2. DỰ BÁO XU HƯỚNG KỸ THUẬT 5-10 NGÀY")
            prediction_results = compare_predictions(symbol, custom_start_date, end_date)
            if not prediction_results.empty:
                df_120 = forecast_and_backtest(symbol, (datetime.now() - timedelta(days=120)).strftime("%Y-%m-%d"), end_date)
                if df_120 and df_120["Dataframe"] is not None:
                    technical_fig = create_technical_chart(df_120["Dataframe"], symbol)
                    if technical_fig:
                        technical_fig.show()

            # Phân tích cổ đông lớn
            print("\n3. PHÂN TÍCH CỔ ĐÔNG LỚN")
            shareholder_analysis = analyze_major_shareholders(symbol)
            print(f"Tổng tỷ lệ sở hữu cổ đông lớn: {shareholder_analysis['Total Major Share (%)']}%")
            print(f"Cổ đông lớn nhất: {shareholder_analysis['Top Shareholder (%)']}%")
            print(f"Đánh giá: {shareholder_analysis['Assessment']}")

            # Phân tích mức độ gắn bó của lãnh đạo
            print("\n4. PHÂN TÍCH MỨC ĐỘ GẮN BÓ CỦA LÃNH ĐẠO")
            management_analysis = analyze_management_commitment(symbol)
            print(f"Tỷ lệ sở hữu của lãnh đạo: {management_analysis['Management Ownership (%)']}%")
            print(f"Đánh giá: {management_analysis['Assessment']}")

            # Phân tích công ty con
            print("\n5. PHÂN TÍCH CÔNG TY CON")
            subsidiary_analysis = analyze_subsidiaries(symbol)
            print(f"Số lượng công ty con: {subsidiary_analysis['Number of Subsidiaries']}")
            print(f"Tỷ lệ sở hữu trung bình: {subsidiary_analysis['Average Ownership (%)']}%")
            print(f"Đánh giá: {subsidiary_analysis['Assessment']}")

            # Phát hiện tín hiệu nội bộ
            print("\n6. PHÁT HIỆN TÍN HIỆU NỘI BỘ (30 ngày gần nhất)")
            insider_signals = detect_insider_signals(symbol, days=30)
            print(f"Khối lượng giao dịch nội bộ ròng: {insider_signals['Net Insider Volume']}")
            print(f"Đánh giá: {insider_signals['Assessment']}")

            # Dự đoán biến động giá quanh sự kiện
            print("\n7. DỰ ĐOÁN BIẾN ĐỘNG GIÁ QUANH SỰ KIỆN GẦN NHẤT")
            event_impact = predict_event_impact(symbol)
            print(f"Ngày sự kiện: {event_impact['Event Date']}")
            print(f"Mô tả sự kiện: {event_impact['Event Description']}")
            print(f"Biến động giá: {event_impact['Price Change (%)']}%")
            print(f"Đánh giá: {event_impact['Assessment']}")

            # Cập nhật tin tức nóng
            print("\n8. TIN TỨC NÓNG")
            news = get_latest_news(symbol)
            print(f"Ngày: {news['Date']}")
            print(f"Tiêu đề: {news['Title']}")
            print(f"Nguồn: {news['Source']}")

            # Phân tích dòng tiền
            print("\n9. PHÂN TÍCH DÒNG TIỀN")
            cash_flow = analyze_cash_flow(symbol, custom_start_date, end_date)
            print(f"Phạm vi dữ liệu: {cash_flow['Data Range']}")
            print(f"Khối lượng trung bình: {cash_flow['Average Volume']}")
            print(f"Khối lượng mới nhất: {cash_flow['Latest Volume']}")
            print(f"Tăng đột biến: {'Có' if cash_flow['Volume Spike'] else 'Không'}")
            print(f"Đánh giá: {cash_flow['Assessment']}")

            display(widgets.HBox([button_continue_yes, button_continue_no]))

    def on_yes_clicked(b):
        with output:
            clear_output()
            display(date_picker, symbol_input, button_confirm, output)

    def on_no_clicked(b):
        with output:
            clear_output()
            print("Đã thoát chương trình.")

    button_confirm.on_click(on_confirm_clicked)
    button_continue_yes.on_click(on_yes_clicked)
    button_continue_no.on_click(on_no_clicked)

    with output:
        print("PHÂN TÍCH CHỨNG KHOÁN TOÀN DIỆN")
        display(date_picker, symbol_input, button_confirm, output)

    display(output)

# Thực thi chương trình
if __name__ == "__main__":
    main_interface()

ModuleNotFoundError: No module named 'ta'